# Imported libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, time #timedelta, timezone not in use
import os                                                   # for operations within directory
import re                                                   # for operations within directory
from dateutil import parser
import time     # for calculating duration of code¨
#import math # for basic mathematic operations such as math.ceil()

from scipy.integrate import trapezoid # for integrating response spectra 

from scipy.interpolate import RegularGridInterpolator # for interpolating wave spectrum from wave direction from north to wave direction rel to vessel heading

from geopy.distance import geodesic # for calculating distances between points 

from misc_func import * # Raphael's misc func for converting between absolute and encounter frequency

from scipy import signal #welch and csd for FFT

from scipy.signal import butter,filtfilt, freqz, welch # low-pass and high-pass filters as well as welch for FFT

from geneticalgorithm import geneticalgorithm as ga # the Genetic Algorithm which is used for solving the optimization problem

from scipy.optimize import minimize, differential_evolution

import pygad

import random # for generating initial population in GA with a fixed seed for reproducable results
# 
from icecream import ic
# gowtham interpolate
import xarray as xr

# Helper functions

## Datasets


In [ ]:
def getData(filename):
    filename = 'data/' + filename
    df = pd.read_csv(filename)
    return df

def cleanRAO(df):
    # Ensure the DataFrame is copied before modification to prevent affecting the original data
    df = df.copy()
    # Select only the required columns
    columns_needed = ['period', 'heaveamp', 'rollamp', 'pitchamp']
    df = df[columns_needed]

    # Remove the first row
    df = df.iloc[2:]  # This skips the 2 first rows as the period equals 0 on the first and there is only values of zero in amplitude in the second

    # Calculate the frequency from period and add it as a new column using .loc to avoid SettingWithCopyWarning
    df.loc[:, 'freq'] = 1 / df['period']

    df['rollamp'] = np.deg2rad(df['rollamp'])
    df['pitchamp'] = np.deg2rad(df['pitchamp'])

    # Rename columns to be more descriptive
    rename_map = {
        'heaveamp': 'Heave',
        'rollamp': 'Roll_rad',
        'pitchamp': 'Pitch_rad'
    }
    df.rename(columns=rename_map, inplace=True)

    return df

# retrieve the vessel RAO from Subsea 7 csv files
def getRAOs():
    folder_path = 'data/raos'
    directions = np.arange(15, 181, 15)  # Array of directions
    directions = np.append(directions, 360)

    # Initialize dictionaries to hold numpy arrays for each response
    response_data = {key: [] for key in ['Heave', 'Roll_rad', 'Pitch_rad']}
    frequencies = None  # To store frequency values common across all files

    # Generate the file names based on the specified format and known increments
    for x in directions:
        file_name = f"wave_dir_{x}.csv"
        file_path = os.path.join(folder_path, file_name)

        # Check if the file exists to avoid errors
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            df_cleaned = cleanRAO(df)

            if frequencies is None:  # Store frequency values from the first file
                frequencies = df_cleaned['freq'].values
            
            # Extract response data and append as numpy arrays
            for key in response_data:
                response_data[key].append(df_cleaned[key].values)
                
        else:
            print(f"File not found: {file_path}")

    # Convert lists of arrays into 2D numpy arrays
    for key in response_data:
        response_data[key] = np.column_stack(response_data[key])

    # Return the response data as a list of 2D numpy arrays along with frequencies
    TRF_2d_data = [response_data['Heave'], response_data['Roll_rad'], response_data['Pitch_rad']]

    return TRF_2d_data, frequencies, directions

# sort the RAOs so it is in the same format as the CFE RAO
def sort_RAOs(TRF_2d_data, f0_RAO, beta_deg_RAO):
    sorted_RAOs = []
    for trf in TRF_2d_data:
        if isinstance(trf, np.ndarray) and trf.ndim == 2:
            reversed_rows = trf[::-1]
            sorted_trf = np.roll(reversed_rows, shift=1, axis=1)
            sorted_RAOs.append(sorted_trf)
    
    # Reverse the numpy array f0_RAO
    f0_RAO = f0_RAO[::-1]  # This modifies the array by reversing it

    # If beta_deg_RAO is a numpy array, handle the popping of the last element and insertion differently
    if isinstance(beta_deg_RAO, np.ndarray):
        # Remove the last element and prepend 0 to the array
        beta_deg_RAO = np.delete(beta_deg_RAO, -1)  # Removes the last element
        beta_deg_RAO = np.insert(beta_deg_RAO, 0, 0)  # Inserts 0 at the start of the array

    return sorted_RAOs, f0_RAO, beta_deg_RAO

# Helper function to rename the first column to Time
def renameColTime(df):

    # Rename unnamed column by its index
    column_names = list(df.columns)
    column_names[0] = 'Time'  # Assuming the unnamed column is the first column
    df.columns = column_names

    return df

def getVesselData():
    df_motions = getData('motions_ts.csv')
    df_pos = getData('position_and_heading.csv')
    df_acc = getData('raw_accelerometer_motions_ts.csv')
    df_speed = getData('speed_lat_lon_ts.csv')    

    # Rename unnamed column by its index
    df_motions = renameColTime(df_motions)
    df_pos = renameColTime(df_pos)
    df_acc = renameColTime(df_acc)

    #df_speed = df_speed.drop(columns=df_speed.columns[0]) # Gets rid of first column which is some kind of indexing
    #df_speed.columns = ['Time', 'latitude', 'longitude', 'SOG']

    df_pos = df_pos.dropna() # gets rid of first row
    df_pos['Time'] = pd.to_datetime(df_pos['Time'], utc=True)
    df_pos = df_pos.rename(columns={'Time': 'Timestamp'})
    df_motions['Timestamp'] = pd.to_datetime(df_motions['Timestamp'], utc=True)
    df_speed['Timestamp'] = pd.to_datetime(df_speed['Timestamp'], utc=True)

    #df_speed['Speed'] = df_speed['SOG'] * 0.514444

    df_acc['Timestamp'] = pd.to_datetime(df_acc['Timestamp'], utc=True)
  
    return df_motions, df_pos, df_acc, df_speed

def getWaveSpectra(csv_files):
    # Initialize an empty list to store each DataFrame
    dfs = []

        # Loop through the list of CSV files
    for file in csv_files:
        # Check if file exists to avoid FileNotFoundError
        file = 'data/' + file
        if os.path.exists(file):
            # Read the current CSV file into a DataFrame
            df = pd.read_csv(file)
            
            # Append the DataFrame to the list
            dfs.append(df)
        else:
            print(f"File {file} not found.")

    # Concatenate all DataFrames in the list into a single DataFrame
    final_df = pd.concat(dfs, ignore_index=True)
    final_df= final_df.rename(columns={'time': 'Timestamp'}) # ensure it follows the standard notation
    #final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp'], utc=True)
    final_df.set_index(pd.to_datetime(final_df['Timestamp']), inplace=True)
    
    return final_df

def dfTimeFilter(df, start_time, end_time):
    return df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]

def countRowsOutsideTime(df, start_time, end_time):
    return len(df[~((df['Time'] >= start_time) & (df['Time'] <= end_time))])

# Datasets
def SampleRateDF(df):
    time_data = df['Time']
    #print(time_data.head())
    time1 = parser.parse(time_data.iloc[0])
    time2 = parser.parse(time_data.iloc[1])

    timestamp1 = time1.timestamp()
    timestamp2 = time2.timestamp()

    period = (timestamp2 - timestamp1)
    samplingRate = 1/period

    return round(samplingRate,2)

# input date is a string on the following format:
def toDatetime(date):
    date = date[0:-6]
    #year = date[]
    #print(f"this is date after slicing: {date}")
    date = pd.to_datetime(date)
    
    return date

def dfTimeDuration(df):
    # Calculate the difference between the last and first datetime values
    time_difference = df['Time'].iloc[-1] - df['Time'].iloc[0]
    return time_difference

def printStartEndTime(df):
    print(f"this is the start time: {df['Timestamp'].iloc[0]}")
    print(f"this is the end time: {df['Timestamp'].iloc[-1]}")


def saveSVG(title, folder_path):
    """
    Saves the plot as an SVG in a specified folder under the current directory.
    Automatically appends the current date to organize the files.

    :param title: Title and base name for the file to be saved.
    :param folder_path: Relative path from the current directory where the file should be saved.
                        
    """
    # Generate a unique filename using the current date and time
    current_time = datetime.now().strftime("%H%M")
    filename = f"{current_time}_"

    # Generate today's date string in the format YYYY-MM-DD
    date_string = datetime.now().strftime('%Y-%m-%d')

    # Create the new directory path by appending the date string
    date_folder_path = os.path.join(folder_path, date_string)

    # Check if the directory exists, if not, create it
    if not os.path.exists(date_folder_path):
        os.makedirs(date_folder_path)
    
    # Append hour and minute to the folder path
    hour_minute_folder_path = os.path.join(date_folder_path, current_time)

    # Check if the hour-minute directory exists, if not, create it
    if not os.path.exists(hour_minute_folder_path):
        os.makedirs(hour_minute_folder_path)

    title = filename + title

    # Define the file path including the name of the SVG file you want to save
    file_path = os.path.join(hour_minute_folder_path, title + '.svg')

    # Save the plot as an SVG file in the specified directory
    plt.savefig(file_path)
    print(f"Plot saved as {file_path}")

def saveSVG(title, folder_path, method):
    """
    Saves the plot as an SVG in a specified folder under the current directory.
    Automatically appends the current date to organize the files.

    :param title: Title and base name for the file to be saved.
    :param folder_path: Relative path from the current directory where the file should be saved.
                        
    """
    # Create the new directory path by appending the date string
    method_folder_path = os.path.join(folder_path, method)
     # Check if the directory exists, if not, create it
    if not os.path.exists(method_folder_path):
        os.makedirs(method_folder_path)

    # Generate a unique filename using the current date and time
    current_time = datetime.now().strftime("%H%M")
    filename = f"{current_time}_"

    # Generate today's date string in the format YYYY-MM-DD
    date_string = datetime.now().strftime('%Y-%m-%d')

    # Create the new directory path by appending the date string
    date_folder_path = os.path.join(method_folder_path, date_string)

    # Check if the directory exists, if not, create it
    if not os.path.exists(date_folder_path):
        os.makedirs(date_folder_path)
    
    # Append hour and minute to the folder path
    hour_minute_folder_path = os.path.join(date_folder_path, current_time)

    # Ensure the hour-minute directory exists
    if not os.path.exists(hour_minute_folder_path):
        os.makedirs(hour_minute_folder_path)
        print(f"Created directory: {hour_minute_folder_path}")

    # Generate full path for the file, checking if it already exists
    file_path = os.path.join(hour_minute_folder_path, f"{filename}{title}.svg")
    counter = 1
    while os.path.exists(file_path):
        file_path = os.path.join(hour_minute_folder_path, f"{filename}{title}_{counter}.svg")
        counter += 1

    # Save the plot as an SVG file in the specified directory
    plt.savefig(file_path)
    print(f"Plot saved as {file_path}")



def saveSVGContour(title, folder_path):
    """
    Saves the plot as an SVG in a specified folder under the current directory.
    Automatically appends the current date to organize the files.

    :param title: Title and base name for the file to be saved.
    :param folder_path: Relative path from the current directory where the file should be saved.
                        
    """
    # Generate a unique filename using the current date and time
    current_time = datetime.now().strftime("%H%M")
    filename = f"{current_time}_"

    # Generate today's date string in the format YYYY-MM-DD
    date_string = datetime.now().strftime('%Y-%m-%d')

    # Create the new directory path by appending the date string
    date_folder_path = os.path.join(folder_path, date_string)

    # Check if the directory exists, if not, create it
    if not os.path.exists(date_folder_path):
        os.makedirs(date_folder_path)
    
    # Append hour and minute to the folder path
    hour_minute_folder_path = os.path.join(date_folder_path, current_time)

    # Ensure the hour-minute directory exists
    if not os.path.exists(hour_minute_folder_path):
        os.makedirs(hour_minute_folder_path)
        print(f"Created directory: {hour_minute_folder_path}")

    # Generate full path for the file, checking if it already exists
    file_path = os.path.join(hour_minute_folder_path, f"{filename}{title}.svg")
    counter = 1
    while os.path.exists(file_path):
        file_path = os.path.join(hour_minute_folder_path, f"{filename}{title}_{counter}.svg")
        counter += 1

    # Save the plot as an SVG file in the specified directory
    plt.savefig(file_path)
    print(f"Plot saved as {file_path}")


def saveSVGWS(title, folder_path):
    """
    Saves the plot as an SVG in a specified folder under the current directory.
    Automatically appends the current date to organize the files.

    :param title: Title and base name for the file to be saved.
    :param folder_path: Relative path from the current directory where the file should be saved.
                        
    """

    # Generate a unique filename using the current date and time
    current_time = datetime.now().strftime("%H%M")
    filename = f"{current_time}_"

    # Generate today's date string in the format YYYY-MM-DD
    date_string = datetime.now().strftime('%Y-%m-%d')

    # Create the new directory path by appending the date string
    date_folder_path = os.path.join(folder_path, date_string)

    # Check if the directory exists, if not, create it
    if not os.path.exists(date_folder_path):
        os.makedirs(date_folder_path)
    
    # Append hour and minute to the folder path
    hour_minute_folder_path = os.path.join(date_folder_path, current_time)

    # Ensure the hour-minute directory exists
    if not os.path.exists(hour_minute_folder_path):
        os.makedirs(hour_minute_folder_path)
        print(f"Created directory: {hour_minute_folder_path}")

    # Generate full path for the file, checking if it already exists
    file_path = os.path.join(hour_minute_folder_path, f"{filename}{title}.svg")
    counter = 1
    while os.path.exists(file_path):
        file_path = os.path.join(hour_minute_folder_path, f"{filename}{title}_{counter}.svg")
        counter += 1

    # Save the plot as an SVG file in the specified directory
    plt.savefig(file_path)
    print(f"Plot saved as {file_path}")



def savetoCSV(df, base_name, directory="."):
    """
    Saves the DataFrame with a name that increments based on the highest existing iteration number.
    Depending on the base_name, files are saved in specific folders.

    :param df: DataFrame to save.
    :param base_name: Base name for the file, before the iteration part and without the extension.
    :param directory: Directory to check for existing files and save the new file.
    """

    # Determine the folder based on base_name
    if 'alpha_segment' in base_name:
        sub_directory = 'df_results'
    elif 'responseSpectrum_calculated' in base_name:
        sub_directory = 'responseSpectrumResults'
    else:
        raise ValueError("Unsupported base name. Expected names containing 'df_results' or 'responseSpectrum'.")
    
    # Create the full directory path if it does not exist
    full_directory = os.path.join(directory, sub_directory)
    os.makedirs(full_directory, exist_ok=True)
    pattern = re.compile(rf"{base_name}_iter_(\d+)\.csv$")
    highest_num = 0

    # Look through all files in the specified directory to find the highest iteration number
    for file_name in os.listdir(full_directory):
        match = pattern.match(file_name)
        if match:
            current_num = int(match.group(1))
            highest_num = max(highest_num, current_num)

    # Increment the highest number found for the new file name
    new_file_name = f"{base_name}_iter_{highest_num + 1}.csv"
    df.to_csv(os.path.join(full_directory, new_file_name), index=False)
    print(f"DataFrame saved as {os.path.join(full_directory, new_file_name)}")

def savetoCSVAllSegments(df, base_name, directory="."):
    """
    Saves the DataFrame with a name that increments based on the highest existing iteration number.
    Depending on the base_name, files are saved in specific folders.

    :param df: DataFrame to save.
    :param base_name: Base name for the file, before the iteration part and without the extension.
    :param directory: Directory to check for existing files and save the new file.
    """

    # Determine the folder based on base_name
    if 'alpha_segment' in base_name:
        sub_directory = 'df_results_ALL'
    elif 'responseSpectrum_calculated' in base_name:
        sub_directory = 'responseSpectrumResults_ALL'
    else:
        raise ValueError("Unsupported base name. Expected names containing 'df_results' or 'responseSpectrum'.")
    
    # Create the full directory path if it does not exist
    full_directory = os.path.join(directory, sub_directory)
    os.makedirs(full_directory, exist_ok=True)
    pattern = re.compile(rf"{base_name}_iter_(\d+)\.csv$")
    highest_num = 0

    # Look through all files in the specified directory to find the highest iteration number
    for file_name in os.listdir(full_directory):
        match = pattern.match(file_name)
        if match:
            current_num = int(match.group(1))
            highest_num = max(highest_num, current_num)

    # Increment the highest number found for the new file name
    new_file_name = f"{base_name}_iter_{highest_num + 1}.csv"
    df.to_csv(os.path.join(full_directory, new_file_name), index=False)
    print(f"DataFrame saved as {os.path.join(full_directory, new_file_name)}")

def savetoCSVAllSegmentsCombined(df, base_name, directory="."):
    """
    Saves the DataFrame with a name that increments based on the highest existing iteration number.
    Depending on the base_name, files are saved in specific folders.

    :param df: DataFrame to save.
    :param base_name: Base name for the file, before the iteration part and without the extension.
    :param directory: Directory to check for existing files and save the new file.
    """

    # Determine the folder based on base_name
    if 'alpha_segment' in base_name:
        sub_directory = 'df_results_ALL_combined'
    elif 'responseSpectrum_calculated' in base_name:
        sub_directory = 'responseSpectrumResults_ALL_combined'
    else:
        raise ValueError("Unsupported base name. Expected names containing 'df_results' or 'responseSpectrum'.")
    
    # Create the full directory path if it does not exist
    full_directory = os.path.join(directory, sub_directory)
    os.makedirs(full_directory, exist_ok=True)
    pattern = re.compile(rf"{base_name}_iter_(\d+)\.csv$")
    highest_num = 0

    # Look through all files in the specified directory to find the highest iteration number
    for file_name in os.listdir(full_directory):
        match = pattern.match(file_name)
        if match:
            current_num = int(match.group(1))
            highest_num = max(highest_num, current_num)

    # Increment the highest number found for the new file name
    new_file_name = f"{base_name}_iter_{highest_num + 1}.csv"
    df.to_csv(os.path.join(full_directory, new_file_name), index=False)
    print(f"DataFrame saved as {os.path.join(full_directory, new_file_name)}")

## Segmenting the data

In [ ]:
def segmentData(df, segment_duration, start, end):
    sr = SampleRateDF(df)
    print(f"this is the sample rate of the df: {sr}")

    # Convert start and end timestamps to pandas Timestamp objects
    start_time = pd.Timestamp(start)
    end_time = pd.Timestamp(end)

    # Calculate segment duration in seconds
    segment_duration_sec = segment_duration * 60

    segments = []

    # Initialize the segment start time
    segment_start_time = start_time

    while segment_start_time < end_time:
        # Calculate segment end time
        segment_end_time = segment_start_time + pd.Timedelta(seconds=segment_duration_sec)

        # Extract segment data from the dataframe
        segment = df[(df['Timestamp'] >= segment_start_time) & (df['Timestamp'] < segment_end_time)]

        # Append segment to the list
        segments.append(segment)

        # Move to the next segment start time
        segment_start_time = segment_end_time

    return segments

def avgSegments(segments):
    averaged_segments = []
    for segment in segments:
        # Calculate the mean only for numeric columns
        averaged_segment = segment.mean(numeric_only=True)
        
        # Get the 'Timestamp' from the first row of the segment
        first_timestamp = segment['Timestamp'].iloc[0]
        averaged_segment['Timestamp'] = first_timestamp
        
        averaged_segments.append(averaged_segment)

    return averaged_segments

def segmentData_pos(df, segment_duration, start, end):

    # Sample rate in samples per hour given sampling once every 10 minutes
    sr = 6  # 60 minutes / 10 minutes = 6 samples per hour

    print(f"This is the assumed sample rate of the df: {sr} samples per hour")

    # Assuming df['Timestamp'] is already in datetime format, if not, convert
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Samples per segment is fixed at 4 given the 10-minute sampling rate for a 30-minute duration
    samples_per_segment = 4  # As we include the start and end within the 30-minute window

    segments = []

    # Filter df for rows within the start and end times to reduce processing
    df_filtered = df[(df['Timestamp'] >= pd.to_datetime(start)) & (df['Timestamp'] <= pd.to_datetime(end))]

    # Calculate segment start indices
    segment_indices = range(0, len(df_filtered), samples_per_segment - 1)

    # Create segments
    for i in segment_indices:
        # Ensure not to exceed the DataFrame length
        if i + samples_per_segment <= len(df_filtered):
            segment = df_filtered.iloc[i:i + samples_per_segment]

            segments.append(segment)

    return segments

def segmentData_ws(df_ws_interpolated, start, end):
    """
    Filters the dictionary of arrays by timestamps to include only those within
    the specified start and end timestamps.

    Parameters:
    - df_ws_interpolated: Dictionary with keys as timestamps and values as arrays.
    - start: Start timestamp (inclusive).
    - end: End timestamp (inclusive).

    Returns:
    - A filtered dictionary with timestamps within the specified range.
    """
    # Convert start and end to pandas timestamps if they are not already
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)

    start = start.tz_convert(None)
    end = end.tz_convert(None)

    # Filter the dictionary
    filtered_dict = {timestamp: array for timestamp, array in df_ws_interpolated.items() if start <= timestamp < end} # does not include the end segment as it has to match nr of elements in df_pos

    return filtered_dict
    
def measure_duration_column(df, datetime_column):
    #Measure the duration between the first and last datetime in a specified DataFrame column.

    if datetime_column == 'Timestamp': # does not need to be converted to perform calculations
        time_start = df[datetime_column].iloc[0]
        time_end = df[datetime_column].iloc[-1]

    else:

        time_start = toDatetime(df[datetime_column].iloc[0])

        time_end = toDatetime(df[datetime_column].iloc[-1])
    
    duration = time_end - time_start

    return duration

## Plotting

In [ ]:
def motionPlot(df, step=1):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))

    for ax in axes.flat:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H: %M'))
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        ax.set_xlabel("Time")
        ax.tick_params(axis='x', rotation=45)

    # Plotting "Roll"
    axes[0, 0].plot_date(df['Timestamp'][::step], df['Roll'][::step], '-')
    axes[0, 0].set_ylabel("Degrees")
    axes[0, 0].legend(["Roll"])

    # Plotting "Pitch"
    axes[0, 1].plot_date(df['Timestamp'][::step], df['Pitch'][::step], '-')
    axes[0, 1].set_ylabel("Degrees")
    axes[0, 1].legend(["Pitch"])

    # Plotting "Heave"
    axes[1, 0].plot_date(df['Timestamp'][::step], df['Heave'][::step], '-')
    axes[1, 0].set_ylabel("Meters")
    axes[1, 0].legend(["Heave"])

    # Plotting "Heave Velocity"
    axes[1, 1].plot_date(df['Timestamp'][::step], df['Heave_velocity'][::step], '-')
    axes[1, 1].set_ylabel("Meters/Second")
    axes[1, 1].legend(["Heave Velocity"])

    # Rotate date labels for clarity
    #plt.setp(axes.flat, rotation=45, ha="right")

    # Adjust layout for a cleaner look
    plt.tight_layout()

    plt.show()
    
def PosPlotTotal(df_pos):
    #print(f"the df_pos contain the following types: {df_pos.info()}")
    plt.figure(figsize=(10, 6))

    # Plot trajectory
    plt.plot(df_pos['longitude'], df_pos['latitude'], 'b-', label='Trajectory')

    # targets the indeces of every hour in the df_pos
    hourly_indices = range(0, df_pos.shape[0], 6)

    # Overlay red dots for every hour
    plt.plot(df_pos.iloc[hourly_indices]['longitude'], df_pos.iloc[hourly_indices]['latitude'], 'r.', markersize=10, label='Hourly Position')  # Red dots at hourly intervals

    # Adding plot details
    plt.title('Vessel Trajectory with Hourly Marks')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(True)
    plt.legend()
    
    plt.show()

def posPlotheading(df_pos):
    #df_pos = df_pos.set_index('Timestamp')

    # Now, plot the 'Heading' column
    plt.figure(figsize=(10, 6))
    df_pos.plot(x='Timestamp', y='hdt')

    plt.title('Heading Over Time')
    plt.xlabel('Time') # Since 'Timestamp' is the index, x-axis represents time
    plt.ylabel('Heading Value')
    plt.xticks(rotation=45) # Rotate x-axis labels for better readability
    plt.tight_layout() # Adjust layout to not cut off labels

    plt.show()

def plotError(df_results_alpha, save = False):
    title_fig = 'Error_progress'
    error_values = df_results_alpha['Error']

    # Plotting the error values
    plt.figure(figsize=(10, 6))
    plt.plot(error_values, marker='o', linestyle='-')
    plt.title('Error Progress')
    plt.xlabel('Segment')
    plt.ylabel('Error')
    plt.grid(True)

    if (save):
            folder_path = 'Plots/Results'

            saveSVG(title_fig, folder_path)

    plt.show()



### Response Spectrum plots

In [ ]:

def plotResponseSpectrum(freq, df, title, save = False):
  # Want  to only plot for frequencies [0, 0.3] as there is very little response for frequencies above 0.3 Hz
  freq = freq[freq < 0.3]
  indices = len(freq)
  df = df.iloc[:indices]
              
  fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))

  fig.suptitle(title, fontsize=16)


  # Plotting "Roll"

  axes[0,0].plot(freq, np.abs(df['Roll_rad']))
  axes[0,0].set_xlabel("frequency [Hz]")
  axes[0,0].set_ylabel("Roll [rad**2/Hz]")
  axes[0,0].set_title('Roll')

  # Plotting "Pitch"

  axes[0,1].plot(freq, np.abs(df['Pitch_rad']))
  axes[0,1].set_xlabel("frequency [Hz]")
  axes[0,1].set_ylabel("Pitch [rad**2/Hz]")
  axes[0,1].set_title('Pitch')


  # Plotting "Heave"
  axes[1,0].plot(freq, np.abs(df['Heave']))
  axes[1,0].set_xlabel("frequency [Hz]")
  axes[1,0].set_ylabel("Heave [m**2/Hz]")
  axes[1,0].set_title('Heave')


  # Plotting "Heave Velocity"

  axes[1,1].plot(freq, np.abs(df['Heave_velocity']))
  axes[1,1].set_xlabel("frequency [Hz]")
  axes[1,1].set_ylabel("Heave velocity [(m/s)**2/Hz]")
  axes[1,1].set_title('Heave Velocity')

  if (save):
      folder_path = 'Plots/ResponseSpectra'
      
      saveSVG(title, folder_path)

  # Adjust layout for a cleaner look
  plt.tight_layout()

  plt.show()

  return
    
def plotResponseSpectrumResult(segment, freq, responseSpectrum_measured, responseSpectrum_calculated, title_file, save = False):

  mask_freq = freq < 0.3
  freq_filtered = freq[mask_freq]
  
  responseSpectrum_measured_filtered = responseSpectrum_measured[segment].loc[mask_freq]              # responseSpectrum_measured is a list of dfs
  
  # is only a single df. Sort by column segment so that it matches current segment
  mask_segment = responseSpectrum_calculated['segment'] == segment                                    # Creating a mask where 'segment' equals the current segment

  filtered_responseSpectrum_calculated = responseSpectrum_calculated[mask_segment]                    # Applying the mask to the DataFrame to filter rows
  

  segment_array = filtered_responseSpectrum_calculated['segment']


  filtered_responseSpectrum_calculated =  filtered_responseSpectrum_calculated.loc[mask_freq]         # apply mask for only containing frequencies up to 0.3 hz 

  filtered_responseSpectrum_calculated = filtered_responseSpectrum_calculated.reset_index(drop=True)  # reset the index so that it matches the measured response spectrum. 

  segment_array = filtered_responseSpectrum_calculated['segment']
  title_fig = f'Response estimation results for segment: {segment}'       
  fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 10))  # Adjusted width
  fig.suptitle(title_fig, fontsize=16)


  # Plotting "Roll"
  linewidth_measured = 2
  linewidth_calculated = 1.5
  measured_color = 'g'      # green
  calculated_color = 'b'    # Blue
  error_color = 'r'         # A bright red for error
  measured_marker = 's'     # Square marker

  # Titles for each subplot
  titles = ['Heave', 'Roll', 'Pitch']
  y_labels = ['Heave [m**2/Hz]', 'Roll [rad**2/Hz]', 'Pitch [rad**2/Hz]']

  # Data keys for each plot, assuming these are column names in your DataFrames
  data_keys = ['Heave', 'Roll_rad', 'Pitch_rad']

  # Iterate over the axes, titles, and data keys to create each subplot
  for ax, title, y_label, key in zip(axes, titles, y_labels, data_keys):
      # Plot measured data
      ax.plot(freq_filtered, np.abs(responseSpectrum_measured_filtered[key]), 
              label='Measured', color=measured_color, linewidth=linewidth_measured, 
              markersize=4, marker=measured_marker)

      # Plot calculated data
      ax.plot(freq_filtered, np.abs(filtered_responseSpectrum_calculated[key]), 
              label='Calculated', color=calculated_color, linestyle='-', 
              linewidth=linewidth_calculated)

      # Calculate and plot error
      error = np.abs(responseSpectrum_measured_filtered[key] - filtered_responseSpectrum_calculated[key])
      ax.plot(freq_filtered, error, label='Error', color=error_color, linestyle='--', 
              linewidth=1.5)

      # Set titles and labels
      ax.set_title(title)
      ax.set_xlabel("Frequency [Hz]")
      ax.set_ylabel(y_label)

      # Add legend
      ax.legend()
      ax.grid(True)
  
  if (save):
      folder_path = 'Plots/Results'
      saveSVG(title_file, folder_path)

  # Adjust layout for a cleaner look
  plt.tight_layout()
  plt.show()
  return

def plotErrorBetweenSpectra(df_error_SM, df_error_WA, title_file, save= False):
    title_fig = f'Error between Measured and Calculated Response Spectra for different averaging methods'

    segment_array = np.arange(len(df_error_SM))
         
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 10))  # Adjusted width
    fig.suptitle(title_fig, fontsize=16)

    linewidth_measured = 2
    linewidth_calculated = 2
    measured_color = 'r'      # red
    calculated_color = 'b'    # Blue
    #error_color = 'r'         # A bright red for error
    #measured_marker = 's'     # Square marker

    # Titles for each subplot
    titles = ['Heave', 'Roll', 'Pitch']
    y_labels = ['Heave [m**2/Hz]', 'Roll [rad**2/Hz]', 'Pitch [rad**2/Hz]']

    # Data keys for each plot, assuming these are column names in your DataFrames
    data_keys = ['Heave', 'Roll_rad', 'Pitch_rad']

    # Iterate over the axes, titles, and data keys to create each subplot
    for ax, title, y_label, key in zip(axes, titles, y_labels, data_keys):
        # Plot measured data
        ax.plot(segment_array, np.abs(df_error_SM[key]), 
                label='Simple Mean', color=measured_color, linestyle = '--', linewidth=linewidth_measured, 
                markersize=4)

        # Plot calculated data
        ax.plot(segment_array, np.abs(df_error_WA[key]), 
                label='Weighted avg', color=calculated_color, linestyle='-', 
                linewidth=linewidth_calculated)

        # Set titles and labels
        ax.set_title(title)
        ax.set_xlabel("Segment")
        ax.set_ylabel(y_label)

        # Add legend
        ax.legend()
   
    if (save):
        folder_path = 'Plots/Results'
        saveSVG(title_file, folder_path)

    # Adjust layout for a cleaner look
    plt.tight_layout()
    plt.show()
    return

def sliceCalculatedResponseSpectrum(responseSpectrum_calculated, segment, mask_freq):
      # is only a single df. Sort by column segment so that it matches current segment
    mask_segment = responseSpectrum_calculated['segment'] == segment
    filtered_responseSpectrum_calculated = responseSpectrum_calculated[mask_segment] 
    filtered_responseSpectrum_calculated =  filtered_responseSpectrum_calculated.loc[mask_freq]         # apply mask for only containing frequencies up to 0.3 hz 
    filtered_responseSpectrum_calculated = filtered_responseSpectrum_calculated.reset_index(drop=True)  # reset the index so that it matches the measured response spectrum.

    return filtered_responseSpectrum_calculated


def plotResponseSpectrumResult_avg_method(segment, freq, responseSpectrum_calculated_SM, responseSpectrum_calculated_WA, df_error_SM, df_error_WA, df_error_calculated, df_results_alpha_current, responseSpectrum_measured, responseSpectrum_calculated, title_file, save = False):
    mask_freq = freq < 0.3
    freq_filtered = freq[mask_freq]

    responseSpectrum_measured_curr = responseSpectrum_measured[segment]
    mask_segment = responseSpectrum_calculated['segment'] == segment
    filtered_responseSpectrum_calculated = responseSpectrum_calculated[mask_segment] 

    #check_dataframe_indices(responseSpectrum_measured_curr, filtered_responseSpectrum_calculated, responseSpectrum_calculated_SM, responseSpectrum_calculated_WA)
    
    responseSpectrum_measured_filtered = responseSpectrum_measured[segment].loc[mask_freq]              # responseSpectrum_measured is a list of dfs
    
    filtered_responseSpectrum_calculated = sliceCalculatedResponseSpectrum(responseSpectrum_calculated, segment,mask_freq)
    filtered_responseSpectrum_calculated_SM = sliceCalculatedResponseSpectrum(responseSpectrum_calculated_SM, segment, mask_freq)
    filtered_responseSpectrum_calculated_WA = sliceCalculatedResponseSpectrum(responseSpectrum_calculated_WA, segment, mask_freq)

    df_error_calculated_curr = df_error_calculated.iloc[segment]
    df_error_SM_curr = df_error_SM.iloc[segment] # on the form ['Heave' : x, 'Pitch_rad': x, 'Roll_rad': x ]
    df_error_WA_curr = df_error_WA.iloc[segment]

    segment_array = filtered_responseSpectrum_calculated['segment']

    title_fig = f'Response estimation results for segment: {segment}'       
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 10))  # Adjusted width
    fig.suptitle(title_fig, fontsize=20, fontweight='bold')


    # Plotting "Roll"
    linewidth_measured = 1.0
    linewidth_calculated = 1.0
    measured_color = 'g'      # green
    calculated_color = 'k'    # black
    SM_color = 'r'         # A bright red for error
    WA_color = 'b'
    measured_marker = 's'     # Square marker

    # Titles for each subplot
    titles = ['Heave', 'Roll', 'Pitch']
    y_labels = ['Heave [m**2/Hz]', 'Roll [rad**2/Hz]', 'Pitch [rad**2/Hz]']

    label_strings = [
    [fr'$\varepsilon_{{zz}} = {df_error_calculated_curr["Heave"]:.3f}$', fr'$\varepsilon_{{zz}} = {df_error_SM_curr["Heave"]:.3f}$', fr'$\varepsilon_{{zz}} = {df_error_WA_curr["Heave"]:.3f}$'],
    [fr'$\varepsilon_{{\phi \phi}} = {df_error_calculated_curr["Roll_rad"]:.3f}$', fr'$\varepsilon_{{\phi \phi}} = {df_error_SM_curr["Roll_rad"]:.3f}$', fr'$\varepsilon_{{\phi \phi}} = {df_error_WA_curr["Roll_rad"]:.3f}$'],
    [fr'$\varepsilon_{{\theta \theta}} = {df_error_calculated_curr["Pitch_rad"]:.3f}$', fr'$\varepsilon_{{\theta \theta}} = {df_error_SM_curr["Pitch_rad"]:.3f}$', fr'$\varepsilon_{{\theta \theta}} = {df_error_WA_curr["Pitch_rad"]:.3f}$']
    ]

    legend_titles = [fr'$S_{{zz}}: Seg. {segment}$', fr'$S_{{\phi \phi}}: Seg. {segment}$', fr'$S_{{\theta \theta}}: Seg. {segment}$']
   
    # Data keys for each plot, assuming these are column names in your DataFrames
    data_keys = ['Heave', 'Roll_rad', 'Pitch_rad']

    # Iterate over the axes, titles, and data keys to create each subplot
    for ax, title, y_label, key, label_list, legend_title in zip(axes, titles, y_labels, data_keys, label_strings, legend_titles):
        # Plot measured data

        ax.plot(freq_filtered, np.abs(responseSpectrum_measured_filtered[key]), 
                label='Measured', color=measured_color, linewidth=linewidth_measured, 
                markersize=4, marker=measured_marker)

        # Plot calculated data
        ax.plot(freq_filtered, np.abs(filtered_responseSpectrum_calculated[key]), 
                label=label_list[0], color=calculated_color,  linestyle='-.', 
                linewidth=linewidth_calculated)
        
        ax.plot(freq_filtered, np.abs(filtered_responseSpectrum_calculated_SM[key]), label=label_list[1], color=SM_color, linestyle='--', 
                linewidth=1.0)
        
        ax.plot(freq_filtered, np.abs(filtered_responseSpectrum_calculated_WA[key]), label=label_list[2], color=WA_color, linestyle='-', 
                linewidth=1.0)

        # Set titles and labels
        ax.set_title(title, fontsize=16)
        ax.set_xlabel("Encounter frequency [Hz]", fontsize=14)
        ax.set_ylabel(y_label, fontsize=14)
        # Add legend
        legend = ax.legend(fancybox=True, fontsize=12, title_fontsize=14, loc = "upper right", framealpha=0.5) # frame alpha controls transparency
        legend.set_title(legend_title)

        ax.tick_params(axis='both', which='major', labelsize=12)  # Adjust tick parameters for readability

        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax.lines[0].set_linewidth(3)

    if (save):
        folder_path = 'Plots/Results'
        saveSVG(title_file, folder_path)

    # Adjust layout for a cleaner look
    plt.tight_layout()

    plt.show()

    return
    
def sliceCalculatedResponseSpectrum_same_segment(responseSpectrum_calculated, iteration, mask_freq, segment):
      # is only a single df. Sort by column segment so that it matches current segment
    mask_segment = responseSpectrum_calculated['segment'] == segment
    filtered_responseSpectrum_calculated = responseSpectrum_calculated[mask_segment] 
    filtered_responseSpectrum_calculated =  filtered_responseSpectrum_calculated.loc[mask_freq]         # apply mask for only containing frequencies up to 0.3 hz 
    filtered_responseSpectrum_calculated = filtered_responseSpectrum_calculated.reset_index(drop=True)  # reset the index so that it matches the measured response spectrum.

    return filtered_responseSpectrum_calculated


def plotResponseSpectrumResult_same_segment(segment, f0, responseSpectrum_measured_curr_seg_abs, responseSpectrum_calculated, title_file, save = False):
    # only select the last iteration as it gives the best result
    # Step 1: Trim responseSpectrum_calculated to match the length of responseSpectrum_measured
    n_rows = responseSpectrum_measured_curr_seg_abs.shape[0]


    responseSpectrum_calculated = responseSpectrum_calculated.tail(n_rows)

    # Step 2: Create a mask for frequency values less than 0.3
    mask_freq = f0 < 0.3

    # Apply the mask to frequency array to create a filtered frequency array (optional, if needed)
    freq_filtered = f0[mask_freq]
 
    # Apply the mask to both DataFrames
    responseSpectrum_measured_filtered = responseSpectrum_measured_curr_seg_abs[mask_freq].reset_index(drop=True)
    responseSpectrum_calculated_filtered = responseSpectrum_calculated[mask_freq].reset_index(drop=True)

    title_fig = f'Response estimation results after optimizing parameters on segment: {segment}'       
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 10))  # Adjusted width
    fig.suptitle(title_fig, fontsize=16)

    #df_motions["Roll"].plot(ax=axes[0,0], legend=True, grid=True, title="Roll")
    linewidth_measured = 2
    linewidth_calculated = 1.5
    measured_color = 'g'      # green
    calculated_color = 'b'    # Blue
    error_color = 'r'         # A bright red for error
    measured_marker = 's'     # Square marker

    # Titles for each subplot
    titles = ['Heave', 'Roll', 'Pitch']
    y_labels = ['Heave [m**2/Hz]', 'Roll [rad**2/Hz]', 'Pitch [rad**2/Hz]']

    # Data keys for each plot, assuming these are column names in your DataFrames
    data_keys = ['Heave', 'Roll_rad', 'Pitch_rad']

    # Iterate over the axes, titles, and data keys to create each subplot
    for ax, title, y_label, key in zip(axes, titles, y_labels, data_keys):
        # Plot measured data
        ax.plot(freq_filtered, np.abs(responseSpectrum_measured_filtered[key]), 
                label='Measured', color=measured_color, linewidth=linewidth_measured, 
                markersize=4, marker=measured_marker)

        # Plot calculated data
        ax.plot(freq_filtered, np.abs(responseSpectrum_calculated_filtered[key]), 
                label='Calculated', color=calculated_color, linestyle='-', 
                linewidth=linewidth_calculated)

        # Set titles and labels
        ax.set_title(title)
        ax.set_xlabel("Absolute Frequency [Hz]")
        ax.set_ylabel(y_label)

        # Add legend
        ax.legend()
        ax.grid(True)

    if (save):
        folder_path = 'Plots/Results'
        saveSVG(title_file, folder_path)

    # Adjust layout for a cleaner look
    plt.tight_layout()

    plt.show()

    return
       
       

## Functions for RAOs

In [ ]:
# Use the result from df_results_alpha to find the best performing alpha. Use the alpha in the CFE to calculate the transfer functions.

def segmentData_TRF(om0_rad, segment, df_speed_segments_avg):
    #yaw = df_pos_segments_avg[segment]['hdt']
    U = df_speed_segments_avg[segment]['Speed']
    Nabla = None   # Volume Deplacement

    return om0_rad, U, Nabla

def calculateTRF(alpha, segment_data_TRF, beta_plot):
    df_TRF_list = []

    om0_rad, U, Nabla = segment_data_TRF

    C_B_1 = Nabla / (alpha[0] * alpha[1] * alpha[2])
    C_B_2 = Nabla / (alpha[4] * alpha[5] * alpha[6])

    responses = ['Heave', 'Roll_rad', 'Pitch_rad']

    for response in responses:
        # Calculate the CFEs
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(om0_rad,beta_plot,U,alpha[0],alpha[1],alpha[2],C_B_1) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = TRF_2d * alpha[3] # multiply by the gain
            
        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF_2d = pitchCF(om0_rad,beta_plot,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[7] - alpha[9]
            TRF_2d = rollCF(om0_rad,beta_plot,U,alpha[4],alpha[5],alpha[6],C_B_2,alpha[7],alpha[8],kappa,alpha[10],T_N=0)

        df_TRF_list.append(TRF_2d)

    return df_TRF_list

def calculateTRFDeg(alpha, segment_data_TRF, beta_plot):
    df_TRF_list = []

    om0_rad, U, Nabla = segment_data_TRF

    C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])

    responses = ['Heave', 'Roll_rad', 'Pitch_rad']

    for response in responses:
         # Calculate the CFEs
         # the resulting TRF is a numpy array of size (len(om0_rad), len(beta_TRF))
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(om0_rad,beta_plot,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF_2d = pitchCFDeg(om0_rad,beta_plot,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[3] - alpha[5]
            
            TRF_2d = rollCFDeg(om0_rad,beta_plot,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)
            
        df_TRF_list.append(TRF_2d)

    return df_TRF_list

def plotIsolatedTransferFunctionsWithSubplots(TRF_init, TRF_best, om0_rad, beta_plot, responses,title_file, save=False):
    # Ensure beta_plot is a numpy array and contains the desired beta values
    beta_degrees = [60, 120, 180]  # Define the specific beta degree values to plot
    beta_indices = [np.where(beta_plot == bd)[0][0] for bd in beta_degrees]  # Get indices for these values in beta_plot

    for idx, response in enumerate(responses):
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # Create a figure with three subplots
        fig.suptitle(f'{response} Transfer Function Results', fontsize=20)

        # Iterate through the desired beta indices and plot the initial and best TRF
        for ax, beta_idx in zip(axes, beta_indices):
            beta_degree = beta_plot[beta_idx]
            label_init = f'Initial'
            label_best = f'Optimized'
            ax.plot(om0_rad, np.abs(TRF_init[idx][:, beta_idx]), label=label_init, linestyle = '--', linewidth=2)
            ax.plot(om0_rad, np.abs(TRF_best[idx][:, beta_idx]), label=label_best, linestyle='-', linewidth=2)

            # Set titles, labels, and grid for each subplot
            ax.set_title(f'Beta = {beta_degree}°', fontsize = 14)
            ax.set_xlabel('Frequency [rad/s]', fontsize=11)
            y_label = f'{response} [{"m/" if response == "Heave" else "rad/"}m]'
            ax.set_ylabel(y_label,fontsize=11)

            # Customize the legend
            legend_title = 'Initial CFE | Optimized CFE'  # You can customize this title

                        # Adjust the legend properties to make it smaller
            legend = ax.legend(
                fancybox=True, 
                fontsize=10, 
                title_fontsize=12, 
                loc="upper right", 
                framealpha=0.5, 
                borderpad=0.5,  # Smaller border padding
                labelspacing=0.5,  # Smaller label spacing
                handlelength=2  # Shorter handles
            )
            legend.set_title(legend_title)
            legend.get_frame().set_edgecolor('black')  # Set the legend frame edge color
            plt.setp(legend.get_texts(), fontsize='10')  # Adjust font size of legend text
            """
            # Add legend
            legend = ax.legend(fancybox=True, fontsize=12, title_fontsize=14, loc = "upper right", framealpha=0.5) # frame alpha controls transparency
            legend.set_title(legend_title)
            legend.get_frame().set_edgecolor('black')  # Set the legend frame edge color
            #legend.get_title().set_fontweight('bold')  # Set the title to bold
            """
            #ax.tick_params(axis='both', which='major', labelsize=12)  # Adjust tick parameters for readability

            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            #ax.lines[0].set_linewidth(3)
            

        # Adjust layout and show/save the plot
        plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust the top margin to not overlap with suptitle
        if (save):
            folder_path = 'Plots/TransferFunctionResults'
            saveSVG(title_file, folder_path)
        plt.show()

def plotTransferFunction(df_results_alpha_current, alpha_init, df_speed_segments_avg, segment, om0_rad, title_file, save = False):

    # want to make a plotting of transfer functions for comparing CFE transfer functions before and after optimizing
    # should currently just work for the same segment. 
    # Therefore 

    segment_data_TRF = segmentData_TRF(om0_rad, segment, df_speed_segments_avg)

    best_alpha = df_results_alpha_current.tail(1)
    best_alpha = best_alpha.to_numpy()

    print(f"this is best_alpha: {best_alpha}")
    error = best_alpha[-1, -1]
    print(f"this is the error: {error}")
    best_alpha = best_alpha[0, :-1].flatten()
    print(f"this is best_alpha as a 1d array: {best_alpha}")

    beta_plot = np.arange(0, 181, 30)

    TRF_init = calculateTRF(alpha_init, segment_data_TRF, beta_plot)
    TRF_best = calculateTRF(best_alpha, segment_data_TRF, beta_plot)
    responses = ['Heave', 'Roll_rad', 'Pitch_rad']

    plotIsolatedTransferFunctionsWithSubplots(TRF_init, TRF_best, om0_rad, beta_plot, responses,title_file, save=save)
    
    return
        

## Misc

In [ ]:

def c(theta):
    return np.cos(theta)

def s(theta):
    return np.sin(theta)

# Euler angles for rotation from body to NED frame

# roll - phi, pitch - theta, yaw - psi
def rotationBN(pitch, roll, yaw):
    Rx = np.matrix([
        [1,0,0],
        [0, c(roll), -s(roll)],
        [0, s(roll), c(roll)]
    ])
    Ry = np.matrix([
        [c(pitch),0,s(pitch)],
        [0, 1, 0],
        [-s(pitch), 0, c(pitch)]
    ])
    Rz = np.matrix([
        [c(yaw),-s(yaw),0],
        [s(yaw), c(yaw), 0],
        [0, 0, 1]
    ])

    R = Rx*Ry*Rz
    return R

def print_basic_info(mu):
    print("Type of mu:", type(mu))
    print("Length of mu:", len(mu))
    print("First few elements:", mu[:5])
    print("Last few elements:", mu[-5:])

def distance(point1, point2):
# Calculate the distance between the points
    distance = geodesic(point1, point2).kilometers
    print(f"The distance between ({str(point1)}, {str(point2)}) is : {distance} kilometers.")
    return distance  # or .miles for distance in miles



### Functions for ensuring stationary data segments

In [ ]:
def stationaryDFHeading(df_pos_segments):
    # Function to calculate the normalized heading difference accounting for wrap-around
    def normalized_heading_diff(h1, h2):
        return min(abs(h1 - h2), 360 - abs(h1 - h2))

    # List to hold stationary dataframes
    stationary_segments = []

    non_stationary_segments_idx =[]

    segment = 0
    # Iterate through each segment
    for df in df_pos_segments:
        if 'hdt' in df.columns:
            # Calculate differences between all pairs of headings in the segment
            max_diff = 0
            for i in range(len(df['hdt'])):
                for j in range(i + 1, len(df['hdt'])):
                    diff = normalized_heading_diff(df['hdt'].iloc[i], df['hdt'].iloc[j])
                    if diff > max_diff:
                        max_diff = diff

            # Check if the maximum difference exceeds 10 degrees
            if max_diff > 10:
                # Assuming there's a time column called 'time' to identify the start time of the segment
                print(f"Segment nr: {segment} starting at {df['Timestamp'].iloc[0]} is non-stationary and will be removed.")
                non_stationary_segments_idx.append(segment)
            else:
                df = df.reset_index(drop=True)
                stationary_segments.append(df)
                # want to reset the index for each df anywars
        else:
            print("The dataframe does not contain a 'hdt' column.")
        
        segment += 1

    return stationary_segments, non_stationary_segments_idx

# function for checking if there are any negative values of speed in the segments. There are none.
def check_negative_speed(df_speed_segments):
    for segment, df in enumerate(df_speed_segments):
        if 'Speed' in df.columns:
            if (df['Speed'] < 0).any():
                print(f"Segment nr: {segment} contains negative speeds.")
            else:
                print(f"Segment nr: {segment} does not contain negative speeds.")
        else:
            print("The dataframe does not contain a 'Speed' column.")

#check_negative_speed(df_speed_segments)

# Checked: speed does not exceed 1 m/s
# function for ensuring speed in segments does not exceed 1 m/s. 
def stationaryDFSpeed(df_speed_segments):
    # List to hold stationary dataframes
    stationary_segments = []
    
    segment = 0
    # Iterate through each segment
    for df in df_speed_segments:
        start_time = time.time()
        print(f"Currently in segment nr: {segment}")
        #print(f"This is the number of rows in this segment: {len(df)}")
        
        if 'Speed' in df.columns:
            # Sample speeds every tenth row
            sampled_indices = range(0, len(df['Speed']), 10)
            speeds_sampled = df['Speed'].iloc[sampled_indices]

            # Check if any sampled speed exceeds the threshold of 1 m/s
            if all(speed <= 1 for speed in speeds_sampled):
                # Reset the index for each dataframe that is appended to the list
                df = df.reset_index(drop=True)
                stationary_segments.append(df)
            else:
                print(f"Segment nr: {segment} starting at {df['Timestamp'].iloc[0]} has speeds exceeding 1 m/s and will be removed.")
        else:
            print("The dataframe does not contain a 'Speed' column.")
        
        segment += 1  # Increment segment counter
        end_time = time.time()
        #print(f"Execution time for a segment: {(end_time - start_time)/60} minutes")

    return stationary_segments

def remove_indices_from_list(df_motions_segments, non_stationary_segments_idx):
    # Sort the indices in reverse order to ensure correct removal
    non_stationary_segments_idx.sort(reverse=True)
    
    # Remove elements from the list using the indices
    for idx in non_stationary_segments_idx:
        if 0 <= idx < len(df_motions_segments):
            del df_motions_segments[idx]
    
    return df_motions_segments

def remove_indices_from_dict(df_ws_segments, non_stationary_segments_idx):
    # Convert the dictionary to a list of key-value pairs
    items = list(df_ws_segments.items())

    # Iterate over the list of indices
    for index in non_stationary_segments_idx:
        # Check if the index is within the range of the list of items
        if 0 <= index < len(items):
            # Remove the item with the corresponding index
            del items[index]

    # Convert the modified list back to a dictionary
    df_ws_segments = dict(items)

    return df_ws_segments

### Functions for averaging parameters for CFE

In [ ]:

def calculateResponseSpectrum(alpha, ws, segment):

    responseSpectrum_calculated_tmp = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
    responseSpectrum_calculated_tmp[response] = calculated_spectrum_values_tmp
    Nabla = None

    C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])

    responses = ['Heave', 'Roll_rad', 'Pitch_rad']

    for response in responses:

        if response == "Heave":
                # transfer function for Heave is in units [m/m]
            TRF1 = heaveCF(om0_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF1 = pitchCF(om0_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[3] - alpha[5]
            #mu = alpha[6]
            #B0 = alpha[1] # assigned B0 = B. Not sure if this is right.
            # transfer function for Roll is in units [rad/m]
            TRF1 = rollCF(om0_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)

        # Copy TRF2 as TRF1 to get a Power Spectral Density and not a Cross Spectral Density
        TRF2 = TRF1

        calculated_spectrum_values_tmp = resp_spec_2d(f0, mu, ws, beta_TRF, TRF1, TRF2, fe, U, yaw, conv='to')

        responseSpectrum_calculated_tmp[response] = calculated_spectrum_values_tmp
    
    # fill in
    responseSpectrum_calculated_tmp['segment'] = segment
        
    return responseSpectrum_calculated_tmp 

def simpleMean(alpha_final):
   return alpha_final.mean()

def weightedAvg(alpha_final):
   fmax = alpha_final['Error'].max()

   param_sum_numerator = pd.Series([0] * (len(alpha_final.columns) - 1), index=alpha_final.columns[:-1])
   param_sum_denominator = 0

   for idx, row in alpha_final.iterrows():
      if row['Error'] < fmax:
         weight_i = fmax - row['Error']
      else:
         weight_i = 0
      
      #alpha['Weight'] = weight_i
      # update the sums for weighted avg calc
      param_sum_numerator += weight_i * row[:-1] # to slice off the last column: error
      param_sum_denominator += weight_i
   
   # Calculate weighted average if denominator is not zero
   if param_sum_denominator != 0:
      alpha_WA = param_sum_numerator / param_sum_denominator
   else:
      alpha_WA = pd.Series([None] * len(alpha_final.columns[:-1]), index=alpha_final.columns[:-1])

   return alpha_WA


# Calculates the error between the measured and calculated response spectra using the 1d list of alpha
def calculateErrorBetweenSpectra(df_alpha, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, responseSpectrum_measured, nr_seg_compile, alpha1D = True):
    df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

    if alpha1D == True:
        alpha = df_alpha.tolist()

    for segment in range(nr_seg_compile):

        # if alpha is a df with several rows, such as df_results_alpha_current
        
        if alpha1D == False:
            
            #print(f"in if loop. type of alpha: {type(alpha)}")
            alpha = df_alpha.iloc[segment].tolist()

        error_list = []
        m0 = 0
        timestamp, ws = list(df_ws_segments.items())[segment]
        yaw = df_pos_segments_avg[segment]['hdt']  # 1 value per segment. Heading [deg]
        U = df_speed_segments_avg[segment]['Speed']  # Speed Over Ground [m/s]
        beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
        beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]

        responseSpectrum_calculated_tmp = calculateResponseSpectrum(alpha, ws, segment)

        responses = ['Heave', 'Roll_rad', 'Pitch_rad']

        for response in responses:
            m0 =  trapezoid(responseSpectrum_measured[segment][response],fe) # variance

            measured_spectrum = responseSpectrum_measured[segment][response].values
            calculated_spectrum = responseSpectrum_calculated_tmp[response]

            expression = measured_spectrum - calculated_spectrum
            error = trapezoid(np.abs(expression), fe) / m0
                
            # Integrate the absolute value of the expression over 'fe'. Divide by m0
            error_list.append(error)

        df_error_tmp = pd.DataFrame([error_list], columns=df_error.columns)
        df_error = pd.concat([df_error, df_error_tmp], ignore_index=True)

    # after iterating through all segments
    return df_error

def calculateResponseSpectrum_entire_run(alpha, df_ws_segments, nr_seg_compile):
    responseSpectrum_calculated = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
    alpha = alpha.tolist()
    for segment in range(nr_seg_compile):
        timestamp, ws = list(df_ws_segments.items())[segment]
        responseSpectrum_calculated_tmp = calculateResponseSpectrum(alpha, ws, segment)

         # for first iteration: Create df
        if segment == 0:
            responseSpectrum_calculated = responseSpectrum_calculated_tmp

        # else: update existing df
        else:
            responseSpectrum_calculated = pd.concat([responseSpectrum_calculated, responseSpectrum_calculated_tmp], ignore_index=True)
    
    return responseSpectrum_calculated

# Data

In [ ]:
df_motions, df_pos, df_acc, df_speed = getVesselData()

df_motions['Pitch_rad'] = np.radians(df_motions['Pitch'])
df_motions['Roll_rad'] = np.radians(df_motions['Roll'])

csv_files = None#

df_ws = getWaveSpectra(csv_files)



#print("this is the type of df_ws: {df_ws.dtype}")

#print(df_ws.dtypes)

om0 = df_ws['freq']

# ensure that mu and yaw have corresponding time ranges

#### Preparing and sorting RAO data

In [ ]:
TRF_2d_data, f0_RAO, beta_deg_RAO = getRAOs()

## Apply Filters

### Low-pass filter
 Cut off frequency at 1 hz as the majority of the energy is way below this. 

 Max freq from ws is 0.54 Hz and most of the energy from response spectra is below 0.3 Hz.

 However, the response and transfer functions can be shifted to higher frequencies 

 at forward speeds > 0 for head and following seas. Therefore it can be smart to set 
 
 the cut off frequency with a margin at 1 hz.

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = fs/2 # nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    
    return y

### High- Pass filter

In [ ]:
# Should apply a high-pass filter to remove any physical effetcs caused by winds.
# By inspecting the reponse spectra this can be seen for roll below 0.025

def butter_highpass_filter(data, cutoff, fs, order):
    nyq = fs/2 # nyquist frequency
    normal_cutoff = cutoff / nyq
    # the correct syntax of btype might be highpass
    b, a = butter(order, normal_cutoff, btype='high', analog=False) # order controls the steepness of the filter's slope at the cut off frequency
    y = filtfilt(b, a, data)
    
    return y

### Filter Motion data

In [ ]:
cutoff_low = 1 # [Hz]
cutoff_high = 0.025 # [Hz]
order = 10
sr_motions = SampleRateDF(df_motions)
print(f"this is the sample rate for df_motions: {sr_motions}")

# ensure that they are not referenced to the same object. Changes to df_motions_filtered wont effect the original df
df_motions_filtered = df_motions.copy(deep=True)

for column in df_motions_filtered.columns:
        
        if column != "Time" and column != "Timestamp":
                   
            df_motions_filtered[column] =  butter_highpass_filter(df_motions_filtered[column], cutoff_high, sr_motions, order)

            df_motions_filtered[column] =  butter_lowpass_filter(df_motions_filtered[column], cutoff_low, sr_motions, order)

## Response Spectrum

In [ ]:
# works only with one dataframe at a time
def responseSpectrum(df_motions, nperseg):
    # 
    ## Perform the Fast Fourier Transform (FFT). From: https://pythonnumericalmethods.berkeley.edu/notebooks/chapter24.04-FFT-in-Python.html
    # think i noticed a mistake where the response_s
    columns_motions =  ['Heave', 'Heave_velocity', 'Pitch', 'Pitch_rad', 'Roll', 'Roll_rad']
    df_motions_spectra = pd.DataFrame(columns = columns_motions)

    #columns_list = list(df_motions.columns)
    sampling_rate = SampleRateDF(df_motions)
    
    for column in df_motions.columns:
        
        if column != "Time" and column != "Timestamp":
                   
            motion_data = df_motions[column].values
            index_onesided = (nperseg // 2) + 1
            df_motions_spectra = df_motions_spectra.iloc[:index_onesided]
            # return twosided so that the size of the output dataframe is the same as the input dataframe
            freqs, psd = signal.welch(motion_data, fs=sampling_rate, window='hann',nperseg=nperseg, noverlap = nperseg // 2, scaling='density')

            df_motions_spectra[column] = psd

    # set the index of the df to be the encounter frequency
    df_motions_spectra['freq'] = freqs
    #df_motions_spectra = df_motions_spectra.set_index('freq')
    return freqs, df_motions_spectra

In [ ]:
# returns encounter frequency and list of response spectra
def getListResponseSpectrum(df_motions, nperseg, plot = False, save = False):

    listResponseSpectrum = []

    # frequencies should be the same no matter the motion segment.
    # Encounte freqs should only depend on sampling rate and nperseg
    fe = None

    counter = 0
    for motion_segment in df_motions:
        freq, df_responseSpectrum = responseSpectrum(motion_segment, nperseg)
        listResponseSpectrum.append(df_responseSpectrum)

        if counter == 0:
            fe = freq
            responseSpectrum_first = listResponseSpectrum[0]
            if plot:

                plotResponseSpectrum(fe, responseSpectrum_first, title = f'ResponseSpectrum_segment_0', save = save)

        counter += 1
    return fe, listResponseSpectrum

nperseg = 2048

fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg, plot = True, save = False)

fe_unfiltered, responseSpectrum_measured_unfiltered = getListResponseSpectrum(df_motions_segments_unfiltered, nperseg, plot = True, save = False)

#### Validation of measured response spectrum

In [ ]:
df_motions_first = df_motions_segments[0]

variance_heave_time_series = df_motions_first['Heave'].var()
#print("Variance in Heave:", variance_heave_time_series)

# calculate the 0th spectral moment of the response spectrum of segment 0 
responseSpectrum_measured_first = responseSpectrum_measured[0]
m0 = trapezoid(responseSpectrum_measured_first['Heave'],fe) # variance


difference = np.abs(variance_heave_time_series - m0)

fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg, plot = False, save = False)

In [ ]:

def saveSVGRoll(title, folder_path):
    """
    Saves the plot as an SVG in a specified folder under the current directory.
    Automatically appends the current date to organize the files.

    :param title: Title and base name for the file to be saved.
    :param folder_path: Relative path from the current directory where the file should be saved.
                        
    """
    # Generate a unique filename using the current date and time
    current_time = datetime.now().strftime("%H%M")
    filename = f"{current_time}_"

    # Generate today's date string in the format YYYY-MM-DD
    date_string = datetime.now().strftime('%Y-%m-%d')

    # Create the new directory path by appending the date string
    date_folder_path = os.path.join(folder_path, date_string)

    # Check if the directory exists, if not, create it
    if not os.path.exists(date_folder_path):
        os.makedirs(date_folder_path)
    
    # Append hour and minute to the folder path
    hour_minute_folder_path = os.path.join(date_folder_path, current_time)

    # Check if the hour-minute directory exists, if not, create it
    if not os.path.exists(hour_minute_folder_path):
        os.makedirs(hour_minute_folder_path)

    title = filename + title

    # Define the file path including the name of the SVG file you want to save
    file_path = os.path.join(hour_minute_folder_path, title + '.svg')

    # Save the plot as an SVG file in the specified directory
    plt.savefig(file_path)
    print(f"Plot saved as {file_path}")

def plotResponseSpectrumRoll(freq, df, title, save=False):
  # Filter frequencies to only include those less than 0.3 Hz
  freq = freq[freq < 0.3]
  indices = len(freq)
  df = df.iloc[:indices]

  # Initialize a single plot
  fig, ax = plt.subplots(figsize=(8, 4))

  # Plotting "Roll" spectrum
  ax.plot(freq, np.abs(df['Roll_rad']), label='Roll', color='b')
  ax.set_xlabel("Frequency [Hz]")
  ax.set_ylabel("Roll [Rad**2/Hz]")
  ax.set_title('Roll Response Spectrum')
  ax.grid(True)

  # Set the overall figure title
  #fig.suptitle(title, fontsize=16)

  # Option to save the plot as SVG
  if save:
      folder_path = 'Plots/ResponseSpectra/Roll'
      saveSVGRoll(title, folder_path)  # Ensure saveSVG is a defined function or replace this with actual saving code

  # Show the plot
  plt.show()

In [ ]:
nperseg = 2048
df_motions_first = df_motions_segments_unfiltered[0]
freq, df_responseSpectrum = responseSpectrum(df_motions_first, nperseg)


plotResponseSpectrumRoll(freq, df_responseSpectrum, title = f'ResponseSpectrum_segment_unfiltered', save = False)

In [ ]:
nperseg = 2048
df_motions_first = df_motions_segments[0]
freq, df_responseSpectrum = responseSpectrum(df_motions_first, nperseg)


plotResponseSpectrumRoll(freq, df_responseSpectrum, title = f'ResponseSpectrum_segment_filtered', save = False)

# Optimization

## Parameters

In [ ]:
#for df in df_motions_segments:
df_motions_first = df_motions_segments[0]
df_motions_last = df_motions_segments[-1]

nsegments = len(df_motions_segments)

# constants
f0 = om0  # [Hz]
nf0 = len(f0)
om0_rad = f0 * 2* np.pi # [rad/s]

#### Vessel Dimensions.

Actual values is not available for public. None is used instead.

In [ ]:
A_WP = None # Waterplane area
L = None     # Length [m]
B = None        # Breadth [m]
T = None      # Draught [m]
C_WP = A_WP / (L* B)      # Waterplane area coefficient [-]. Guessed value
GM_T = None     # Given Metacentric height [m]
delta = 0.5     # Hull geometry parameter [-]. Guessed value
tau = 0.1       # ratio of viscous roll damping to critical damping [-]. Guessed value
d_IMU = 5       # Distance between IMU position and centre of flotation [m]. Guessed value

alpha_init = [L, B, T, C_WP, GM_T, delta, tau, d_IMU] # parameters which are to be optimized

# Constants
Nabla = None   # Volume Deplacement

# Starting value for variable according to alpha
C_B = Nabla / (alpha_init[0] * alpha_init[1] * alpha_init[2])      # Block coefficient

varbound = np.array([
[(1/2)*L, (3/2)*L],   # L
[(1/2)*B, (3/2)*B],   # B
[(1/2)*T, (3/2)*T],   # T
[(1/2)*C_WP, (3/2)*C_WP],     # C_WP
[0.1, 2*B],   # GM_T
[0.1, 0.99],     # delta. if it is 1 a singularity occurs
[0.1, 1],     # tau
[5, 20]    # d_IMU
])


In [ ]:
def savePolarPlot(segment, save):
    title_file = f'PolarPlot_{segment}'
    if (save):
            folder_path = 'Plots/WaveSpectrum/PolarPlots'
            saveSVG(title_file, folder_path)

    # Function to interpolate rows

def plotWS1D(f0, freq_intrp, mu_deg, beta_TRF, ws_2d_mu, ws_2d_intrp):

    # Custom order for beta_TRF: Start from -4, loop to end, then from start to -5
    start_index = -4 % len(beta_TRF)
    end_index = -5 % len(beta_TRF)
    order_beta = list(range(start_index, len(beta_TRF))) + list(range(0, end_index + 1))

    # Regular order for mu_deg
    order_mu = range(len(mu_deg))
    
    # Iterate through the number of directions in mu_deg
    for i in range(len(mu_deg)):
        plt.figure(figsize=(10, 4))

        # Index for mu_deg and beta_TRF (beta_TRF uses the custom order)
        idx_mu = i
        idx_beta = order_beta[i % len(order_beta)]  # Use modulo to wrap around the custom beta_TRF order

        max_freq = f0[-1]

        # Plot for original data
        #plt.subplot(1, 2, 1)
        plt.plot(f0, ws_2d_mu[:, idx_mu], label='Original Data')
        plt.title(f'Original Data at {mu_deg[idx_mu]:.2f} degrees')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Wave Spectrum Energy (m^2/Hz)')
        plt.legend()
        plt.xlim(0, max_freq)

        plt.tight_layout()
        plt.show()

# this is the correct sorted version
def plotContourPlot(mu_deg, beta_TRF, f0, freq_intrp, ws_2d_mu, ws_2d_beta, title_file, save = False):
    # Plot original and interpolated data
    f0_rad = f0 * 2*np.pi
    freq_intrp_rad = freq_intrp *2 * np.pi
 
    plt.figure(figsize=(8, 6))
    #plt.subplot(1, 2, 1)
    plt.contourf(mu_deg, f0, ws_2d_mu, levels=50, cmap='viridis')
    #plt.title('Original Data')
    plt.xlabel(r'Direction $[\degree]$')
    plt.ylabel('Encounter Frequency [Hz]')
    cbar = plt.colorbar()
    cbar.set_label(r'$[m^2 / Hz]$', rotation=270, labelpad=15)  # labelpad is the space between the label and the colorbar
 
    # have to sort the ws_2d_beta relative to the directions
   
    # Get the indices that would sort beta_TRF
    sorted_indices = np.argsort(beta_TRF)
 
    # Use these indices to sort the columns of ws_2d_intrp
    sorted_ws_2d_beta = ws_2d_beta[:, sorted_indices]

    plt.tight_layout()

    if (save):
        folder_path = 'Plots/WaveSpectrum/ContourPlots'
        saveSVGContour(title_file, folder_path)
   
    plt.show()

def plotPolarPlot(mu_deg, beta_TRF, f0, freq_intrp, ws_2d_mu, ws_2d_beta, savePolar):
    
    # Assuming f0, mu_deg, beta_TRF, ws_2d_mu, ws_2d_beta are defined and available
    mu_rad = np.deg2rad(mu_deg)  # Convert degrees to radians for polar plot
    beta_rad = np.deg2rad(beta_TRF)

    # Append the first angle plus 360 degrees to close the loop visually
    mu_rad = np.append(mu_rad, mu_rad[0] + 2*np.pi)
    ws_2d_mu_polar = np.hstack((ws_2d_mu, ws_2d_mu[:, [0]]))

    ws_2d_beta_polar = np.hstack((ws_2d_beta, ws_2d_beta[:, [0]]))
    beta_rad = np.append(beta_rad, beta_rad[0] + 2*np.pi)

    # Create meshgrids for polar coordinates
    R_mu, Theta_mu = np.meshgrid(f0, mu_rad)  # R is frequency, Theta is direction in radians
    R_beta, Theta_beta = np.meshgrid(freq_intrp, beta_rad)

    # Create a figure for polar plots
    fig = plt.figure(figsize=(14, 12))

    # Plot original data using pcolormesh
    ax1 = plt.subplot(1, 2, 1, projection='polar')
    contour1_f = ax1.contourf(Theta_mu, R_mu, ws_2d_mu_polar.T, levels=50, cmap='viridis')

    ax1.set_title('Original Spectrum') 
    cbar = plt.colorbar(contour1_f, ax=ax1, label=r'$m^2 s/rad$', fraction=0.046, pad=0.04)

    ax1.tick_params(axis='y', colors='white')  # 'y' axis for radial ticks   


    # Plot interpolated data in polar coordinates
    ax2 = plt.subplot(1, 2, 2, projection='polar')
    contour2_f = ax2.contourf(Theta_beta, R_beta, ws_2d_beta_polar.T, levels=50, cmap='viridis')
 

    ax2.set_title('Interpolated Spectrum')     

    cbar2 = plt.colorbar(contour2_f, ax=ax2, label=r'$m^2 s/rad$', fraction=0.046, pad=0.04)

    # Change radial tick label colors to white
    ax2.tick_params(axis='y', colors='white')  # 'y' axis for radial ticks

    plt.tight_layout()  # Adjust layout to make room for the main title
    savePolarPlot(segment, save = savePolar)
    
    #plt.tight_layout()
    plt.show()

def plotResponseSpectrumResult_same_segment_initial_CFE(segment, fe_sliced, responseSpectrum_measured_curr_seg_abs, responseSpectrum_calculated,responseSpectrum_calculated_init, title_file, method, save = False):
    # only select the last iteration as it gives the best result
    # Step 1: Trim responseSpectrum_calculated to match the length of responseSpectrum_measured
    n_rows = responseSpectrum_measured_curr_seg_abs.shape[0]
    
    print(f" this is responseSpectrum_measured[segment]'s shape: {responseSpectrum_measured_curr_seg_abs.shape}")
    print(f"this is nrows: {n_rows}")
    #print(responseSpectrum_measured_curr_seg_abs)
    
    responseSpectrum_calculated = responseSpectrum_calculated.tail(n_rows)
    #freq_limit_rad = 0.3 * 2*np.pi
    # Step 2: Create a mask for frequency values less than 0.3
    #mask_freq = f0 < freq_limit_rad
    mask_freq = fe_sliced < 0.3

    # Apply the mask to frequency array to create a filtered frequency array (optional, if needed)
    freq_filtered = fe_sliced[mask_freq]
    #freq_filtered = f0
    #freq_filtered = f0* 2*np.pi
 
    # Apply the mask to both DataFrames


    responseSpectrum_measured_filtered = responseSpectrum_measured_curr_seg_abs[mask_freq].reset_index(drop=True)
    responseSpectrum_calculated_filtered = responseSpectrum_calculated[mask_freq].reset_index(drop=True)
    responseSpectrum_calculated_init_filtered = responseSpectrum_calculated_init[mask_freq].reset_index(drop=True)

    title_fig = f'Response estimation results after using optimization method {method} on segment {segment}'   
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 10))  # Adjusted width
    fig.suptitle(title_fig, fontsize=16)

    # Plotting "Roll"

    linewidth_measured = 2
    linewidth_calculated = 2
    measured_color = 'g'      # green
    calculated_color_init = 'orange'    # Blue
    calculated_color = 'b'    # Blue
    error_color = 'r'         # A bright red for error
    measured_marker = 's'     # Square marker
    calc_marker = 'd'           # diamond marker

    # Titles for each subplot
    titles = ['Heave', 'Roll', 'Pitch']
    y_labels = ['Heave [m**2/Hz]', 'Roll [rad**2/Hz]', 'Pitch [rad**2/Hz]']

    # Data keys for each plot, assuming these are column names in your DataFrames
    data_keys = ['Heave', 'Roll_rad', 'Pitch_rad']

    # Iterate over the axes, titles, and data keys to create each subplot
    for ax, title, y_label, key in zip(axes, titles, y_labels, data_keys):
        # Plot measured data
        
        ax.plot(freq_filtered, np.abs(responseSpectrum_measured_filtered[key]), 
                label='Measured', color=measured_color, linewidth=linewidth_measured, 
                markersize=4, marker=measured_marker)
        
        ax.plot(freq_filtered, np.abs(responseSpectrum_calculated_init_filtered[key]), 
                label='Calculated with initial parameters', color=calculated_color_init, linestyle='-', 
                linewidth=linewidth_calculated, markersize=4, marker=calc_marker)

        # Plot calculated data
        ax.plot(freq_filtered, np.abs(responseSpectrum_calculated_filtered[key]), 
                label='Calculated with optimized parameters', color=calculated_color, linestyle='-', 
                linewidth=linewidth_calculated)
    

        # Set titles and labels
        ax.set_title(title)
        ax.set_xlabel("Encounter Frequency [Hz]")
        ax.set_ylabel(y_label)

                # Adjust the legend properties to make it smaller
        legend = ax.legend(
        fancybox=True, 
        fontsize=10, 
        title_fontsize=12, 
        loc="upper right", 
        framealpha=0.5, 
        borderpad=0.5,  # Smaller border padding
        labelspacing=0.5,  # Smaller label spacing
        handlelength=2  # Shorter handles
        )
        #legend.set_title(legend_title)
        legend.get_frame().set_edgecolor('black')  # Set the legend frame edge color
        plt.setp(legend.get_texts(), fontsize='10')  # Adjust font size of legend text

        #ax.legend()
        ax.grid(True)

    if (save):
        folder_path = 'Plots/Results'
        saveSVG(title_file, folder_path, method)

    # Adjust layout for a cleaner look
    plt.tight_layout()

    plt.show()

    return

def plotResponseSpectrumResult_same_segment_initial_CFE_ALL(segment, fe_sliced, responseSpectrum_measured_curr_seg_abs, responseSpectrum_calculated,responseSpectrum_calculated_init, title_file, method, save = False):
    # only select the last iteration as it gives the best result
    # Step 1: Trim responseSpectrum_calculated to match the length of responseSpectrum_measured
    n_rows = responseSpectrum_measured_curr_seg_abs.shape[0]

    
    responseSpectrum_calculated = responseSpectrum_calculated.tail(n_rows)

    mask_freq = fe_sliced < 0.3

    # Apply the mask to frequency array to create a filtered frequency array (optional, if needed)
    freq_filtered = fe_sliced[mask_freq]

 
    # Apply the mask to both DataFrames


    responseSpectrum_measured_filtered = responseSpectrum_measured_curr_seg_abs[mask_freq].reset_index(drop=True)
    responseSpectrum_calculated_filtered = responseSpectrum_calculated[mask_freq].reset_index(drop=True)
    responseSpectrum_calculated_init_filtered = responseSpectrum_calculated_init[mask_freq].reset_index(drop=True)
    
    #print(f"this is segment array after masking freq: {segment_array}")
    #responseSpectrum_calculated = responseSpectrum_calculated[segment].loc[mask]
    title_fig = f'Response estimation results after using optimization method {method} on segment {segment}'   
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 10))  # Adjusted width
    fig.suptitle(title_fig, fontsize=16)
    #plt.style.use('seaborn-dark-palette')

    #axes.set_color_cycle(sns.color_palette("coolwarm_r",num_lines))

    # Plotting "Roll"
    #df_motions["Roll"].plot(ax=axes[0,0], legend=True, grid=True, title="Roll")
    linewidth_measured = 2.5
    linewidth_calculated = 2.0
    measured_color = 'g'      # green
    calculated_color_init = 'orange'    # Blue
    calculated_color = 'b'    # Blue
    error_color = 'r'         # A bright red for error
    measured_marker = 's'     # Square marker
    calc_marker = 'd'           # diamond marker

    # Titles for each subplot
    titles = ['Heave', 'Roll', 'Pitch']
    y_labels = ['Heave [m**2/Hz]', 'Roll [rad**2/Hz]', 'Pitch [rad**2/Hz]']

    # Data keys for each plot, assuming these are column names in your DataFrames
    data_keys = ['Heave', 'Roll_rad', 'Pitch_rad']
    step = 3

    # Iterate over the axes, titles, and data keys to create each subplot
    for ax, title, y_label, key in zip(axes, titles, y_labels, data_keys):
        # Plot measured data
        
        ax.plot(freq_filtered[::step], np.abs(responseSpectrum_measured_filtered[key][::step]), 
                label='Measured', color=measured_color, linewidth=linewidth_measured, 
                markersize=4, marker=measured_marker)
        
        ax.plot(freq_filtered[::step], np.abs(responseSpectrum_calculated_init_filtered[key][::step]), 
                label='Calculated with initial parameters', color=calculated_color_init, linestyle='--', 
                linewidth=linewidth_calculated, markersize=3, marker=calc_marker)

        # Plot calculated data
        ax.plot(freq_filtered[::step], np.abs(responseSpectrum_calculated_filtered[key][::step]), 
                label='Calculated with optimized parameters', color=calculated_color, linestyle='-', 
                linewidth=linewidth_calculated, marker = measured_marker, markersize = 3)
        
        # Set titles and labels
        ax.set_title(title)
        ax.set_xlabel("Encounter Frequency [Hz]")
        ax.set_ylabel(y_label)

                # Adjust the legend properties to make it smaller
        legend = ax.legend(
        fancybox=True, 
        fontsize=10, 
        title_fontsize=12, 
        loc="upper right", 
        framealpha=0.5, 
        borderpad=0.5,  # Smaller border padding
        labelspacing=0.5,  # Smaller label spacing
        handlelength=2  # Shorter handles
        )
        #legend.set_title(legend_title)
        legend.get_frame().set_edgecolor('black')  # Set the legend frame edge color
        plt.setp(legend.get_texts(), fontsize='10')  # Adjust font size of legend text

        #ax.legend()
        ax.grid(True)

    if (save):
        folder_path = 'Plots/Results/AllSegments'
        saveSVG(title_file, folder_path, method)

    # Adjust layout for a cleaner look
    plt.tight_layout()
    plt.show()

    return


def plotResponseSpectrumResult_single_response(response, segment, fe_sliced, responseSpectrum_measured_curr_seg_abs, responseSpectrum_calculated, responseSpectrum_calculated_init, title_file, method, save=False):
    # Dictionary for mapping response to data keys and y labels
    response_mapping = {
        "heave": {"key": "Heave", "label": "Heave [m**2/Hz]", "title": "Heave"},
        "roll": {"key": "Roll_rad", "label": "Roll [rad**2/Hz]", "title": "Roll"},
        "pitch": {"key": "Pitch_rad", "label": "Pitch [rad**2/Hz]", "title": "Pitch"},
    }

    # Validate the response
    if response.lower() not in response_mapping:
        raise ValueError(f"Invalid response type '{response}'. Available responses are: {list(response_mapping.keys())}")

    # Select the data key and label
    data_key = response_mapping[response.lower()]["key"]
    y_label = response_mapping[response.lower()]["label"]
    title = response_mapping[response.lower()]["title"]

    # Trim the calculated spectrum to match the measured spectrum's length
    n_rows = responseSpectrum_measured_curr_seg_abs.shape[0]
    responseSpectrum_calculated = responseSpectrum_calculated.tail(n_rows)

    # Create a mask for frequency values less than 0.3 Hz
    mask_freq = fe_sliced < 0.3
    freq_filtered = fe_sliced[mask_freq]

    # Apply the mask to both measured and calculated dataframes
    responseSpectrum_measured_filtered = responseSpectrum_measured_curr_seg_abs[mask_freq].reset_index(drop=True)
    responseSpectrum_calculated_filtered = responseSpectrum_calculated[mask_freq].reset_index(drop=True)
    responseSpectrum_calculated_init_filtered = responseSpectrum_calculated_init[mask_freq].reset_index(drop=True)

    # Set plot parameters
    linewidth_measured = 2.5
    linewidth_calculated = 2.5
    measured_color = 'g'      # Green
    calculated_color_init = 'orange'    # Orange
    calculated_color = 'b'    # Blue
    measured_marker = 's'     # Square marker
    calc_marker = 'd'         # Diamond marker
    step = 3
    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(freq_filtered[::step], np.abs(responseSpectrum_measured_filtered[data_key][::step]), 
            label='Measured', color=measured_color, linewidth=linewidth_measured, 
            markersize=4, marker=measured_marker)
    ax.plot(freq_filtered[::step], np.abs(responseSpectrum_calculated_init_filtered[data_key][::step]), 
            label='Calculated with initial parameters', color=calculated_color_init, linestyle='--', 
            linewidth=linewidth_calculated, markersize=4, marker=calc_marker)
    ax.plot(freq_filtered[::step], np.abs(responseSpectrum_calculated_filtered[data_key][::step]), 
            label='Calculated with optimized parameters', color=calculated_color, linestyle='-', 
            linewidth=linewidth_calculated, marker = measured_marker, markersize = 4)

    # Set titles and labels
    title_fig = f'Response estimation results using {method} on segment {segment}'
    ax.set_title(title_fig, fontsize=14)
    ax.set_xlabel("Encounter Frequency [Hz]")
    ax.set_ylabel(y_label)

    # Add legend
    legend = ax.legend(
        fancybox=True, fontsize=10, title_fontsize=12, loc="upper right", 
        framealpha=0.5, borderpad=0.5, labelspacing=0.5, handlelength=2
    )
    legend.get_frame().set_edgecolor('black')
    plt.setp(legend.get_texts(), fontsize='10')

    # Add grid and adjust layout
    ax.grid(True)
    plt.tight_layout()

    # Save the plot if required
    if save:
        folder_path = 'Plots/Results/AllSegments/SingleResponse/'
        saveSVG(title_file, folder_path, method)

    plt.show()

    return


#### Want to interpolate to an increased number of frequency points to get higher accuracy

In [ ]:
def responseSpectrum_measured_abs(responseSpectrum_measured_curr_seg, freq_intrp, fe):

    df_responseSpectrum_abs = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

    # Interpolate to the new frequency points
    
    responses = ['Heave', 'Roll_rad', 'Pitch_rad']

    for response in responses:
        response_abs = np.interp(freq_intrp, fe, responseSpectrum_measured_curr_seg[response])
        df_responseSpectrum_abs[response] = response_abs

    return df_responseSpectrum_abs

def responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs, max_fe=0.5):
    # Limit the fe array to max_fe and apply the step size
    step = 1    # currently not using step
    fe_sliced = fe[fe <= max_fe]

    fe_sliced = np.linspace(fe_sliced[0], fe_sliced[-1], nr_freqs)

    # Determine the number of rows required based on fe_sliced
    num_rows = len(fe_sliced)

    #print(f" this is the number of rows in fe_sliced: {num_rows}")

    # Slice the DataFrame to match the fe_sliced
    df_responseSpectrum_enc = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

    responses = ['Heave', 'Roll_rad', 'Pitch_rad']

    # Interpolate the measured response spectrum to the new encounter frequency. 
    for response in responses:
        response_abs = np.interp(fe_sliced, fe, responseSpectrum_measured_curr_seg[response])
        df_responseSpectrum_enc[response] = response_abs

    return df_responseSpectrum_enc, fe_sliced

def interpolate2dWS(ws_2d_mu, mu_deg, f0, freq_intrp, beta_TRF):
    # add the final
    values_first_column = ws_2d_mu[:,0]
    values_last_column = ws_2d_mu[:,-1]

    values_first_column = values_first_column.reshape(-1, 1)
    values_last_column = values_last_column.reshape(-1, 1)

    # Horizontal stack the first column, original array, and last column
    ws_2d_mu_ext = np.hstack((values_last_column, ws_2d_mu, values_first_column))

    #Then I would have to append these directions to mu_deg and beta_TRF as well
    mu_deg_ext = np.concatenate(([-7.5], mu_deg, [367.5]))

    # Interpolate across directions
    interpolated_ws_dir = np.zeros((len(f0), len(beta_TRF)))
    for i in range(ws_2d_mu_ext.shape[0]):
        interpolated_ws_dir[i, :] = np.interp(beta_TRF, mu_deg_ext, ws_2d_mu_ext[i, :])

    # Interpolate across frequencies
    interpolated_ws = np.zeros((len(freq_intrp), len(beta_TRF)))
    for j in range(len(beta_TRF)):
        interpolated_ws[:, j] = np.interp(freq_intrp, f0, interpolated_ws_dir[:, j])
    
    return interpolated_ws


def plotIntegratedWS(segment, timestamp, freq_intrp, beta_TRF, ws_2d_intrp, title_file , plot = False, save = False):
    # Compute the integrated wave spectrum using the trapezoidal rule

    #print(f"shape of ws_2d_intrp before trapzoid: {ws_2d_intrp.shape}")
 
    #print(f"shape of beta_TRF before trapzoid: {beta_TRF.shape}")
 
    beta_TRF_rad = np.deg2rad(beta_TRF)
    sorted_indices_dirs = np.argsort(beta_TRF_rad)
    sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]

    ws_1d_intrp = trapezoid(sorted_ws_2d_beta_dirs, x = np.sort(beta_TRF_rad), axis=1)


    freq_intrp_rad = freq_intrp * 2* np.pi
 
    if plot:
        # Create a figure with specific dimensions
        plt.figure(figsize=(10, 4))
       
        # Plot the integrated wave spectrum against the interpolated frequency
        plt.plot(freq_intrp, ws_1d_intrp, label='Wave Spectrum', color='blue')
        #plt.title(f'1D Wave Spectrum for segment using trapezoidal: {segment}, {timestamp}')
        plt.xlabel(r'Encounter Frequency $[Hz]$')
        #ylabel = r'$m^2 / Hz$'
        plt.ylabel(r'$[m^2 / Hz]$')
        plt.legend()
        plt.grid(True)  # Optional: add grid for better visualization
       
        # Tight layout for neatness
        plt.tight_layout()

        # Save the plot if required
        if save:
            folder_path = 'Plots/WaveSpectra'
            saveSVGWS(title_file, folder_path)
       
        # Display the plot
        plt.show()
 
    return ws_1d_intrp

def print_params(param_values):
    param_names = ["L", "B", "T", "C_WP", "GM_T", "delta", "tau", "d_IMU"]

    for name, value in zip(param_names, param_values):
        print(f"{name} = {value:.3f}")

def print_params_heave(param_values):
    param_names = ["L", "B", "T", "C_WP", "GM_T", "delta", "tau", "d_IMU", 'gain_heave']

    for name, value in zip(param_names, param_values):
        print(f"{name} = {value:.3f}")

def print_params_combined(param_values):
    param_names = ['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2']

    for name, value in zip(param_names, param_values):
        print(f"{name} = {value:.3f}")
    

def interpolateFreq(f0, fe, segment, timestamp, responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False):
    responseSpectrum_measured_curr_seg =  responseSpectrum_measured[segment]

    responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs, max_fe=0.5)
    ws_2d_intrp = interpolate2dWS(ws_2d_mu, mu_deg, f0, fe_sliced, beta_TRF)

    title_file = f'1DWS_{segment}'
    ws_1d_intrp = plotIntegratedWS(segment, timestamp, fe_sliced, beta_TRF, ws_2d_intrp, title_file,  plot = plot, save = False)

    title_file = f'Contourplot_{segment}'
    #plotContourPlot(mu_deg, beta_TRF, f0, fe_sliced, ws_2d_mu, ws_2d_intrp, title_file, save = False)

    return responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced

# Calculates 1D response Spectrum
def PlotTRF(TRF_2d, freq_sliced, response, beta_TRF):
    for i in range(len(beta_TRF)):
        plt.figure(figsize=(10, 4))
        plt.plot(freq_sliced, TRF_2d[:, i], label='TRF Data')
        plt.title(f'TRF_2d for {response} at {beta_TRF[i]:.2f} degrees')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('TRF')
        plt.legend()

        plt.tight_layout()
        plt.show()


def calculateResponsSpectrum_own(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs):

    RspecAbs = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])

    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']  # 1 value per segment. Heading [deg]
    U = df_speed_segments_avg[segment]['Speed']  # Speed Over Ground [m/s]

    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    beta_TRF_rad = np.deg2rad(beta_TRF)

    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)
    fe_sliced_rad = 2* np.pi* (fe_sliced)
    
    responses = ['Roll_rad','Heave',  'Pitch_rad']

    
    A_WP = None # Waterplane area
    Nabla = None   # Volume Deplacement
    C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])
    #alpha[3] = A_WP / (alpha[0] * alpha[1])

    sorted_indices_dirs = np.argsort(beta_TRF_rad)
    sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
    beta_TRF_sorted = np.sort(beta_TRF)
    beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

    for response in responses:
        
        #m0 =  trapezoid(responseSpectrum_measured_ground_truth[response],fe) # variance

        # Calculate the CFEs
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(fe_sliced_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[7]**2) * TRF_roll_tmp**2))

        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF_2d = pitchCF(fe_sliced_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[3] - alpha[5]
            
            TRF_2d = rollCF(fe_sliced_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)
            TRF_roll_tmp = TRF_2d

      
        # for every response: calculate the response spectrum
        integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
        RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)
        RspecAbs[response] = RspecAbs_response

    RspecAbs['segment'] = segment

    return RspecAbs, responseSpectrum_measured_curr_seg_sliced


In [ ]:

    
def prepare_segment_data(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False):
    # This function prepares the data required for each segment
    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']
    U = df_speed_segments_avg[segment]['Speed']


    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    

    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced  = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = True)


    # Plot original and interpolated data
    #plotContourPlot(mu_deg, beta_TRF, f0, fe_sliced, ws_2d_mu, ws_2d_intrp)

    #plotWS1D(f0, fe_sliced, mu_deg, beta_TRF, ws_2d_mu, ws_2d_intrp)
    
    # --------------------------Polar Plot
    #plotPolarPlot(mu_deg, beta_TRF, f0, fe_sliced, ws_2d_mu, ws_2d_intrp, savePolar)

    return fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced

### NM and L-BFGS-B 8 Params

In [ ]:

def run_simulation_single_segment(iterations, segment, alpha_init, varbound, segment_data, param_options, step, nr_freqs, saveInitRS = False):

    # unpack current segment data
    
    fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data
    beta_TRF_rad = np.deg2rad(beta_TRF)

    fe_sliced_rad = 2* np.pi* (fe_sliced)
    #ws_2d_intrp_rad = ws_2d_intrp / (2* np.pi)

    # setting alpha equal to initial conditions for first iteration
    alpha = alpha_init

    RspecAbs_result_init, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)
    
    #RspecAbs_result_init = calculateResponsSpectrum_own(segment, mu_deg, f0, om0_rad, df_pos_segments_avg, df_speed_segments_avg, alpha_init)
    df_responseSpectrum_calculated_init = pd.DataFrame(RspecAbs_result_init, columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
    df_responseSpectrum_calculated = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
    df_results_alpha = pd.DataFrame(columns=['L', 'B', 'T', 'C_WP', 'GM_T', 'delta', 'tau', 'd_IMU', 'Error'])
    
    error = 0
    

    for iteration in range(iterations):
        print(f"current iteration: {iteration}")
        print(f"these are the starting values of alpha: {alpha}")
        count = 0
        # Cost function which is to be minimised.
        def func(alpha):
            nonlocal count
            responses = ['Heave', 'Roll_rad', 'Pitch_rad']
        
            #print(f"In cost function. alpha: {alpha}")
            m0 = {} # variance of response spectrum initialized
            CF = 0 # cost function initialized
            
            
            Nabla = None # Volume Deplacement
            A_WP = None # Waterplane area

            C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])
            #alpha[3] = A_WP / (alpha[0] * alpha[1])

            sorted_indices_dirs = np.argsort(beta_TRF_rad)
            sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
            beta_TRF_sorted = np.sort(beta_TRF)

            for response in responses:
                
                m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response], fe_sliced) # variance of measured response spectrum. Note: is given in abs freq

                # Calculate the CFEs
                if response == "Heave":
                    # transfer function for Heave is in units [m/m]
                    TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
                    TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[7]**2) * TRF_roll_tmp**2))

                if response == "Pitch_rad":
                    # transfer function for Pitch is in units [rad/m]
                    TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)
                if response == "Roll_rad":
                    kappa = alpha[3] - alpha[5]
                    TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)
                    TRF_roll_tmp = TRF_2d

                integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
        
                RspecAbs_response = trapezoid(integrand, x = np.sort(beta_TRF_rad), axis = 1)

                calculated_spectrum = RspecAbs_response

                measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values

                # Difference between measured and calculated spectra
                expression = measured_spectrum - calculated_spectrum
                
                error = np.sqrt(trapezoid(np.abs(expression), fe_sliced) / m0)
                # Integrate the absolute value of the expression over 'fe'. Divide by m0
                CF += error
            #only print every 100th count
            if count % 100 == 0 or count == 0:
                print(f"count: {count}, alpha: {alpha}, error: {CF}")
            count += 1
        
            return CF

         # want to use the previous results of the parameters in the next iteration
        if iteration != 0:
            #print(f"these are the parameters of previous segment finalist alpha: {alpha}")
            alpha = alpha_final[:-1] # slice off the last parameter error.
            print(f"new alpha after using previous iteration best result: {alpha}")

        param_options = {
            'ftol' : 1e-9,
            'gtol' : 1e-9
        }

        #solution = minimize(func, alpha, method="nelder-mead", bounds = varbound, options = param_options) # can use tol = 1e-3
        solution = minimize(func, alpha, method="L-BFGS-B", bounds = varbound, options = param_options) # can use tol = 1e-3

        print(f"this is the solution: {solution}")

        alpha_final = solution['x']
        alpha_final = alpha_final.tolist()

        print("These are the final parameters:")
        print_params(alpha_final)
        RspecAbs_result, _ = calculateResponsSpectrum_own(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final, df_ws_segments, fe,step, nr_freqs)
        #RspecAbs_result_rad = RspecAbs_result / (2*np.pi)

        error = solution['fun']

        print(f"this was the final error of the cost function {error}")
        # Append results to df_results_alpha
        alpha_final.append(error)

    # write results to temporary dfs
        alpha_final_df = pd.DataFrame([alpha_final], columns=df_results_alpha.columns)
        
        # for first iteration: Create df
        if iteration == 0:
            df_results_alpha = alpha_final_df
            df_responseSpectrum_calculated = RspecAbs_result
    
        # else: update existing df
        else:
            df_results_alpha = pd.concat([df_results_alpha, alpha_final_df], ignore_index=True)
            df_responseSpectrum_calculated = pd.concat([df_responseSpectrum_calculated, RspecAbs_result], ignore_index=True)

        #print(f"at segment {segment} the responsespectrum_calculated is : {responseSpectrum_calculated}")

    savetoCSV(df_results_alpha, f"alpha_segment_{segment}_fe_{nr_freqs}_mtd_NM")
    savetoCSV(df_responseSpectrum_calculated, f'responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_NM')
    if saveInitRS:
        savetoCSV(df_responseSpectrum_calculated_init, f'responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}')


    return df_results_alpha, df_responseSpectrum_calculated, df_responseSpectrum_calculated_init

In [ ]:
#nr_freqs = 600

iterations = 1
segment = 0
step = 1
nr_freqs = 1000

segment_data = prepare_segment_data(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

# Set options for the Nelder-Mead optimizer
param_options = {
    'xatol': 1e-6,   # Tolerance for convergence on x
    'fatol': 1e-6,   # Tolerance for convergence on the function value
    'maxiter': 1600*20,  # Maximum number of iterations # standard is N*200 where N is len(alpha) -> standard: 1600. If both maxiter and maxfev are set: will stop at first reached.
    'maxfev': 1600*20    # Maximum number of function evaluations
}
# additional options are:

start_time = time.time()

df_results_alpha, responseSpectrum_calculated, df_responseSpectrum_calculated_init = run_simulation_single_segment(iterations, segment, \
                                                        alpha_init, varbound, segment_data, param_options, step, nr_freqs, saveInitRS = True)

# Capture the end time
end_time = time.time()

# Calculate and print the execution time
print(f"Ran successfully the simulation with {iterations} compiled iterations.")
print(f"Execution time: {(end_time - start_time)/60} minutes")

In [ ]:
def readResults(segment, nr_run, nr_freqs):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  
  # Define the paths to the files
  df_results_alpha_path = f"df_results/alpha_segment_{segment}_fe_{nr_freqs}_mtd_NM_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_NM_iter_{nr_run}.csv"
  responseSpectrum_calculated_init_path = f"responseSpectrumResults/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"
  

  # Read the DataFrames from the CSV files
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  responseSpectrum_calculated_current = pd.read_csv(responseSpectrum_calculated_path)
  responseSpectrum_calculated_init = pd.read_csv(responseSpectrum_calculated_init_path)

  return df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init

segment = 0
nr_run = 1
step = 1
nr_freqs = 1000
df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init = readResults(segment, nr_run, nr_freqs)

title_file = f'After_optm_Response_estimation_results_same_segment_{segment}_scipy_nr_run_{nr_run}'


nperseg = 2048

fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg, plot = False) # this is given in encounter freq

responseSpectrum_measured_curr_seg = responseSpectrum_measured[segment]


responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs)


In [ ]:
title_file = f'After_optm_TRF_results_same_segment_{segment}_fe_{nr_freqs}_mtd_NM'

alpha_init = [L, B, T, C_WP, GM_T, delta, tau, d_IMU] # parameters which are to be optimized

method = 'Nelder Mead'

plotResponseSpectrumResult_same_segment_initial_CFE(segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current,responseSpectrum_calculated_init, title_file, method, save = True)


In [ ]:
def callback_generation(ga_instance):
    generation = ga_instance.generations_completed
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    average_fitness = np.mean([fitness for fitness in ga_instance.last_generation_fitness if fitness is not None])
    worst_fitness = min([fitness for fitness in ga_instance.last_generation_fitness if fitness is not None])
    diversity = len(set(ga_instance.last_generation_fitness))

    print(f"Generation = {generation}")
    print(f"Average Fitness = {average_fitness:.2f}")
    print(f"Best Fitness = {solution_fitness:.4f}")
    print(f"Worst Fitness = {worst_fitness:.2f}")
    print(f"Diversity (unique fitness values) = {diversity}")

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness

    error = 1/ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    #print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]))
    print(f"Error     = {error}")
    print("Change     = {change}".format(change=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1] - last_fitness))
    last_fitness = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]

def plotGA(ga_instance, segment, nr_freqs):

        ga_instance.summary()
        print("plot fitness")
        ga_instance.plot_fitness()

        print("plot new solution rate")
        ga_instance.plot_new_solution_rate()

        print("plot genes")
        #ga_instance.plot_genes(plot_type= "scatter", title = f"gene_plot_segment_{segment}_fe_{nr_freqs}",save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_scatter_segment_{segment}_fe_{nr_freqs}"))
        ga_instance.plot_genes(plot_type= "plot", title = f"gene_plot_segment_{segment}_fe_{nr_freqs}",save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_segment_{segment}_fe_{nr_freqs}"))
        # plot boxplot for every parameter
        ga_instance.plot_genes(graph_type="boxplot", save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_boxplot_segment_{segment}_fe_{nr_freqs}"))


        # slightly implements some variation to the initial population so that not all individuals are the same
def generate_initial_population(base_solution, population_size, variation_range=0.1):
    initial_population = []
    for _ in range(population_size):
        varied_solution = [np.random.uniform(low=max(0, gene - abs(gene * variation_range)),
                                             high=gene + abs(gene * variation_range))
                           for gene in base_solution]
        initial_population.append(varied_solution)
    return initial_population


### 9 params using PyGAD

In [ ]:
def callback_generation(ga_instance):
    generation = ga_instance.generations_completed
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    average_fitness = np.mean([fitness for fitness in ga_instance.last_generation_fitness if fitness is not None])
    worst_fitness = min([fitness for fitness in ga_instance.last_generation_fitness if fitness is not None])
    diversity = len(set(ga_instance.last_generation_fitness))

    print(f"Generation = {generation}")
    print(f"Average Fitness = {average_fitness:.2f}")
    print(f"Best Fitness = {solution_fitness:.4f}")
    print(f"Worst Fitness = {worst_fitness:.2f}")
    print(f"Diversity (unique fitness values) = {diversity}")

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness

    error = 1/ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    #print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]))
    print(f"Error     = {error}")
    print("Change     = {change}".format(change=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1] - last_fitness))
    last_fitness = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]

def plotGA(ga_instance, segment, nr_freqs):

        #ga_instance.summary()
        #print("plot fitness")
        ga_instance.plot_fitness()

        print("plot new solution rate")
        #ga_instance.plot_new_solution_rate()

        print("plot genes")
        #ga_instance.plot_genes(plot_type= "scatter", title = f"gene_plot_segment_{segment}_fe_{nr_freqs}",save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_scatter_segment_{segment}_fe_{nr_freqs}"))
        ga_instance.plot_genes(plot_type= "plot", title = f"gene_plot_segment_{segment}_fe_{nr_freqs}",save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_segment_{segment}_fe_{nr_freqs}"))
        # plot boxplot for every parameter
        ga_instance.plot_genes(graph_type="boxplot", save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_boxplot_segment_{segment}_fe_{nr_freqs}"))


        # slightly implements some variation to the initial population so that not all individuals are the same
def generate_initial_population(base_solution, population_size, variation_range=0.1):
    initial_population = []
    for _ in range(population_size):
        varied_solution = [np.random.uniform(low=max(0, gene - abs(gene * variation_range)),
                                             high=gene + abs(gene * variation_range))
                           for gene in base_solution]
        initial_population.append(varied_solution)
    return initial_population


def prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments):
    return f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments

In [ ]:

def run_simulation_single_segment_for_all_segments_NM_PYGAD_heave(constant_data, df_ws_segments,df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, \
                                                                   alpha_init_heave, varbound_heave, algorithm_param_pygad, saveInitRS = False):

    
    f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments = constant_data
    count = 0

    for segment in range(start_seg, stop_seg):
        print(f"Current segment: {segment}")

        #print(f"these are the starting values of alpha: {alpha}")
        start_time_seg = time.time()

        if count != 0:
            #print(f"these are the parameters of previous segment finalist alpha: {alpha}")
            alpha = alpha_final_pygad[:-1] # slice off the last parameter error.
        else:
            alpha = alpha_init_heave

        alpha = alpha_init_heave # hardcode alpha to be initialized for every start of the segment

        segment_data = prepare_segment_data(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)
        fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data
        beta_TRF_rad = np.deg2rad(beta_TRF)

        fe_sliced_rad = fe_sliced * 2 * np.pi

        RspecAbs_result_init, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_heave(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)
 
        df_responseSpectrum_calculated_init = pd.DataFrame(RspecAbs_result_init, columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_responseSpectrum_calculated = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_results_alpha = pd.DataFrame(columns=['L', 'B', 'T', 'C_WP', 'GM_T', 'delta', 'tau', 'd_IMU', 'gain_heave' ,'Error'])

        sorted_indices_dirs = np.argsort(beta_TRF_rad)
        sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
        beta_TRF_sorted = np.sort(beta_TRF)
        beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

      
        count = 0
        error = 0
        # Cost function which is to be minimised.
        def fitness_func(ga_instance, solution, solution_idx):
            nonlocal count
            responses = ['Roll_rad','Heave',  'Pitch_rad']
        
            alpha = solution
        
            #print(f"In cost function. alpha: {alpha}")
            m0 = {} # variance of response spectrum initialized
            CF = 0 # cost function initialized
            
            Nabla = None  # Volume Deplacement
            A_WP = None # Waterplane area

            C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])
            #alpha[3] = A_WP / (alpha[0] * alpha[1])

            #responseSpectrum_measured_curr_seg_sliced_s_rad = responseSpectrum_measured_curr_seg_sliced / (2* np.pi)
            for response in responses:
                m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response], fe_sliced) # variance of measured response spectrum. Note: is given in abs freq

                # Calculate the CFEs
                if response == "Heave":
                    # transfer function for Heave is in units [m/m]
                    TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
                    TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[7]**2) * TRF_roll_tmp**2))
                    TRF_2d = TRF_2d * alpha[8] # multiply by the gain

                if response == "Pitch_rad":
                        # transfer function for Pitch is in units [rad/m]
                    TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

                if response == "Roll_rad":
                    kappa = alpha[3] - alpha[5]
                    TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)
                    TRF_roll_tmp = TRF_2d
           
                #TRF_2d = TRF_2d / (2 * np.pi) # convert the transfer function from rad/s to Hz
                integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
            
                RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)
            
                calculated_spectrum = RspecAbs_response

                measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values

                # Difference between measured and calculated spectra
                expression = measured_spectrum - calculated_spectrum

                #error = np.sqrt(trapezoid(np.abs(expression), fe_sliced) / m0)
                error = np.sqrt(trapezoid(np.abs(expression), fe_sliced) / m0)
                # Integrate the absolute value of the expression over 'fe'. Divide by m0
                CF += error
  
        
            return 1/CF
        # ---------------after iterating through cost function

         # want to use the previous results of the parameters in the next iteration
        

        # uses GA for finding global optimum
        init_pop =  generate_initial_population(alpha, population_size = algorithm_param_pygad['population_size'], variation_range=0.1)
        
        ga_instance = pygad.GA(num_generations= algorithm_param_pygad['max_num_iteration'],
                               sol_per_pop= algorithm_param_pygad['population_size'],
                               num_parents_mating=algorithm_param_pygad['num_parents_mating'],
                               fitness_func = fitness_func,
                               num_genes=len(alpha),
                               gene_type=float,
                               gene_space=algorithm_param_pygad['gene_space'], # parent_selection_type="rws",
                               mutation_num_genes=algorithm_param_pygad['mutation_num_genes'], #when mutation_type = 'adaptive' has to be set to the following format: [60,30]
                               crossover_probability=algorithm_param_pygad['crossover_probability'],
                               mutation_type="random",
                               mutation_by_replacement=True, # to make sure genes do not go beyond range after mutation. prev true
                               crossover_type=algorithm_param_pygad['crossover_type'], 
                               on_generation=callback_generation,
                               save_solutions = True,
                               initial_population = init_pop,
                               stop_criteria = [f"saturate_{algorithm_param_pygad['max_iteration_without_improv']}"],
                               keep_elitism = algorithm_param_pygad['elit_ratio']
                            
                            )
        
        ga_instance.run()
        solution, solution_fitness, solution_idx  = ga_instance.best_solution()
        plotGA(ga_instance, segment, nr_freqs)

    
        alpha_final_pygad = solution
        alpha_final_pygad = alpha_final_pygad.tolist()
        error = 1/solution_fitness

        print("These are the final parameters after using GA for finding global optimum:")
        print_params_heave(alpha_final_pygad)
        print(f"this is the final error of GA: {error}")

        RspecAbs_result, _ = calculateResponsSpectrum_own_heave(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final_pygad, df_ws_segments, fe,step, nr_freqs)
        #RspecAbs_result_rad = RspecAbs_result / (2*np.pi)
        #error = solution['fun']
        print(f"this was the final error of the cost function using NM: {error}")
        # Append results to df_results_alpha
        alpha_final_pygad.append(error)

        #print(f"Parameters of the best solution : {solution}")
        #print(f"Fitness value of the best solution = {solution_fitness}")
        #print(f"Index of the best solution : {solution_idx}")

    # write results to temporary dfs
        alpha_final_df = pd.DataFrame([alpha_final_pygad], columns=df_results_alpha.columns)

        df_results_alpha = alpha_final_df
        df_responseSpectrum_calculated = RspecAbs_result

        #print(f"at segment {segment} the responsespectrum_calculated is : {responseSpectrum_calculated}")
        savetoCSVAllSegments(df_results_alpha, f"alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_9_param_runs_{algorithm_param_pygad['max_num_iteration']}_same_init_alpha_inc_d_IMU")
        savetoCSVAllSegments(df_responseSpectrum_calculated, f"responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_9_param_runs_{algorithm_param_pygad['max_num_iteration']}_same_init_alpha_inc_d_IMU")
        if saveInitRS:
            savetoCSVAllSegments(df_responseSpectrum_calculated_init, f'responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_9_param_inc_d_IMU')

        end_time_seg = time.time()
        print(f"Ran successfully. Execution time: {(end_time_seg - start_time_seg)/60} minutes")
        count += 1

    #nr_freqs = 1025/step
    # Save results to CSV
   


    return

In [ ]:

def run_simulation_multiple_segments_NM_PYGAD_heave(alpha_init_heave, varbound_heave,constant_data,\
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured,algorithm_param_pygad,saveInitRS = False):
    start_time = time.time()
        
    run_simulation_single_segment_for_all_segments_NM_PYGAD_heave(constant_data, df_ws_segments,df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured,\
                                                        alpha_init_heave, varbound_heave, algorithm_param_pygad, saveInitRS)

    # Capture the end time
    end_time = time.time()

    # Calculate and print the execution time
    print(f"Ran successfully the ssimulation with {nr_segments} compiled segments.")
    print(f"Execution time: {(end_time - start_time)/60} minutes")



### 9 params PYGAD including d_IMU same init alpha

In [ ]:
#nr_freqs = 600
#nr_segments = 2
step = 1
nr_freqs = 1000
start_seg = 24
stop_seg = 136
nr_segments = stop_seg - start_seg

gene_space = [{'low': float(bounds[0]), 'high': float(bounds[1])} for bounds in varbound_heave] # create continous voundaries

# Set options for the Nelder-Mead optimizer
algorithm_param_pygad_4 = {'max_num_iteration': 100, # 1000
                    'population_size': 50, # can try 150 
                    'mutation_num_genes': 1, # prev 0.3
                    'elit_ratio': 2, # prev 0.2
                    'crossover_probability': 0.7,
                    'num_parents_mating': 15, # prev 0.2
                    'crossover_type': 'uniform',
                    'max_iteration_without_improv': 150,
                    'gene_space' : gene_space   
                    }

constant_data = prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments)


run_simulation_multiple_segments_NM_PYGAD_heave(alpha_init_heave, varbound_heave, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, algorithm_param_pygad_4, saveInitRS = True)

In [ ]:
def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  
  # Define the paths to the files
  df_results_alpha_path = f"df_results_ALL/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_9_param_runs_{iterations}_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults_ALL/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_9_param_runs_{iterations}_iter_{nr_run}.csv"

  responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"

  # Read the DataFrames from the CSV files
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  responseSpectrum_calculated_current = pd.read_csv(responseSpectrum_calculated_path)
  responseSpectrum_calculated_init = pd.read_csv(responseSpectrum_calculated_init_path)

  return df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init


def readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq
  for segment in range(start_seg, stop_seg):
    df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init = readResults(segment, nr_run, nr_freqs, iterations)
    
    title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_runs_{iterations}'


    responseSpectrum_measured_curr_seg = responseSpectrum_measured[segment]
    responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs)

    method = 'PyGAD'
    
    plotResponseSpectrumResult_same_segment_initial_CFE_ALL(segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current,responseSpectrum_calculated_init, title_file, method, save = True)

    if response != None:
      title_file = f'After_optm_heave_results_segment_{segment}_fe_{nr_freqs}__resp_{response}_mtd_PYGAD_runs_{iterations}'
      plotResponseSpectrumResult_single_response(response, segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current, responseSpectrum_calculated_init, title_file, method, save=True)

In [ ]:
start_seg = 1
stop_seg = 2
nr_run = 2
response = 'heave'
iterations = 200 # max nr of iterations/ generations

nr_segments = stop_seg - start_seg
step = 1
nr_freqs = 1000
readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments)

### 12 params. Separate tuning parameters for heave

In [ ]:

def calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs):

    RspecAbs = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])

    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']  # 1 value per segment. Heading [deg]
    U = df_speed_segments_avg[segment]['Speed']  # Speed Over Ground [m/s]

    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    beta_TRF_rad = np.deg2rad(beta_TRF)


    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)
    fe_sliced_rad = 2* np.pi* (fe_sliced)

    
    responses = ['Roll_rad','Heave',  'Pitch_rad']

    Nabla = None  # Volume Deplacement
    A_WP = None # Waterplane area

    C_B_1 = Nabla / (alpha[0] * alpha[1] * alpha[2])
    C_B_2 = Nabla / (alpha[4] * alpha[5] * alpha[6])

    #alpha[3] = A_WP / (alpha[0] * alpha[1])

    sorted_indices_dirs = np.argsort(beta_TRF_rad)
    sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
    
    beta_TRF_sorted = np.sort(beta_TRF)
    beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

    for response in responses:
        
        #m0 =  trapezoid(responseSpectrum_measured_ground_truth[response],fe) # variance

        # Calculate the CFEs
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B_1) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[11]**2) * TRF_roll_tmp**2))
            TRF_2d = TRF_2d * alpha[3] # multiply by the gain
            
        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[7] - alpha[9]
            
            TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2,alpha[7],alpha[8],kappa,alpha[10],T_N=0)
            TRF_roll_tmp = TRF_2d

        #PlotTRF(TRF_2d, freq_intrp_rad, response, beta_TRF)
        #TRF_2d = TRF_2d / (2 * np.pi) # convert the transfer function from rad/s to Hz

        # for every response: calculate the response spectrum
        #dx = beta_TRF_rad[1] - beta_TRF_rad[0]
        integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
        
        RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)
        RspecAbs[response] = RspecAbs_response

    RspecAbs['segment'] = segment

    return RspecAbs, responseSpectrum_measured_curr_seg_sliced

In [ ]:
def run_simulation_single_segment_for_all_segments_PYGAD_combined(constant_data, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, alpha_init_combined, varbound_combined, algorithm_param_pygad, saveInitRS = False):
    
    f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments = constant_data
    count = 0
    for segment in range(start_seg, stop_seg):
        start_time_seg = time.time()
        print(f"Current segment: {segment}")

        if count != 0:
            #print(f"these are the parameters of previous segment finalist alpha: {alpha}")
            alpha = alpha_final_pygad[:-1] # slice off the last parameter error.
        else:
            alpha = alpha_init_combined

        alpha = alpha_init_combined

        segment_data = prepare_segment_data(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)
        fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data
        beta_TRF_rad = np.deg2rad(beta_TRF)

        fe_sliced_rad = fe_sliced * 2 * np.pi

        RspecAbs_result_init, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)
 
        df_responseSpectrum_calculated_init = pd.DataFrame(RspecAbs_result_init, columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_responseSpectrum_calculated = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_results_alpha = pd.DataFrame(columns=['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2', 'Error'])
       
        
        sorted_indices_dirs = np.argsort(beta_TRF_rad)
        sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
        beta_TRF_sorted = np.sort(beta_TRF)
        beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

        error = 0

        #print(f"these are the starting values of alpha: {alpha}")
        count = 0
        # Cost function which is to be minimised.
        def fitness_func(ga_instance, solution, solution_idx):
            nonlocal count
            responses = ['Roll_rad','Heave',  'Pitch_rad']
            alpha = solution
            #print(f"In cost function. alpha: {alpha}")
            m0 = {} # variance of response spectrum initialized
            CF = 0 # cost function initialized
            
            Nabla = None  # Volume Deplacement
            A_WP = None# Waterplane area

            C_B_1 = Nabla / (alpha[0] * alpha[1] * alpha[2])
            C_B_2 = Nabla / (alpha[4] * alpha[5] * alpha[6])
        
            for response in responses:
                
                m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response], fe_sliced) # variance of measured response spectrum. Note: is given in abs freq

                # Calculate the CFEs
                if response == "Heave":
                    # transfer function for Heave is in units [m/m]
                    TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B_1) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
                    TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[11]**2) * TRF_roll_tmp**2))
                    TRF_2d = TRF_2d * alpha[3] # multiply by the gain
                    
                if response == "Pitch_rad":
                    # transfer function for Pitch is in units [rad/m]
                    TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

                if response == "Roll_rad":
                    kappa = alpha[7] - alpha[9]
                    TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2,alpha[7],alpha[8],kappa,alpha[10],T_N=0)
                    TRF_roll_tmp = TRF_2d


                integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
                RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)

                calculated_spectrum = RspecAbs_response

                measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values

                # Difference between measured and calculated spectra
                expression = measured_spectrum - calculated_spectrum
                
                error = np.sqrt(trapezoid(np.abs(expression), fe_sliced) / m0)
                # Integrate the absolute value of the expression over 'fe'. Divide by m0
                CF += error
            #only print every 100th count
            #if count % 100 == 0 or count == 0:
                #print(f"count: {count}, alpha: {alpha}, error: {CF}")
            #count += 1
        
            return 1/CF
        # ---------------after iterating through cost function
        # generate init population for new alpha
        #init_population = generate_initial_population(alpha, population_size = algorithm_param_pygad['population_size'], variation_range=0.1)
        #init_population =  generate_initial_population(alpha, algorithm_param_pygad['population_size'], varbound_combined,variation_range=0.1)
        init_population  = generate_initial_population(alpha_init_combined, population_size = 50, variation_range=0.1)

        ga_instance = pygad.GA(num_generations= algorithm_param_pygad['max_num_iteration'],
                               sol_per_pop= algorithm_param_pygad['population_size'],
                               num_parents_mating=algorithm_param_pygad['num_parents_mating'],
                               fitness_func = fitness_func,
                               num_genes=len(alpha),
                               gene_type=float,
                               gene_space=algorithm_param_pygad['gene_space'], # parent_selection_type="rws",
                               mutation_num_genes=algorithm_param_pygad['mutation_num_genes'], #when mutation_type = 'adaptive' has to be set to the following format: [60,30]
                               crossover_probability=algorithm_param_pygad['crossover_probability'],
                               mutation_type="random",
                               mutation_by_replacement=True, # to make sure genes do not go beyond range after mutation. prev true
                               crossover_type=algorithm_param_pygad['crossover_type'], 
                               on_generation=callback_generation,
                               save_solutions = True,
                               initial_population = init_population,
                               stop_criteria = [f"saturate_{algorithm_param_pygad['max_iteration_without_improv']}"],
                               keep_elitism = algorithm_param_pygad['elit_ratio']
                            )
        
        ga_instance.run()
        solution, solution_fitness, solution_idx  = ga_instance.best_solution()
        plotGA(ga_instance)

        alpha_final_pygad = solution
        alpha_final_pygad = alpha_final_pygad.tolist()
        error = 1/solution_fitness

        print("These are the final parameters after using GA for finding global optimum:")
        print_params_combined(alpha_final_pygad)
        print(f"this is the final error of GA: {error}")
        #RspecAbs_result, _ = calculateResponsSpectrum_own(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final, df_ws_segments, fe,step, nr_freqs)
        RspecAbs_result, _ =  calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final_pygad, df_ws_segments, fe,step, nr_freqs)
        #RspecAbs_result_rad = RspecAbs_result / (2*np.pi)

        print(f"this was the final error of the cost function {error}")
        # Append results to df_results_alpha
        alpha_final_pygad.append(error)

    # write results to temporary dfs
        alpha_final_df = pd.DataFrame([alpha_final_pygad], columns=df_results_alpha.columns)

        df_results_alpha = alpha_final_df
        df_responseSpectrum_calculated = RspecAbs_result
 
        # Save results to CSV
        savetoCSVAllSegmentsCombined(df_results_alpha, f"alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{algorithm_param_pygad['max_num_iteration']}_same_init_alpha_inc_d_IMU")
        savetoCSVAllSegmentsCombined(df_responseSpectrum_calculated, f"responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{algorithm_param_pygad['max_num_iteration']}_same_init_alpha_inc_d_IMU")
        if saveInitRS:
            #savetoCSVAllSegmentsCombined(df_responseSpectrum_calculated_init, f'responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}')
            savetoCSVAllSegmentsCombined(df_responseSpectrum_calculated_init, f'responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_same_init_alpha')


        end_time_seg = time.time()
        print(f"Ran successfully. Execution time: {(end_time_seg - start_time_seg)/60} minutes")

        count += 1

    return

In [ ]:

def run_simulation_multiple_segments_PYGAD_combined(alpha_init_combined, varbound_combined, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, param_options,saveInitRS = False):

    start_time = time.time()
    run_simulation_single_segment_for_all_segments_PYGAD_combined(constant_data, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, alpha_init_combined, varbound_combined,param_options, saveInitRS)

    # Capture the end time
    end_time = time.time()

    # Calculate and print the execution time
    print(f"Ran successfully the simulation with {nr_segments} compiled segments.")
    print(f"Execution time: {(end_time - start_time)/60} minutes")

    

In [ ]:
algorithm_param_pygad_4 = {'max_num_iteration':100, # 1000
                    'population_size': 50, #  150 
                    'mutation_num_genes': 1, # prev 0.3
                    'elit_ratio': 2, # prev 0.2
                    'crossover_probability': 0.7,
                    'num_parents_mating': 15, # prev 0.2
                    'crossover_type': 'uniform',
                    'max_iteration_without_improv': 150,
                    'gene_space' : gene_space   
                    }

random.seed(42)
np.random.seed(42)

constant_data = prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments)

run_simulation_multiple_segments_PYGAD_combined(alpha_init_combined, varbound_combined, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, algorithm_param_pygad_4, saveInitRS = False)

In [ ]:
def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  
  # Define the paths to the files
  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_iter_{nr_run}.csv"
  responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"

  # Read the DataFrames from the CSV files
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  responseSpectrum_calculated_current = pd.read_csv(responseSpectrum_calculated_path)
  responseSpectrum_calculated_init = pd.read_csv(responseSpectrum_calculated_init_path)

  return df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init


def readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq
  for segment in range(start_seg, stop_seg):
    df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init = readResults(segment, nr_run, nr_freqs, iterations)
    
    title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_runs_{iterations}'

    responseSpectrum_measured_curr_seg = responseSpectrum_measured[segment]
    ic(len(responseSpectrum_measured_curr_seg))
    responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs)
    ic(len(responseSpectrum_measured_curr_seg_sliced))
    ic(len(fe_sliced))

    method = 'PyGAD'

    plotResponseSpectrumResult_same_segment_initial_CFE_ALL(segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current,responseSpectrum_calculated_init, title_file, method, save = True)

    if response != None:
      title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_resp_{response}_mtd_PYGAD_runs_{iterations}'
      plotResponseSpectrumResult_single_response(response, segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current, responseSpectrum_calculated_init, title_file, method, save=True)

In [ ]:
start_seg = 0
stop_seg = 136
nr_run = 1
response = None
iterations = 800 # max number of iterations

nr_segments = stop_seg - start_seg
step = 1
nr_freqs = 1000
readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments)

#### L-BFGS-B 12 param

In [ ]:
def run_simulation_single_segment_for_all_segments_LBFGS_combined(constant_data, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, alpha_init_combined, varbound_combined, algorithm_param_pygad, saveInitRS = False):
    
    f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments = constant_data
    count = 0
    for segment in range(start_seg, stop_seg):
        start_time_seg = time.time()
        print(f"Current segment: {segment}")

        if count != 0:
            #print(f"these are the parameters of previous segment finalist alpha: {alpha}")
            alpha = alpha_final[:-1] # slice off the last parameter error.
        else:
            alpha = alpha_init_combined

        alpha = alpha_init_combined

        segment_data = prepare_segment_data(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)
        fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data
        beta_TRF_rad = np.deg2rad(beta_TRF)

        fe_sliced_rad = fe_sliced * 2 * np.pi

        RspecAbs_result_init, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)
 
        df_responseSpectrum_calculated_init = pd.DataFrame(RspecAbs_result_init, columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_responseSpectrum_calculated = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_results_alpha = pd.DataFrame(columns=['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2', 'Error'])
        
        sorted_indices_dirs = np.argsort(beta_TRF_rad)
        sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
        beta_TRF_sorted = np.sort(beta_TRF)
        beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

        error = 0

        #print(f"these are the starting values of alpha: {alpha}")
        count = 0
        # Cost function which is to be minimised.
        def func(alpha):
            nonlocal count
            responses = ['Roll_rad','Heave',  'Pitch_rad']
            #print(f"In cost function. alpha: {alpha}")
            m0 = {} # variance of response spectrum initialized
            CF = 0 # cost function initialized
            
            Nabla = None   # Volume Deplacement
            A_WP = None # Waterplane area

            C_B_1 = Nabla / (alpha[0] * alpha[1] * alpha[2])
            C_B_2 = Nabla / (alpha[4] * alpha[5] * alpha[6])

            for response in responses:
                
                m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response], fe_sliced) # variance of measured response spectrum. Note: is given in abs freq

                # Calculate the CFEs
                if response == "Heave":
                    # transfer function for Heave is in units [m/m]
                    TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B_1) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
                    TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[11]**2) * TRF_roll_tmp**2))

                    TRF_2d = TRF_2d * alpha[3] # multiply by the gain
                    
                if response == "Pitch_rad":
                    # transfer function for Pitch is in units [rad/m]
                    TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

                if response == "Roll_rad":
                    kappa = alpha[7] - alpha[9]
                    TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2,alpha[7],alpha[8],kappa,alpha[10],T_N=0)
                    TRF_roll_tmp = TRF_2d

                integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
                
                RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)

                calculated_spectrum = RspecAbs_response

                measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values

                # Difference between measured and calculated spectra
                expression = measured_spectrum - calculated_spectrum
                
                error = np.sqrt(trapezoid(np.abs(expression), fe_sliced) / m0)
                # Integrate the absolute value of the expression over 'fe'. Divide by m0
                CF += error
            #only print every 100th count
            #if count % 100 == 0 or count == 0:
                #print(f"count: {count}, alpha: {alpha}, error: {CF}")
            #count += 1
        
            return CF
        # ---------------after iterating through cost function
        # generate init population for new alpha

        param_options_BFGS = {
            'ftol' : 1e-9,
            'gtol' : 1e-9
        }

        param_options_NM = {
            'xatol': 1e-9,   # Tolerance for convergence on x
            'fatol': 1e-9,   # Tolerance for convergence on the function value
            'maxiter': 32*1e3,  # Maximum number of iterations # standard is N*200 where N is len(alpha) -> standard: 1600. If both maxiter and maxfev are set: will stop at first reached.
            'maxfev': 32*1e3    # Maximum number of function evaluations
        }

        #solution = minimize(func, alpha, method="L-BFGS-B", bounds = varbound_combined, options = param_options_BFGS) # can use tol = 1e-3
        solution = minimize(func, alpha, method="nelder-mead", bounds = varbound_combined, options = param_options_NM) # can use tol = 1e-3

        alpha_final = solution['x']
        alpha_final = alpha_final.tolist()

        print("These are the final parameters:")
        print_params_combined(alpha_final)
        RspecAbs_result, _ = calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final, df_ws_segments, fe,step, nr_freqs)
        #RspecAbs_result_rad = RspecAbs_result / (2*np.pi)

        error = solution['fun']

        print(f"this is the final error of LBFGS: {error}")
        
        alpha_final.append(error)

    # write results to temporary dfs
        alpha_final_df = pd.DataFrame([alpha_final], columns=df_results_alpha.columns)

        df_results_alpha = alpha_final_df
        df_responseSpectrum_calculated = RspecAbs_result
    

        savetoCSVAllSegmentsCombined(df_results_alpha, f"alpha_segment_{segment}_fe_{nr_freqs}_mtd_NM_12_param_same_init_alpha_inc_d_IMU")
        savetoCSVAllSegmentsCombined(df_responseSpectrum_calculated, f'responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_NM_12_param_same_init_alpha_inc_d_IMU')
        if saveInitRS:
            savetoCSVAllSegmentsCombined(df_responseSpectrum_calculated_init, f'responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_same_init_alpha_12_params_inc_d_IMU')

        end_time_seg = time.time()
        print(f"Ran successfully. Execution time: {(end_time_seg - start_time_seg)/60} minutes")

        count += 1

    return

In [ ]:
def prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments):
    return f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments

In [ ]:

def run_simulation_multiple_segments_LBFGS_combined(alpha_init_combined, varbound_combined, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, param_options,saveInitRS = False):

    start_time = time.time()
    run_simulation_single_segment_for_all_segments_LBFGS_combined(constant_data, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, alpha_init_combined, varbound_combined,param_options, saveInitRS)

    # Capture the end time
    end_time = time.time()

    # Calculate and print the execution time
    print(f"Ran successfully the simulation with {nr_segments} compiled segments.")
    print(f"Execution time: {(end_time - start_time)/60} minutes")

    

In [ ]:
#nr_freqs = 600
#nr_segments = 2
step = 1
nr_freqs = 1000
start_seg = 79
stop_seg = 136
nr_segments = stop_seg - start_seg


gene_space = [{'low': float(bounds[0]), 'high': float(bounds[1])} for bounds in varbound_combined] # create continous voundaries


# Set options for the Nelder-Mead optimizer
algorithm_param_pygad_4 = {'max_num_iteration': 2, # 1000
                    'population_size': 150, # can try 150 
                    'mutation_num_genes': 1, # prev 0.3
                    'elit_ratio': 2, # prev 0.2
                    'crossover_probability': 0.7,
                    'num_parents_mating': 15, # prev 0.2
                    'crossover_type': 'uniform',
                    'max_iteration_without_improv': 150,
                    'gene_space' : gene_space   
                    }


constant_data = prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments)

run_simulation_multiple_segments_LBFGS_combined(alpha_init_combined, varbound_combined, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, algorithm_param_pygad_4, saveInitRS = False)

In [ ]:
def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  """
  # Define the paths to the files
  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_LBFGS_12_param_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_LBFGS_12_param_iter_{nr_run}.csv"
  responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"
  """
  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_NM_12_param_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_NM_12_param_iter_{nr_run}.csv"
  responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"

  # Read the DataFrames from the CSV files
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  responseSpectrum_calculated_current = pd.read_csv(responseSpectrum_calculated_path)
  responseSpectrum_calculated_init = pd.read_csv(responseSpectrum_calculated_init_path)

  return df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init


def readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, response, df_motions_segments):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq
  for segment in range(start_seg, stop_seg):
    df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init = readResults(segment, nr_run, nr_freqs, iterations)
    
    #title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_LBFGS'
    title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_NM'
    responseSpectrum_measured_curr_seg = responseSpectrum_measured[segment]

    responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs)

    #method = 'L-BFGS-B'
    method = 'NM'

    plotResponseSpectrumResult_same_segment_initial_CFE_ALL(segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current,responseSpectrum_calculated_init, title_file, method, save = True)

    if response != None:
      title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_resp_{response}_mtd_NM'
      #title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_resp_{response}_mtd_LBFGS'
      plotResponseSpectrumResult_single_response(response, segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current, responseSpectrum_calculated_init, title_file, method, save=True)

In [ ]:
start_seg = 0
stop_seg = 135
nr_run = 1
response = None

nr_segments = stop_seg - start_seg
step = 1
nr_freqs = 1000
readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, response, df_motions_segments)

In [ ]:
def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """

  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_NM_12_param_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_NM_12_param_iter_{nr_run}.csv"
  responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"

  # Read the DataFrames from the CSV files
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  responseSpectrum_calculated_current = pd.read_csv(responseSpectrum_calculated_path)
  responseSpectrum_calculated_init = pd.read_csv(responseSpectrum_calculated_init_path)

  return df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init


def readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, response, df_motions_segments):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq
  for segment in range(start_seg, stop_seg):
    df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init = readResults(segment, nr_run, nr_freqs, iterations)
    
    #title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_LBFGS'
    title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_NM'
    responseSpectrum_measured_curr_seg = responseSpectrum_measured[segment]
    ic(len(responseSpectrum_measured_curr_seg))
    responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs)

    #method = 'L-BFGS-B'
    method = 'NM'

    plotResponseSpectrumResult_same_segment_initial_CFE_ALL(segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current,responseSpectrum_calculated_init, title_file, method, save = True)

    if response != None:
      title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_resp_{response}_mtd_NM'
      #title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_resp_{response}_mtd_LBFGS'
      plotResponseSpectrumResult_single_response(response, segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current, responseSpectrum_calculated_init, title_file, method, save=True)

In [ ]:
start_seg = 0
stop_seg = 135
nr_run = 1
response = None

nr_segments = stop_seg - start_seg
step = 1
nr_freqs = 1000
readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, response, df_motions_segments)

# 15 parameters!!!


In [ ]:

def calculateResponsSpectrum_own_combined_all_resp(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs):

    RspecAbs = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])

    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']  # 1 value per segment. Heading [deg]
    U = df_speed_segments_avg[segment]['Speed']  # Speed Over Ground [m/s]

    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    beta_TRF_rad = np.deg2rad(beta_TRF)


    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)
    fe_sliced_rad = 2* np.pi* (fe_sliced)
    
    responses = ['Roll_rad','Heave',  'Pitch_rad']

    Nabla = None   # Volume Deplacement
    A_WP = None # Waterplane area

    C_B_1 = Nabla / (alpha[0] * alpha[1] * alpha[2])
    C_B_2 = Nabla / (alpha[4] * alpha[5] * alpha[6])
    C_B_3 = Nabla / (alpha[7] * alpha[8] * alpha[9])

    #alpha[3] = A_WP / (alpha[0] * alpha[1])

    sorted_indices_dirs = np.argsort(beta_TRF_rad)
    sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
    
    beta_TRF_sorted = np.sort(beta_TRF)
    beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

    for response in responses:
        
        #m0 =  trapezoid(responseSpectrum_measured_ground_truth[response],fe) # variance

        # Calculate the CFEs
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B_1) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[14]**2) * TRF_roll_tmp**2))
            TRF_2d = TRF_2d * alpha[3] # multiply by the gain
            
        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            #TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            #kappa = alpha[7] - alpha[9]
            kappa = alpha[10] - alpha[12]
            
            #TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_3,alpha[7],alpha[8],kappa,alpha[10],T_N=0)
            TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[7],alpha[8],alpha[9],C_B_3,alpha[10],alpha[11],kappa,alpha[13],T_N=0)
            TRF_roll_tmp = TRF_2d

        #PlotTRF(TRF_2d, freq_intrp_rad, response, beta_TRF)
        #TRF_2d = TRF_2d / (2 * np.pi) # convert the transfer function from rad/s to Hz

        # for every response: calculate the response spectrum
        #dx = beta_TRF_rad[1] - beta_TRF_rad[0]
        integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
        
        RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)
        RspecAbs[response] = RspecAbs_response

    RspecAbs['segment'] = segment

    return RspecAbs, responseSpectrum_measured_curr_seg_sliced

In [ ]:
def callback_generation(ga_instance):
    generation = ga_instance.generations_completed
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    average_fitness = np.mean([fitness for fitness in ga_instance.last_generation_fitness if fitness is not None])
    worst_fitness = min([fitness for fitness in ga_instance.last_generation_fitness if fitness is not None])
    diversity = len(set(ga_instance.last_generation_fitness))

    print(f"Generation = {generation}")
    print(f"Average Fitness = {average_fitness:.2f}")
    print(f"Best Fitness = {solution_fitness:.4f}")
    print(f"Worst Fitness = {worst_fitness:.2f}")
    print(f"Diversity (unique fitness values) = {diversity}")

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness

    error = 1/ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    #print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]))
    print(f"Error     = {error}")
    print("Change     = {change}".format(change=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1] - last_fitness))
    last_fitness = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]

def plotGA(ga_instance, segment, nr_freqs):

        #ga_instance.summary()
        print("plot fitness")
        ga_instance.plot_fitness()

        print("plot new solution rate")
        #ga_instance.plot_new_solution_rate()

        print("plot genes")
        #ga_instance.plot_genes(plot_type= "scatter", title = f"gene_plot_segment_{segment}_fe_{nr_freqs}",save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_scatter_segment_{segment}_fe_{nr_freqs}"))
        ga_instance.plot_genes(plot_type= "plot", title = f"gene_plot_segment_{segment}_fe_{nr_freqs}",save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_segment_{segment}_fe_{nr_freqs}"))
        # plot boxplot for every parameter
        #ga_instance.plot_genes(graph_type="boxplot", save_dir = (f"Plots/PYGAD/GeneticAlgorithm/gene_plot_boxplot_segment_{segment}_fe_{nr_freqs}"))


        # slightly implements some variation to the initial population so that not all individuals are the same
def generate_initial_population(base_solution, population_size, variation_range=0.1):
    initial_population = []
    for _ in range(population_size):
        varied_solution = [np.random.uniform(low=max(0, gene - abs(gene * variation_range)),
                                             high=gene + abs(gene * variation_range))
                           for gene in base_solution]
        initial_population.append(varied_solution)
    return initial_population


In [ ]:
def run_simulation_single_segment_for_all_segments_PYGAD_combined_all(constant_data, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, alpha_init_combined, varbound_combined, algorithm_param_pygad, saveInitRS = False):
    
    f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments = constant_data
    count = 0
    for segment in range(start_seg, stop_seg):
        start_time_seg = time.time()
        print(f"Current segment: {segment}")

        if count != 0:
            #print(f"these are the parameters of previous segment finalist alpha: {alpha}")
            alpha = alpha_final_pygad[:-1] # slice off the last parameter error.
        else:
            alpha = alpha_init_combined

        alpha = alpha_init_combined

        segment_data = prepare_segment_data(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)
        fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data
        beta_TRF_rad = np.deg2rad(beta_TRF)

        fe_sliced_rad = fe_sliced * 2 * np.pi

        #RspecAbs_result_init, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)
        RspecAbs_result_init, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_combined_all_resp(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)

        df_responseSpectrum_calculated_init = pd.DataFrame(RspecAbs_result_init, columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_responseSpectrum_calculated = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
        df_results_alpha = pd.DataFrame(columns=['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'L_3', 'B_3', 'T_3', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2', 'Error'])
        sorted_indices_dirs = np.argsort(beta_TRF_rad)
        sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
        beta_TRF_sorted = np.sort(beta_TRF)
        
        error = 0

        #print(f"these are the starting values of alpha: {alpha}")
        count = 0
        # Cost function which is to be minimised.
        def fitness_func(ga_instance, solution, solution_idx):
            nonlocal count
            responses = ['Roll_rad','Heave',  'Pitch_rad']
            alpha = solution
            #print(f"In cost function. alpha: {alpha}")
            m0 = {} # variance of response spectrum initialized
            CF = 0 # cost function initialized
            
            Nabla = None  # Volume Deplacement
            A_WP = None # Waterplane area

            C_B_1 = Nabla / (alpha[0] * alpha[1] * alpha[2])
            C_B_2 = Nabla / (alpha[4] * alpha[5] * alpha[6])
            C_B_3 = Nabla / (alpha[7] * alpha[8] * alpha[9])

            sorted_indices_dirs = np.argsort(beta_TRF_rad)
            sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
            beta_TRF_sorted = np.sort(beta_TRF)
            beta_TRF_rad_sorted = np.sort(beta_TRF_rad)
        
            for response in responses:
                
                m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response], fe_sliced) # variance of measured response spectrum. Note: is given in abs freq

                # Calculate the CFEs
                if response == "Heave":
                    # transfer function for Heave is in units [m/m]
                    TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B_1) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
                    TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[14]**2) * TRF_roll_tmp**2))
                    TRF_2d = TRF_2d * alpha[3] # multiply by the gai
                    
                if response == "Pitch_rad":
                    # transfer function for Pitch is in units [rad/m]
                    #TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)
                    TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[4],alpha[5],alpha[6],C_B_2) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

                if response == "Roll_rad":
                    kappa = alpha[10] - alpha[12]
                    TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[7],alpha[8],alpha[9],C_B_3,alpha[10],alpha[11],kappa,alpha[13],T_N=0)
                    TRF_roll_tmp = TRF_2d

                integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
                RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)

                calculated_spectrum = RspecAbs_response

                measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values

                # Difference between measured and calculated spectra
                expression = measured_spectrum - calculated_spectrum
                
                error = np.sqrt(trapezoid(np.abs(expression), fe_sliced) / m0)
                # Integrate the absolute value of the expression over 'fe'. Divide by m0
                CF += error
    
        
            return 1/CF
        # ---------------after iterating through cost function
        # generate init population for new alpha
        init_population = generate_initial_population(alpha, population_size = algorithm_param_pygad['population_size'], variation_range=0.1)

        ga_instance = pygad.GA(num_generations= algorithm_param_pygad['max_num_iteration'],
                               sol_per_pop= algorithm_param_pygad['population_size'],
                               num_parents_mating=algorithm_param_pygad['num_parents_mating'],
                               fitness_func = fitness_func,
                               num_genes=len(alpha),
                               gene_type=float,
                               gene_space=algorithm_param_pygad['gene_space'], # parent_selection_type="rws",
                               mutation_num_genes=algorithm_param_pygad['mutation_num_genes'], #when mutation_type = 'adaptive' has to be set to the following format: [60,30]
                               crossover_probability=algorithm_param_pygad['crossover_probability'],
                               mutation_type="random",
                               mutation_by_replacement=True, # to make sure genes do not go beyond range after mutation. prev true
                               crossover_type=algorithm_param_pygad['crossover_type'], 
                               on_generation=callback_generation,
                               save_solutions = True,
                               initial_population = init_population,
                               stop_criteria = [f"saturate_{algorithm_param_pygad['max_iteration_without_improv']}"],
                               keep_elitism = algorithm_param_pygad['elit_ratio']
                            )
        
        ga_instance.run()
        solution, solution_fitness, solution_idx  = ga_instance.best_solution()
        plotGA(ga_instance, segment, nr_freqs)

        alpha_final_pygad = solution
        alpha_final_pygad = alpha_final_pygad.tolist()
        error = 1/solution_fitness

        print("These are the final parameters after using GA for finding global optimum:")
        print_params_combined(alpha_final_pygad)
        print(f"this is the final error of GA: {error}")
        #RspecAbs_result, _ = calculateResponsSpectrum_own(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final, df_ws_segments, fe,step, nr_freqs)
        #RspecAbs_result, _ =  calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final_pygad, df_ws_segments, fe,step, nr_freqs)
        RspecAbs_result, _ =  calculateResponsSpectrum_own_combined_all_resp(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha_final_pygad, df_ws_segments, fe,step, nr_freqs)

        
        #RspecAbs_result_rad = RspecAbs_result / (2*np.pi)

        print(f"this was the final error of the cost function {error}")
        # Append results to df_results_alpha
        alpha_final_pygad.append(error)

    # write results to temporary dfs
        alpha_final_df = pd.DataFrame([alpha_final_pygad], columns=df_results_alpha.columns)

        df_results_alpha = alpha_final_df
        df_responseSpectrum_calculated = RspecAbs_result

        # Save results to CSV
        savetoCSVAllSegmentsCombined(df_results_alpha, f'alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{algorithm_param_pygad["max_num_iteration"]}_all_responses')
        savetoCSVAllSegmentsCombined(df_responseSpectrum_calculated, f'responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{algorithm_param_pygad["max_num_iteration"]}_all_responses')
        if saveInitRS:
            savetoCSVAllSegmentsCombined(df_responseSpectrum_calculated_init, f'responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_all_responses')

        end_time_seg = time.time()
        print(f"Ran successfully. Execution time: {(end_time_seg - start_time_seg)/60} minutes")

        count += 1

    return

In [ ]:

def run_simulation_multiple_segments_PYGAD_combined_all(alpha_init_combined, varbound_combined, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, param_options,saveInitRS = False):

    start_time = time.time()
    run_simulation_single_segment_for_all_segments_PYGAD_combined_all(constant_data, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, alpha_init_combined, varbound_combined,param_options, saveInitRS)

    # Capture the end time
    end_time = time.time()

    # Calculate and print the execution time
    print(f"Ran successfully the simulation with {nr_segments} compiled segments.")
    print(f"Execution time: {(end_time - start_time)/60} minutes")

    

In [ ]:
len(df_motions_segments)
def prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments):
    return f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments

In [ ]:
#nr_freqs = 600
#nr_segments = 2
#136 max segments. stop seg = 136
step = 1
nr_freqs = 1000
start_seg = 0
stop_seg = 136
nr_segments = stop_seg - start_seg


alpha_init_combined_all = [L, B, T, gain_heave, L, B, T, L, B, T, C_WP, GM_T, delta, tau, d_IMU]

varbound_combined_all = np.array([
[(1/2)*L, (3/2)*L],   # L_1 - Params for heave
[(1/2)*B, (3/2)*B],   # B_1
[(1/2)*T, (3/2)*T],   # T_1
[0.1, 7],     # gain heave,
[(1/2)*L, (3/2)*L],   # L_2 --  param for roll
[(1/4)*B, 2*B],   # B_2
[(1/4)*T, 2*T],   # T_2 -- 
[(1/2)*L, (3/2)*L],   # L_3 --  param for pitch
[(1/4)*B, 2*B],   # B_3
[(1/4)*T, 2.5*T],   # T_3 --  
[(1/2)*C_WP, (3/2)*C_WP],     # C_WP
[0.1, 2*B],   # GM_T
[0.1, 0.99],     # delta. if it is 1 a singularity occurs
[0.1, 1],     # tau
[5, 1/2 * L]    # d_IMU
])

gene_space = [{'low': float(bounds[0]), 'high': float(bounds[1])} for bounds in varbound_combined_all] # create continous voundaries

# Set options for the Nelder-Mead optimizer
algorithm_param_pygad_4 = {'max_num_iteration':300, # 1000
                    'population_size': 50, # can try 150 
                    'mutation_num_genes': 1, # prev 0.3
                    'elit_ratio': 2, # prev 0.2
                    'crossover_probability': 0.7,
                    'num_parents_mating': 15, # prev 0.2
                    'crossover_type': 'uniform',
                    'max_iteration_without_improv': 150, # prev 150
                    'gene_space' : gene_space   
                    }

random.seed(42)
np.random.seed(42)

constant_data = prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments)

run_simulation_multiple_segments_PYGAD_combined_all(alpha_init_combined_all, varbound_combined_all, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, algorithm_param_pygad_4, saveInitRS = False)

In [ ]:
def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  
  # Define the paths to the files
  
  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_all_responses_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_all_responses_iter_{nr_run}.csv"
  #responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"
  responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_all_responses_iter_1.csv"


  # Read the DataFrames from the CSV files
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  responseSpectrum_calculated_current = pd.read_csv(responseSpectrum_calculated_path)
  responseSpectrum_calculated_init = pd.read_csv(responseSpectrum_calculated_init_path)

  return df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init


def readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq
  for segment in range(start_seg, stop_seg):
    df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init = readResults(segment, nr_run, nr_freqs, iterations)
    
    title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_runs_{iterations}'

    responseSpectrum_measured_curr_seg = responseSpectrum_measured[segment]
    ic(len(responseSpectrum_measured_curr_seg))
    responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs)

    method = 'PyGAD'

    plotResponseSpectrumResult_same_segment_initial_CFE_ALL(segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current,responseSpectrum_calculated_init, title_file, method, save = True)

    if response != None:
      title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_resp_{response}_mtd_PYGAD_runs_{iterations}'
      plotResponseSpectrumResult_single_response(response, segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current, responseSpectrum_calculated_init, title_file, method, save=True)

In [ ]:
start_seg = 0
stop_seg = 136
nr_run = 1
response = None
iterations = 100 # max number of iterations

nr_segments = stop_seg - start_seg
step = 1
nr_freqs = 1000
readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments)

In [ ]:
len(df_motions_segments)
def prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments):
    return f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments

In [ ]:
#nr_freqs = 600
#nr_segments = 2
#136 max segments. stop seg = 136
step = 1
nr_freqs = 1000
start_seg = 0
stop_seg = 136
nr_segments = stop_seg - start_seg

alpha_init_combined_all = [L, B, T, gain_heave, L, B, T, L, B, T, C_WP, GM_T, delta, tau, d_IMU]

varbound_combined_all = np.array([
[(1/2)*L, (3/2)*L],   # L_1 - Params for heave
[(1/2)*B, (3/2)*B],   # B_1
[(1/2)*T, (3/2)*T],   # T_1
[0.1, 7],     # gain heave,
[(1/2)*L, (3/2)*L],   # L_2 --  param for roll
[(1/4)*B, 2*B],   # B_2
[(1/4)*T, 2*T],   # T_2 -- 
[(1/2)*L, (3/2)*L],   # L_3 --  param for pitch
[(1/4)*B, 2*B],   # B_3
[(1/4)*T, 2.5*T],   # T_3 --  
[(1/2)*C_WP, (3/2)*C_WP],     # C_WP
[0.1, 2*B],   # GM_T
[0.1, 0.99],     # delta. if it is 1 a singularity occurs
[0.1, 1],     # tau
[5, 1/2 * L]    # d_IMU
])

gene_space = [{'low': float(bounds[0]), 'high': float(bounds[1])} for bounds in varbound_combined_all] # create continous voundaries

# Set options for the Nelder-Mead optimizer
algorithm_param_pygad_4 = {'max_num_iteration':300, # 1000
                    'population_size': 50, # can try 150 
                    'mutation_num_genes': 1, # prev 0.3
                    'elit_ratio': 2, # prev 0.2
                    'crossover_probability': 0.7,
                    'num_parents_mating': 15, # prev 0.2
                    'crossover_type': 'uniform',
                    'max_iteration_without_improv': 150, # prev 150
                    'gene_space' : gene_space   
                    }

random.seed(42)
np.random.seed(42)

constant_data = prepare_constant_data(f0,fe,mu_deg,step,nr_freqs, start_seg,stop_seg, nr_segments)

run_simulation_multiple_segments_PYGAD_combined_all(alpha_init_combined_all, varbound_combined_all, constant_data, \
                                    df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  \
                                    responseSpectrum_measured, algorithm_param_pygad_4, saveInitRS = False)

In [ ]:
def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  
  # Define the paths to the files

  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_all_responses_iter_{nr_run}.csv"
  responseSpectrum_calculated_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_all_responses_iter_{nr_run}.csv"
  #responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_iter_1.csv"
  responseSpectrum_calculated_init_path = f"responseSpectrumResults_ALL_combined/responseSpectrum_calculated_alpha_init_segment_{segment}_fe_{nr_freqs}_all_responses_iter_1.csv"


  # Read the DataFrames from the CSV files
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  responseSpectrum_calculated_current = pd.read_csv(responseSpectrum_calculated_path)
  responseSpectrum_calculated_init = pd.read_csv(responseSpectrum_calculated_init_path)

  return df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init


def readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq
  for segment in range(start_seg, stop_seg):
    df_results_alpha_current, responseSpectrum_calculated_current, responseSpectrum_calculated_init = readResults(segment, nr_run, nr_freqs, iterations)
    
    title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_runs_{iterations}'

    responseSpectrum_measured_curr_seg = responseSpectrum_measured[segment]
    ic(len(responseSpectrum_measured_curr_seg))
    responseSpectrum_measured_curr_seg_sliced, fe_sliced = responseSpectrum_measured_slice(responseSpectrum_measured_curr_seg, fe, step, nr_freqs)
    ic(len(responseSpectrum_measured_curr_seg_sliced))
    ic(len(fe_sliced))

    method = 'PyGAD'

    plotResponseSpectrumResult_same_segment_initial_CFE_ALL(segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current,responseSpectrum_calculated_init, title_file, method, save = True)

    if response != None:
      title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_resp_{response}_mtd_PYGAD_runs_{iterations}'
      plotResponseSpectrumResult_single_response(response, segment, fe_sliced, responseSpectrum_measured_curr_seg_sliced, responseSpectrum_calculated_current, responseSpectrum_calculated_init, title_file, method, save=True)

In [ ]:
start_seg = 0
stop_seg = 136
nr_run = 1
response = None
iterations = 100 # max number of iterations

nr_segments = stop_seg - start_seg
step = 1
nr_freqs = 1000
readResultsAll(nr_segments,start_seg, stop_seg, nr_freqs, nr_run, iterations, response, df_motions_segments)

# Analysis of results - Statistics

In [ ]:

# simple mean method. "gjennomsnittet"
def simpleMean(df_results_alpha):
   df_results_alpha_excluding_last_column = df_results_alpha.iloc[:, :-1]
   
   return df_results_alpha_excluding_last_column.mean()

def weightedAvg(df_results_alpha, fmax = 2.4):
   # raphaels used 1.5. should look at the typical values of the CF
   param_sum_numerator = pd.Series([0] * (len(df_results_alpha.columns) - 1), index=df_results_alpha.columns[:-1])
   param_sum_denominator = 0

   weights = []

   segment_below_acc_fitness = []

   for segment, row in df_results_alpha.iterrows():
      if row['Error'] < fmax:
         weight_i = fmax - row['Error']
         weights.append(weight_i)
      else:
         weight_i = 0
         segment_below_acc_fitness.append(segment)
      
      #alpha['Weight'] = weight_i
      # update the sums for weighted avg calc
      param_sum_numerator += weight_i * row[:-1] # to slice off the last column: error
      param_sum_denominator += weight_i
   
   # Calculate weighted average if denominator is not zero
   if param_sum_denominator != 0:
      alpha_WA = param_sum_numerator / param_sum_denominator
   else:
      alpha_WA = pd.Series([None] * len(df_results_alpha.columns[:-1]), index=df_results_alpha.columns[:-1])

   return alpha_WA, segment_below_acc_fitness, weights


def weighted_std(df, weights):
    """
    Calculate the weighted standard deviation for each column in a DataFrame using
    the provided weights.

    Parameters:
        df (pd.DataFrame): DataFrame containing different responses.
        weights (list or pd.Series): Weights corresponding to each row in df.

    Returns:
        pd.Series: Weighted standard deviations for each column in df.
    """
    if len(weights) != len(df):
        raise ValueError("Length of weights must match the number of rows in DataFrame.")

    mean_weighted = np.average(df, weights=weights, axis=0)
    variance = np.average((df - mean_weighted)**2, weights=weights, axis=0)
    
    N = len(weights)  # Total number of weights assumed to be non-zero

    if N > 1:
        std_weighted = np.sqrt(N / (N - 1) * variance)
    else:
        std_weighted = np.zeros_like(variance)  # Avoid division by zero if N <= 1

    return pd.Series(std_weighted, index=df.columns)

def std_SM(df_results_alpha, alpha_SM):
    # Initialize a dictionary to store the standard deviations
    std_devs = {}
    
    # Calculate standard deviations
    for column in df_results_alpha.columns[:-1]:  # Exclude the 'Error' column
        mean = alpha_SM[column]
        squared_diff = (df_results_alpha[column] - mean) ** 2
        variance = squared_diff.mean()  # Using mean() to compute variance
        std_dev = np.sqrt(variance)
        
        std_devs[column] = std_dev

    return std_devs

def calculateResponseSpectrum_entire_run(alpha, df_ws_segments, nr_seg_compile):
    responseSpectrum_calculated = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])
    alpha = alpha.tolist()
    for segment in range(nr_seg_compile):
        timestamp, ws = list(df_ws_segments.items())[segment]
        responseSpectrum_calculated_tmp = calculateResponseSpectrum(alpha, ws, segment)

         # for first iteration: Create df
        if segment == 0:
            responseSpectrum_calculated = responseSpectrum_calculated_tmp

        # else: update existing df
        else:
            responseSpectrum_calculated = pd.concat([responseSpectrum_calculated, responseSpectrum_calculated_tmp], ignore_index=True)
    
    return responseSpectrum_calculated

def printStatsTable(data):
  init_mean, init_std, segment_specific_mean, segment_specific_std, SM_mean, SM_std, WA_mean, WA_std = data

# Printing the table similar to Table 5 format
  print("{:<20} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format("", "Mean [-]", "", "Standard Deviation [-]", "", "", ""))
  print("{:<20} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format("", "Vert.", "Roll", "Pitch", "Vert.", "Roll", "Pitch"))
  print("{:<20} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}".format("Actual geometry", init_mean['Heave'], init_mean['Roll_rad'], init_mean['Pitch_rad'], init_std['Heave'], init_std['Roll_rad'], init_std['Pitch_rad']))
  print("{:<20} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}".format("Segment-specific", segment_specific_mean['Heave'], segment_specific_mean['Roll_rad'], segment_specific_mean['Pitch_rad'], segment_specific_std['Heave'], segment_specific_std['Roll_rad'], segment_specific_std['Pitch_rad']))
  print("{:<20} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}".format("Simple mean", SM_mean['Heave'], SM_mean['Roll_rad'], SM_mean['Pitch_rad'], SM_std['Heave'], SM_std['Roll_rad'], SM_std['Pitch_rad']))
  print("{:<20} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}".format("Weighted avg.", WA_mean['Heave'], WA_mean['Roll_rad'], WA_mean['Pitch_rad'], WA_std['Heave'], WA_std['Roll_rad'], WA_std['Pitch_rad']))

def print_params_combined(param_values):
    param_names = ['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2']

    for name, value in zip(param_names, param_values):
        print(f"{name} = {value:.3f}")

### INclude staistics for all 12 params

In [ ]:
def calculateResponsSpectrum_own_init(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs):

    RspecAbs = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])

    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']  # 1 value per segment. Heading [deg]
    U = df_speed_segments_avg[segment]['Speed']  # Speed Over Ground [m/s]

    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    beta_TRF_rad = np.deg2rad(beta_TRF)


    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)
    fe_sliced_rad = 2* np.pi* (fe_sliced)
    
    responses = ['Roll_rad','Heave',  'Pitch_rad']

    
    A_WP = None # Waterplane area
    Nabla = None   # Volume Deplacement
    C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])
    #alpha[3] = A_WP / (alpha[0] * alpha[1])

    sorted_indices_dirs = np.argsort(beta_TRF_rad)
    sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
    beta_TRF_sorted = np.sort(beta_TRF)
    beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

    for response in responses:
        
        #m0 =  trapezoid(responseSpectrum_measured_ground_truth[response],fe) # variance

        # Calculate the CFEs
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(fe_sliced_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[7]**2) * TRF_roll_tmp**2))
            TRF_2d = TRF_2d * alpha[3]

        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF_2d = pitchCF(fe_sliced_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[3] - alpha[5]
            
            TRF_2d = rollCF(fe_sliced_rad,beta_TRF,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)
            TRF_roll_tmp = TRF_2d


        # for every response: calculate the response spectrum
        integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
        RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)
        RspecAbs[response] = RspecAbs_response

    RspecAbs['segment'] = segment

    return RspecAbs, responseSpectrum_measured_curr_seg_sliced

In [ ]:
def calculateErrorBetweenSpectra_combined(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data, alpha_is_series = False):
   df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

   #ic(type(df_results_alpha_current))
   if isinstance(df_results_alpha_current, pd.Series):
      #ic("df_results_alpha is a series")
      alpha = df_results_alpha_current.tolist() # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)
      #alpha = alpha[:-1] # don't want the included cost function error
      #ic(alpha)

   elif not isinstance(df_results_alpha_current, list):
      #ic("df_results_alpha is a pd df")
      alpha = df_results_alpha_current.values.tolist()[0] # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)

   else: # df_results_alpha_current is a list -> only alpha
      alpha = df_results_alpha_current
 

   error_list = []
   m0 = 0

   fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data


   responseSpectrum_calculated_tmp, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_combined(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)


   responses = ['Heave', 'Roll_rad', 'Pitch_rad']


   for response in responses:
      m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response],fe_sliced) # variance

      measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values
      calculated_spectrum = responseSpectrum_calculated_tmp[response]

      expression = measured_spectrum - calculated_spectrum
      error = trapezoid(np.abs(expression), fe_sliced) / m0
         
      # Integrate the absolute value of the expression over 'fe'. Divide by m0
      error_list.append(error)


   df_error = pd.DataFrame([error_list], columns=df_error.columns)


   # after iterating through all segments
   return df_error

def calculateErrorBetweenSpectra_combined_init(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data, alpha_is_series = False):
   df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

   #ic(type(df_results_alpha_current))
   if isinstance(df_results_alpha_current, pd.Series):
      #ic("df_results_alpha is a series")
      alpha = df_results_alpha_current.tolist() # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)
      #alpha = alpha[:-1] # don't want the included cost function error
      #ic(alpha)

   elif not isinstance(df_results_alpha_current, list):
      #ic("df_results_alpha is a pd df")
      alpha = df_results_alpha_current.values.tolist()[0] # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)

   else: # df_results_alpha_current is a list -> only alpha
      alpha = df_results_alpha_current
      #ic(alpha)

   error_list = []
   m0 = 0
   #timestamp, ws = list(df_ws_segments.items())[segment]
   fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data


   responseSpectrum_calculated_tmp, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_init(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)

   responses = ['Heave', 'Roll_rad', 'Pitch_rad']

   for response in responses:
      m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response],fe_sliced) # variance

      measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values
      calculated_spectrum = responseSpectrum_calculated_tmp[response]

      expression = measured_spectrum - calculated_spectrum
      error = trapezoid(np.abs(expression), fe_sliced) / m0
         
      # Integrate the absolute value of the expression over 'fe'. Divide by m0
      error_list.append(error)

   df_error = pd.DataFrame([error_list], columns=df_error.columns)

   return df_error
    
def prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False):
    # This function prepares the data required for each segment
    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']
    U = df_speed_segments_avg[segment]['Speed']

    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    
    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced  = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)


    return fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced

def printTableParams(alpha):
   # Iterate over the desired order and print the key-value pairs

   # Define the desired order of the keys
   desired_order = ['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2']
   alpha_list = alpha
   if (type(alpha) != list):
   # Extract the corresponding values from the Series
      alpha_list = [alpha[key] for key in desired_order if key in alpha]

   # Create a DataFrame with the desired order as the header and values as the second row
   df = pd.DataFrame([alpha_list], columns=desired_order)

   # Display the DataFrame as a table
   print(df)

def printTableParams_with_error(alpha):
   # Iterate over the desired order and print the key-value pairs

   # Define the desired order of the keys
   desired_order = ['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2', 'Error']
   alpha_list = alpha
   if (type(alpha) != list):
   # Extract the corresponding values from the Series
      alpha_list = [alpha[key] for key in desired_order if key in alpha]

   # Create a DataFrame with the desired order as the header and values as the second row
   df = pd.DataFrame([alpha_list], columns=desired_order)

   # Display the DataFrame as a table
   print(df)

def normalize_parameters(df, alpha_init_combined):
   """
   Normalizes specified parameters in the DataFrame based on actual physical values.

   Parameters:
      df (pd.DataFrame): DataFrame containing the data.
      params (list of tuples): Each tuple contains the column name to normalize and the actual value.
   """

   params_to_normalize = [
      ('L_1', alpha_init_combined[0]),
      ('B_1', alpha_init_combined[1]),
      ('T_1', alpha_init_combined[2]),
      ('L_2', alpha_init_combined[0]),
      ('B_2', alpha_init_combined[1]),
      ('T_2', alpha_init_combined[2])
      ]

   for param, actual in params_to_normalize:
      normalized_col_name = f'{param}_n'  # Generates new column name
      df[normalized_col_name] = df[param] / actual

   return df

def plot_tuning_parameters(df, parameters, alpha_init):
   """
   Plots boxplots for tuning parameters in a DataFrame.

   Parameters:
      df (pd.DataFrame): The DataFrame containing the data.
      parameters (list): A list of column names to plot.
   """

   # Set up the figure with subplots
   fig, axes = plt.subplots(nrows=len(parameters), ncols=1, figsize=(10, 2 * len(parameters)), constrained_layout=True)

   for i, param in enumerate(parameters):
      # Create boxplot for each parameter
      axes[i].boxplot(df[param].dropna(), vert=False, patch_artist=True, showfliers=True,
                     flierprops={'marker': 'o', 'color': 'black', 'markersize': 5},
                     boxprops={'facecolor': 'white', 'color': 'black'},
                     medianprops={'color': 'orange'},
                     whiskerprops={'color': 'black', 'linestyle': '--'},
                     capprops={'color': 'black'})
      
      # Adding titles and removing y ticks as we only have one box per plot
      axes[i].set_title(param)
      axes[i].set_yticks([])

   # Display the plot
   plt.show()

def plot_tuning_parameters_grouped(df, grouped_params):
   """
   Plots grouped boxplots for tuning parameters in a DataFrame. Each group of parameters
   will be plotted horizontally in each subfigure.

   Parameters:
      df (pd.DataFrame): The DataFrame containing the data.
      grouped_params (list of lists): A list where each sublist contains the names of three columns to plot together.
   """
   # Calculate the number of rows needed
   num_rows = len(grouped_params)
   
   # Set up the figure with subplots
   fig, axes = plt.subplots(nrows=num_rows, ncols=1, figsize=(15, 4 * num_rows), constrained_layout=True)

   # Check if there's only one row of subplots (axes will not be an array if only one subplot)
   if num_rows == 1:
      axes = [axes]  # Make it iterable

   # Translate parameters to LaTeX-friendly names
   latex_dict = {
      'delta': r'\delta',
      'tau': r'\tau'
   }

   for i, params in enumerate(grouped_params):
   # Apply LaTeX formatting to each parameter name
      formatted_params = []
      for param in params:
         if 'delta' in param:
               param = r'\delta'
         elif 'tau' in param:
               param = r'\tau'
         elif '_n' in param:
               param = param.replace('_n', '')
         elif '_' in param and '{' not in param and '}' not in param:
               parts = param.split('_')
               param = f'{parts[0]}_{{{parts[1]}}}'
         formatted_params.append(f'${param}$')


      label_fontsize = 16
      #formatted_params = [fr'${latex_dict.get(param, param)}$' for param in params]

      data_to_plot = [df[param].dropna() for param in params if param in df]
      # Check if the data list is not empty
      if data_to_plot:
         axes[i].boxplot(data_to_plot, vert=False, patch_artist=True, showfliers=True,
                           flierprops={'marker': 'o', 'color': 'black', 'markersize': 5},
                           boxprops={'facecolor': 'white', 'color': 'black'},
                           medianprops={'color': 'orange'},
                           whiskerprops={'color': 'black', 'linestyle': '--'},
                           capprops={'color': 'black'})
      
         # Set the y-tick labels to the parameter names, and ensure there's a title or label
         #formatted_params = [fr'${param}$' for param in params]
         #formatted_params = [param.replace('$delta$', '$\delta$').replace('$tau$', '$\tau$') for param in params]
         axes[i].set_yticklabels(formatted_params, fontsize=label_fontsize)
         axes[i].set_xlabel('[-]', fontsize=label_fontsize)
         directory = 'Plots/Statistics/Boxplots'
         fig.savefig(f"{directory}/Group_{i+1}.svg", format='svg', bbox_inches='tight')

   # Show the plot
   plt.show()

     

def plot_tuning_parameters_grouped(df, grouped_params, output_directory, label_fontsize=14):
   """
   Plots grouped boxplots for tuning parameters in a DataFrame. Each group of parameters
   will be plotted and saved individually with LaTeX formatting for all labels,
   handling double subscripts and normalized variable names.

   Parameters:
      df (pd.DataFrame): The DataFrame containing the data.
      grouped_params (list of lists): Each sublist contains names of three columns to plot together.
      output_directory (str): Directory where the plots will be saved.
      label_fontsize (int): Font size for x and y labels.
   """
   # Create the output directory if it doesn't exist
   if not os.path.exists(output_directory):
      os.makedirs(output_directory)

   for i, params in enumerate(grouped_params):
      # Create a new figure for each group
      fig, ax = plt.subplots(figsize=(8, 4))
      xlabel = '[-]'

      # Apply LaTeX formatting to each parameter name
      formatted_params = []
      for param in params:
            #original_param = param

         # Specific handling for parameters like 'd_IMU_2'
         if param.startswith('d_IMU_'):
            param = 'd_{IMU}'  # Remove numerical suffix and format correctly

         elif 'delta' in param:
               param = r'\delta'
         elif 'tau' in param:
               param = r'\tau'
         elif 'GM' in param:
               print("in GM")
               param = 'GM_T'
               xlabel = r'$[m]$'
         elif '_n' in param:
               param = param.replace('_n', '')
         elif '_' in param and '{' not in param and '}' not in param:
               parts = param.split('_')
               param = f'{parts[0]}_{{{parts[1]}}}'
         formatted_params.append(f'${param}$')

      # Prepare data to plot
      data_to_plot = [df[param].dropna() for param in params if param in df]
      #data_to_plot = [df[original_param].dropna() for original_param in params if original_param in df.columns]


      if data_to_plot:
         # Plotting the boxplot
         box = ax.boxplot(data_to_plot, vert=False, patch_artist=True, showfliers=True,
                           flierprops={'marker': 'o', 'color': 'black', 'markersize': 5},
                           boxprops={'facecolor': 'white', 'color': 'black'},
                           medianprops={'color': 'orange'},
                           whiskerprops={'color': 'black', 'linestyle': '--'},
                           capprops={'color': 'black'})
         # Setting custom y-tick labels
         ax.set_yticklabels(formatted_params, fontsize=label_fontsize)
         ax.set_xlabel(xlabel, fontsize=label_fontsize)

         # Save the individual figure
         fig.savefig(f"{output_directory}/Group_{i+1}.svg", format='svg', bbox_inches='tight')

         # Save the individual figure
       # Close the figure to free up memory
      plt.close(fig)
      #plt.show()
       


In [ ]:
# --------------------------------------------NEW CODE--------------------------------------------

def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """

  # 12 param solution inc d imu
  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_same_init_alpha_inc_d_IMU_iter_{nr_run}.csv"


  df_results_alpha_current = pd.read_csv(df_results_alpha_path)


  return df_results_alpha_current

def readResultsAll_including_statistics_combined(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init, alpha_init_8_param, fmax):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq

  df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_results_alpha = pd.DataFrame(columns=['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2', 'Error'])
 
  print(f"after defining df_results_alpha")

  print(df_results_alpha)
  for segment in range(start_seg, stop_seg):
    #ic(segment)

    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current  = readResults(segment, nr_run, nr_freqs, iterations)
    
    # calculates the error for each response for the best resulting parameters of each segment
    df_error_current = calculateErrorBetweenSpectra_combined(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)

    if segment == 0:
      df_error = pd.DataFrame(df_error_current, columns=df_error.columns)
      df_results_alpha = pd.DataFrame(df_results_alpha_current, columns=df_results_alpha.columns)
    else:
      df_error = pd.concat([df_error, df_error_current], ignore_index=True)
      df_results_alpha = pd.concat([df_results_alpha, df_results_alpha_current], ignore_index=True)
    
  # Single Mean
  alpha_SM = simpleMean(df_results_alpha)
  # Weighted Averaged. Is a dictionar
  alpha_WA, segments_below_acc_fitness, weights = weightedAvg(df_results_alpha, fmax = fmax)



  print("alpha Single Mean")
  printTableParams(alpha_SM)

  print("alpha Weighted average")
  printTableParams(alpha_WA)
  #nr_segments = stop_seg - start_seg
  #nr_nonzero_weights =  nr_segments - len(segments_below_acc_fitness)

  df_error_SM = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA_std = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_init = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

  for segment in range(start_seg, stop_seg):
    #ic(segment)
    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current  = readResults(segment, nr_run, nr_freqs, iterations)
    
    # calculates the error for each response for the best resulting parameters of each segment
    df_error_current_SM = calculateErrorBetweenSpectra_combined(alpha_SM, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_WA = calculateErrorBetweenSpectra_combined(alpha_WA, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_init = calculateErrorBetweenSpectra_combined_init(alpha_init_8_param, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data) # for calculating 

    ###
    if segment == 0:
      df_error_SM = pd.DataFrame(df_error_current_SM, columns=df_error_SM.columns)
      df_error_WA = pd.DataFrame(df_error_current_WA, columns=df_error_WA.columns)
      df_error_init = pd.DataFrame(df_error_current_init, columns=df_error_init.columns)

    else:
      df_error_SM = pd.concat([df_error_SM, df_error_current_SM], ignore_index=True)
      df_error_WA = pd.concat([df_error_WA, df_error_current_WA], ignore_index=True)
      df_error_init = pd.concat([df_error_init, df_error_current_init], ignore_index=True)
    ###

    if segment not in segments_below_acc_fitness:

      if df_error_WA_std.empty and df_error_WA_std.isna().all().all():
        df_error_WA_std = pd.DataFrame(df_error_current_WA, columns=df_error_WA_std.columns)

      else:
        df_error_WA_std = pd.concat([df_error_WA_std, df_error_current_WA], ignore_index=True)

  init_mean = df_error_init.mean()
  init_std = df_error_init.std()    

  segment_specific_tuning_mean = df_error.mean()
  segment_specific_tuning_std = df_error.std()

  SM_mean = df_error_SM.mean()
  SM_std = df_error_SM.std()

  WA_mean = df_error_WA.mean()

  WA_std = df_error_WA_std.std()
 

  WA_std_own_func = weighted_std(df_error_WA_std, weights)
  ic(WA_std_own_func)
  

  data = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std

  printStatsTable(data)

  data_w_own_std = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std_own_func
  printStatsTable(data_w_own_std)

  title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD'


  grouped_params = [
    ['L_1_n', 'B_1_n', 'T_1_n'],
    ['gain_heave', 'GM_T_2'],
    ['L_2_n', 'B_2_n', 'T_2_n'],
    ['C_WP_2', 'delta_2', 'tau_2'],
    # Add more groups as needed
  ]

  directory = 'Plots/Statistics/Boxplots'

  plot_tuning_parameters_grouped(df_results_alpha, grouped_params, directory, label_fontsize=16)

In [ ]:
alpha_init_8_param = [L, B, T, C_WP, GM_T, delta, tau, d_IMU] # parameters which are to be optimized
alpha_init_combined = [L, B, T, gain_heave, L, B, T, C_WP, GM_T, delta, tau, d_IMU]

nr_run = 1 # test that 23 and 24 is the same
step = 1
nr_freqs = 1000

start_seg = 0
stop_seg = 136
nr_segments = stop_seg - start_seg

#param_int = 4
iterations = 100

fmax = 2.0 # PYGAD 100 iters
#fmax = 2.35 #LBFGSB
#fmax = 2.35 #NM

readResultsAll_including_statistics_combined(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init_combined, alpha_init_8_param, fmax)

### 9 param

In [ ]:
def calculateResponsSpectrum_own_heave(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs):

    RspecAbs = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])

    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']  # 1 value per segment. Heading [deg]
    U = df_speed_segments_avg[segment]['Speed']  # Speed Over Ground [m/s]

    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    beta_TRF_rad = np.deg2rad(beta_TRF)

    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)
    fe_sliced_rad = 2* np.pi* (fe_sliced)
    
    responses = ['Roll_rad','Heave',  'Pitch_rad']

    Nabla = None   # Volume Deplacement
    A_WP = None # Waterplane area

    C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])
  

    sorted_indices_dirs = np.argsort(beta_TRF_rad)
    sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
    
    beta_TRF_sorted = np.sort(beta_TRF)

    beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

    for response in responses:
        
        #m0 =  trapezoid(responseSpectrum_measured_ground_truth[response],fe) # variance

        # Calculate the CFEs
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[7]**2) * TRF_roll_tmp**2))
            TRF_2d = TRF_2d * alpha[8] # multiply by the gain
            
        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[3] - alpha[5]
            
            TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)
            TRF_roll_tmp = TRF_2d

        #PlotTRF(TRF_2d, freq_intrp_rad, response, beta_TRF)
        #TRF_2d = TRF_2d / (2 * np.pi) # convert the transfer function from rad/s to Hz

        # for every response: calculate the response spectrum
        #dx = beta_TRF_rad[1] - beta_TRF_rad[0]
        integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
        
        RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)
        RspecAbs[response] = RspecAbs_response

    RspecAbs['segment'] = segment

    return RspecAbs, responseSpectrum_measured_curr_seg_sliced

In [ ]:
def calculateErrorBetweenSpectra_heave(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data, alpha_is_series = False):
   df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

   #ic(type(df_results_alpha_current))
   if isinstance(df_results_alpha_current, pd.Series):
      #ic("df_results_alpha is a series")
      alpha = df_results_alpha_current.tolist() # it is a nested list, but I want it in 1d, so choose the first segment
   

   elif not isinstance(df_results_alpha_current, list):
      #ic("df_results_alpha is a pd df")
      alpha = df_results_alpha_current.values.tolist()[0] # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)

   else: # df_results_alpha_current is a list -> only alpha
      alpha = df_results_alpha_current
      #ic(alpha)

   error_list = []
   m0 = 0

   fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data

   responseSpectrum_calculated_tmp, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_heave(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)
   
   responses = ['Heave', 'Roll_rad', 'Pitch_rad']

   for response in responses:
      m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response],fe_sliced) # variance

      measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values
      calculated_spectrum = responseSpectrum_calculated_tmp[response]

      expression = measured_spectrum - calculated_spectrum
      error = trapezoid(np.abs(expression), fe_sliced) / m0
         
      # Integrate the absolute value of the expression over 'fe'. Divide by m0
      error_list.append(error)
      
   df_error = pd.DataFrame([error_list], columns=df_error.columns)

   return df_error


def normalize_parameters(df, alpha_init_heave):
   """
   Normalizes specified parameters in the DataFrame based on actual physical values.

   Parameters:
      df (pd.DataFrame): DataFrame containing the data.
      params (list of tuples): Each tuple contains the column name to normalize and the actual value.
   """

   params_to_normalize = [
      ('L', alpha_init_heave[0]),
      ('B', alpha_init_heave[1]),
      ('T', alpha_init_heave[2]),
      ]

   for param, actual in params_to_normalize:
      normalized_col_name = f'{param}_n'  # Generates new column name
      df[normalized_col_name] = df[param] / actual

   return df

def plot_tuning_parameters_grouped(df, grouped_params, output_directory, label_fontsize=14):
   """
   Plots grouped boxplots for tuning parameters in a DataFrame. Each group of parameters
   will be plotted and saved individually with LaTeX formatting for all labels,
   handling double subscripts and normalized variable names.

   Parameters:
      df (pd.DataFrame): The DataFrame containing the data.
      grouped_params (list of lists): Each sublist contains names of three columns to plot together.
      output_directory (str): Directory where the plots will be saved.
      label_fontsize (int): Font size for x and y labels.
   """
   # Create the output directory if it doesn't exist
   if not os.path.exists(output_directory):
      os.makedirs(output_directory)

   for i, params in enumerate(grouped_params):
      # Create a new figure for each group
      fig, ax = plt.subplots(figsize=(8, 4))
      xlabel = '[-]'

      # Apply LaTeX formatting to each parameter name
      formatted_params = []
      for param in params:
            #original_param = param

         # Specific handling for parameters like 'd_IMU_2'
         if param.startswith('d_IMU_'):
            param = 'd_{IMU}'  # Remove numerical suffix and format correctly

         elif 'delta' in param:
               param = r'\delta'
         elif 'tau' in param:
               param = r'\tau'
         elif 'GM' in param:
               print("in GM")
               param = 'GM_T'
               xlabel = r'$[m]$'
         elif '_n' in param:
               param = param.replace('_n', '')
         elif '_' in param and '{' not in param and '}' not in param:
               parts = param.split('_')
               param = f'{parts[0]}_{{{parts[1]}}}'
         formatted_params.append(f'${param}$')

      # Prepare data to plot
      data_to_plot = [df[param].dropna() for param in params if param in df]
      #data_to_plot = [df[original_param].dropna() for original_param in params if original_param in df.columns]


      if data_to_plot:
         # Plotting the boxplot
         box = ax.boxplot(data_to_plot, vert=False, patch_artist=True, showfliers=True,
                           flierprops={'marker': 'o', 'color': 'black', 'markersize': 5},
                           boxprops={'facecolor': 'white', 'color': 'black'},
                           medianprops={'color': 'orange'},
                           whiskerprops={'color': 'black', 'linestyle': '--'},
                           capprops={'color': 'black'})
         # Setting custom y-tick labels
         ax.set_yticklabels(formatted_params, fontsize=label_fontsize)
         ax.set_xlabel(xlabel, fontsize=label_fontsize)

         # Save the individual figure
         fig.savefig(f"{output_directory}/Group_{i+1}.svg", format='svg', bbox_inches='tight')

         # Save the individual figure
       # Close the figure to free up memory
      plt.close(fig)
      #plt.show()
       

    

In [ ]:
# --------------------------------------------NEW CODE--------------------------------------------

def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  
  df_results_alpha_path = f"df_results_ALL/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_9_param_runs_{iterations}_inc_d_IMU_iter_{nr_run}.csv"
  
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)


  return df_results_alpha_current

def readResultsAll_including_statistics_heave(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init, alpha_init_8_param, fmax):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq

  df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_results_alpha = pd.DataFrame(columns=['L', 'B', 'T', 'C_WP', 'GM_T', 'delta', 'tau', 'd_IMU', 'gain_heave' ,'Error'])

  
  for segment in range(start_seg, stop_seg):
    #ic(segment)

    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current  = readResults(segment, nr_run, nr_freqs, iterations)
    
    # calculates the error for each response for the best resulting parameters of each segment
    df_error_current = calculateErrorBetweenSpectra_heave(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)

    if segment == 0:
      df_error = pd.DataFrame(df_error_current, columns=df_error.columns)
      df_results_alpha = pd.DataFrame(df_results_alpha_current, columns=df_results_alpha.columns)
    else:
      df_error = pd.concat([df_error, df_error_current], ignore_index=True)
      df_results_alpha = pd.concat([df_results_alpha, df_results_alpha_current], ignore_index=True)
    
  # Single Mean
  alpha_SM = simpleMean(df_results_alpha)
  # Weighted Averaged. Is a dictionar
  alpha_WA, segments_below_acc_fitness, weights = weightedAvg(df_results_alpha, fmax = fmax)

  df_error_SM = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA_std = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_init = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

  for segment in range(start_seg, stop_seg):
    #ic(segment)
    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current  = readResults(segment, nr_run, nr_freqs, iterations)
    
    # calculates the error for each response for the best resulting parameters of each segment
    df_error_current_SM = calculateErrorBetweenSpectra_heave(alpha_SM, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_WA = calculateErrorBetweenSpectra_heave(alpha_WA, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_init = calculateErrorBetweenSpectra_combined_init(alpha_init_8_param, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data) # for calculating 
    # this final works uses 8 param solution. 
    ###
    if segment == 0:
      df_error_SM = pd.DataFrame(df_error_current_SM, columns=df_error_SM.columns)
      df_error_WA = pd.DataFrame(df_error_current_WA, columns=df_error_WA.columns)
      df_error_init = pd.DataFrame(df_error_current_init, columns=df_error_init.columns)

    else:
      df_error_SM = pd.concat([df_error_SM, df_error_current_SM], ignore_index=True)
      df_error_WA = pd.concat([df_error_WA, df_error_current_WA], ignore_index=True)
      df_error_init = pd.concat([df_error_init, df_error_current_init], ignore_index=True)
    ###

    if segment not in segments_below_acc_fitness:

      if df_error_WA_std.empty and df_error_WA_std.isna().all().all():
        df_error_WA_std = pd.DataFrame(df_error_current_WA, columns=df_error_WA_std.columns)

      else:
        df_error_WA_std = pd.concat([df_error_WA_std, df_error_current_WA], ignore_index=True)

  init_mean = df_error_init.mean()
  init_std = df_error_init.std()    

  segment_specific_tuning_mean = df_error.mean()
  segment_specific_tuning_std = df_error.std()

  SM_mean = df_error_SM.mean()
  SM_std = df_error_SM.std()

  WA_mean = df_error_WA.mean()
  WA_std = df_error_WA_std.std()


  WA_std_own_func = weighted_std(df_error_WA_std, weights)
  ic(WA_std_own_func)
  

  data = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std

  printStatsTable(data)

  data_w_own_std = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std_own_func
  printStatsTable(data_w_own_std)

  title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD'

  df_results_alpha_norm = normalize_parameters(df_results_alpha, alpha_init)

  grouped_params = [
  ['T_n' ,'B_n','L_n'],
  [ 'GM_T', 'd_IMU'],
  ['gain_heave'],
  ['tau_2', 'delta_2', 'C_WP_2'],
]


  directory = 'Plots/Statistics/Boxplots'

  plot_tuning_parameters_grouped(df_results_alpha, grouped_params, directory, label_fontsize=16)


In [ ]:
alpha_init_8_param = [L, B, T, C_WP, GM_T, delta, tau, d_IMU] # parameters which are to be optimized
alpha_init_combined = [L, B, T, gain_heave, L, B, T, C_WP, GM_T, delta, tau, d_IMU]

alpha_init_heave = [L, B, T, C_WP, GM_T, delta, tau, d_IMU, gain_heave]

nr_run = 1 # test that 23 and 24 is the same
step = 1
nr_freqs = 1000

start_seg = 0
stop_seg = 136
nr_segments = stop_seg - start_seg

#param_int = 4
iterations = 100

fmax = 2.15 # PYGAD 100 iters


readResultsAll_including_statistics_heave(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init_heave, alpha_init_8_param, fmax)

#### code for 8 param inc d imu

In [ ]:
def calculateResponsSpectrum_own_8_param(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs):

    RspecAbs = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad', 'segment'])

    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']  # 1 value per segment. Heading [deg]
    U = df_speed_segments_avg[segment]['Speed']  # Speed Over Ground [m/s]

    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]
    beta_TRF_rad = np.deg2rad(beta_TRF)

    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)
    fe_sliced_rad = 2* np.pi* (fe_sliced)
    
    responses = ['Roll_rad','Heave',  'Pitch_rad']

    Nabla = None   # Volume Deplacement
    A_WP = None # Waterplane area

    C_B = Nabla / (alpha[0] * alpha[1] * alpha[2])
    #alpha[3] = A_WP / (alpha[0] * alpha[1])

    sorted_indices_dirs = np.argsort(beta_TRF_rad)
    sorted_ws_2d_beta_dirs = ws_2d_intrp[:, sorted_indices_dirs]
    
    beta_TRF_sorted = np.sort(beta_TRF)

    beta_TRF_rad_sorted = np.sort(beta_TRF_rad)

    for response in responses:
        
        #m0 =  trapezoid(responseSpectrum_measured_ground_truth[response],fe) # variance

        # Calculate the CFEs
        if response == "Heave":
            # transfer function for Heave is in units [m/m]
            TRF_2d = heaveCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # heaveCF(om0,beta_deg,U,L,B0,T,C_B=1)
            TRF_2d = np.sqrt(TRF_2d**2 + ((alpha[7]**2) * TRF_roll_tmp**2))
            
        if response == "Pitch_rad":
            # transfer function for Pitch is in units [rad/m]
            TRF_2d = pitchCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B) # pitchCF(om0,beta_deg,U,L,B0,T,C_B=1)

        if response == "Roll_rad":
            kappa = alpha[3] - alpha[5]
            
            TRF_2d = rollCF(fe_sliced_rad,beta_TRF_sorted,U,alpha[0],alpha[1],alpha[2],C_B,alpha[3],alpha[4],kappa,alpha[6],T_N=0)
            TRF_roll_tmp = TRF_2d

        #PlotTRF(TRF_2d, freq_intrp_rad, response, beta_TRF)
        #TRF_2d = TRF_2d / (2 * np.pi) # convert the transfer function from rad/s to Hz

        # for every response: calculate the response spectrum
        #dx = beta_TRF_rad[1] - beta_TRF_rad[0]
        integrand = np.abs(TRF_2d * TRF_2d) * sorted_ws_2d_beta_dirs
        
        RspecAbs_response = trapezoid(integrand, x = beta_TRF_rad_sorted, axis = 1)
        RspecAbs[response] = RspecAbs_response

    RspecAbs['segment'] = segment

    return RspecAbs, responseSpectrum_measured_curr_seg_sliced

In [ ]:
def calculateErrorBetweenSpectra_8_param(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data, alpha_is_series = False):
   df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

   #ic(type(df_results_alpha_current))
   if isinstance(df_results_alpha_current, pd.Series):
      #ic("df_results_alpha is a series")
      alpha = df_results_alpha_current.tolist() # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)
      #alpha = alpha[:-1] # don't want the included cost function error
      #ic(alpha)

   elif not isinstance(df_results_alpha_current, list):
      #ic("df_results_alpha is a pd df")
      alpha = df_results_alpha_current.values.tolist()[0] # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)

   else: # df_results_alpha_current is a list -> only alpha
      alpha = df_results_alpha_current
      #ic(alpha)

   error_list = []
   m0 = 0

   fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data


   responseSpectrum_calculated_tmp, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_8_param(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)


   responses = ['Heave', 'Roll_rad', 'Pitch_rad']

   for response in responses:
      m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response],fe_sliced) # variance

      measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values
      calculated_spectrum = responseSpectrum_calculated_tmp[response]

      expression = measured_spectrum - calculated_spectrum
      error = trapezoid(np.abs(expression), fe_sliced) / m0
         
      # Integrate the absolute value of the expression over 'fe'. Divide by m0
      error_list.append(error)


   df_error = pd.DataFrame([error_list], columns=df_error.columns)


   # after iterating through all segments
   return df_error


def normalize_parameters(df, alpha_init_heave):
   """
   Normalizes specified parameters in the DataFrame based on actual physical values.

   Parameters:
      df (pd.DataFrame): DataFrame containing the data.
      params (list of tuples): Each tuple contains the column name to normalize and the actual value.
   """

   params_to_normalize = [
      ('L', alpha_init_heave[0]),
      ('B', alpha_init_heave[1]),
      ('T', alpha_init_heave[2]),
      ]

   for param, actual in params_to_normalize:
      normalized_col_name = f'{param}_n'  # Generates new column name
      df[normalized_col_name] = df[param] / actual

   return df

def plot_tuning_parameters_grouped(df, grouped_params, output_directory, label_fontsize=14):
   """
   Plots grouped boxplots for tuning parameters in a DataFrame. Each group of parameters
   will be plotted and saved individually with LaTeX formatting for all labels,
   handling double subscripts and normalized variable names.

   Parameters:
      df (pd.DataFrame): The DataFrame containing the data.
      grouped_params (list of lists): Each sublist contains names of three columns to plot together.
      output_directory (str): Directory where the plots will be saved.
      label_fontsize (int): Font size for x and y labels.
   """
   # Create the output directory if it doesn't exist
   if not os.path.exists(output_directory):
      os.makedirs(output_directory)

   for i, params in enumerate(grouped_params):
      # Create a new figure for each group
      fig, ax = plt.subplots(figsize=(8, 4))
      xlabel = '[-]'

      # Apply LaTeX formatting to each parameter name
      formatted_params = []
      for param in params:
            #original_param = param

         # Specific handling for parameters like 'd_IMU_2'
         if param.startswith('d_IMU_'):
            param = 'd_{IMU}'  # Remove numerical suffix and format correctly

         elif 'delta' in param:
               param = r'\delta'
         elif 'tau' in param:
               param = r'\tau'
         elif 'GM' in param:
               print("in GM")
               param = 'GM_T'
               xlabel = r'$[m]$'
         elif '_n' in param:
               param = param.replace('_n', '')
         elif '_' in param and '{' not in param and '}' not in param:
               parts = param.split('_')
               param = f'{parts[0]}_{{{parts[1]}}}'
         formatted_params.append(f'${param}$')

      # Prepare data to plot
      data_to_plot = [df[param].dropna() for param in params if param in df]
      #data_to_plot = [df[original_param].dropna() for original_param in params if original_param in df.columns]


      if data_to_plot:
         # Plotting the boxplot
         box = ax.boxplot(data_to_plot, vert=False, patch_artist=True, showfliers=True,
                           flierprops={'marker': 'o', 'color': 'black', 'markersize': 5},
                           boxprops={'facecolor': 'white', 'color': 'black'},
                           medianprops={'color': 'orange'},
                           whiskerprops={'color': 'black', 'linestyle': '--'},
                           capprops={'color': 'black'})
         # Setting custom y-tick labels
         ax.set_yticklabels(formatted_params, fontsize=label_fontsize)
         ax.set_xlabel(xlabel, fontsize=label_fontsize)

         # Save the individual figure
         fig.savefig(f"{output_directory}/Group_{i+1}.svg", format='svg', bbox_inches='tight')

         # Save the individual figure
       # Close the figure to free up memory
      plt.close(fig)
      #plt.show()
       

    

In [ ]:
# --------------------------------------------NEW CODE--------------------------------------------

def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """
  
  df_results_alpha_path = f"df_results_ALL/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_runs_{iterations}_same_init_alpha_8_param_inc_d_IMU_iter_{nr_run}.csv"
 
  df_results_alpha_current = pd.read_csv(df_results_alpha_path)


  return df_results_alpha_current

def readResultsAll_including_statistics_8_param(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init, alpha_init_8_param, fmax):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq

  df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

  df_results_alpha = pd.DataFrame(columns=['L', 'B', 'T', 'C_WP', 'GM_T', 'delta', 'tau', 'd_IMU', 'Error'])

  
  for segment in range(start_seg, stop_seg):
    #ic(segment)

    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current  = readResults(segment, nr_run, nr_freqs, iterations)
    
    # calculates the error for each response for the best resulting parameters of each segment
    df_error_current = calculateErrorBetweenSpectra_8_param(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)

    if segment == 0:
      df_error = pd.DataFrame(df_error_current, columns=df_error.columns)
      df_results_alpha = pd.DataFrame(df_results_alpha_current, columns=df_results_alpha.columns)
    else:
      df_error = pd.concat([df_error, df_error_current], ignore_index=True)
      df_results_alpha = pd.concat([df_results_alpha, df_results_alpha_current], ignore_index=True)
    
  # Single Mean
  alpha_SM = simpleMean(df_results_alpha)
  # Weighted Averaged. Is a dictionar
  alpha_WA, segments_below_acc_fitness, weights = weightedAvg(df_results_alpha, fmax = fmax)
  
  ic(len(segments_below_acc_fitness))
  ic(segments_below_acc_fitness)
  ic(len(weights))


  df_error_SM = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA_std = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_init = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

  for segment in range(start_seg, stop_seg):
    #ic(segment)
    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current  = readResults(segment, nr_run, nr_freqs, iterations)
    
    # calculates the error for each response for the best resulting parameters of each segment
    df_error_current_SM = calculateErrorBetweenSpectra_8_param(alpha_SM, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_WA = calculateErrorBetweenSpectra_8_param(alpha_WA, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_init = calculateErrorBetweenSpectra_combined_init(alpha_init_8_param, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data) # for calculating 
    # this final works uses 8 param solution. 
    ###
    if segment == 0:
      df_error_SM = pd.DataFrame(df_error_current_SM, columns=df_error_SM.columns)
      df_error_WA = pd.DataFrame(df_error_current_WA, columns=df_error_WA.columns)
      df_error_init = pd.DataFrame(df_error_current_init, columns=df_error_init.columns)

    else:
      df_error_SM = pd.concat([df_error_SM, df_error_current_SM], ignore_index=True)
      df_error_WA = pd.concat([df_error_WA, df_error_current_WA], ignore_index=True)
      df_error_init = pd.concat([df_error_init, df_error_current_init], ignore_index=True)
    ###

    if segment not in segments_below_acc_fitness:

      if df_error_WA_std.empty and df_error_WA_std.isna().all().all():
        df_error_WA_std = pd.DataFrame(df_error_current_WA, columns=df_error_WA_std.columns)

      else:
        df_error_WA_std = pd.concat([df_error_WA_std, df_error_current_WA], ignore_index=True)

  init_mean = df_error_init.mean()
  init_std = df_error_init.std()    

  segment_specific_tuning_mean = df_error.mean()
  segment_specific_tuning_std = df_error.std()

  SM_mean = df_error_SM.mean()
  SM_std = df_error_SM.std()

  WA_mean = df_error_WA.mean()
  WA_std = df_error_WA_std.std()
  
  WA_std_own_func = weighted_std(df_error_WA_std, weights)
  ic(WA_std_own_func)
  

  data = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std

  printStatsTable(data)

  data_w_own_std = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std_own_func
  printStatsTable(data_w_own_std)

  title_file = f'After_optm_results_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD'

  df_results_alpha_norm = normalize_parameters(df_results_alpha, alpha_init)

  grouped_params = [
  ['T_n' ,'B_n','L_n'],
  [ 'GM_T', 'd_IMU'],
  ['tau_2', 'delta_2', 'C_WP_2'],
]



  directory = 'Plots/Statistics/Boxplots'

  plot_tuning_parameters_grouped(df_results_alpha, grouped_params, directory, label_fontsize=16)

In [ ]:


alpha_init_8_param = [L, B, T, C_WP, GM_T, delta, tau, d_IMU] # parameters which are to be optimized
alpha_init_combined = [L, B, T, gain_heave, L, B, T, C_WP, GM_T, delta, tau, d_IMU]

alpha_init_heave = [L, B, T, C_WP, GM_T, delta, tau, d_IMU, gain_heave]

nr_run = 1 # test that 23 and 24 is the same
step = 1
nr_freqs = 1000

start_seg = 0
stop_seg = 136
nr_segments = stop_seg - start_seg

#param_int = 4
iterations = 100

fmax = 2.25 # PYGAD 100 iters


readResultsAll_including_statistics_8_param(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init_8_param, alpha_init_8_param, fmax)

### 15 param statistics


In [ ]:
def calculateErrorBetweenSpectra_combined_15_param(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data, alpha_is_series = False):
   df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

   #ic(type(df_results_alpha_current))
   if isinstance(df_results_alpha_current, pd.Series):
      #ic("df_results_alpha is a series")
      alpha = df_results_alpha_current.tolist() # it is a nested list, but I want it in 1d, so choose the first segment
      #alpha = alpha[:-1] # don't want the included cost function error
      #ic(alpha)

   elif not isinstance(df_results_alpha_current, list):
      #ic("df_results_alpha is a pd df")
      alpha = df_results_alpha_current.values.tolist()[0] # it is a nested list, but I want it in 1d, so choose the first segment
      #ic(alpha)

   else: # df_results_alpha_current is a list -> only alpha
      alpha = df_results_alpha_current
      #ic(alpha)

   error_list = []
   m0 = 0

   fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced = segment_data


   responseSpectrum_calculated_tmp, responseSpectrum_measured_curr_seg_sliced = calculateResponsSpectrum_own_combined_all_resp(segment, mu_deg, f0, df_pos_segments_avg, df_speed_segments_avg, alpha, df_ws_segments, fe,step, nr_freqs)


   responses = ['Heave', 'Roll_rad', 'Pitch_rad']

   for response in responses:
      m0 =  trapezoid(responseSpectrum_measured_curr_seg_sliced[response],fe_sliced) # variance

      measured_spectrum = responseSpectrum_measured_curr_seg_sliced[response].values
      calculated_spectrum = responseSpectrum_calculated_tmp[response]

      expression = measured_spectrum - calculated_spectrum
      error = trapezoid(np.abs(expression), fe_sliced) / m0
         
      # Integrate the absolute value of the expression over 'fe'. Divide by m0
      error_list.append(error)

   df_error = pd.DataFrame([error_list], columns=df_error.columns)
   return df_error


    
def prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False):
    # This function prepares the data required for each segment
    timestamp, ws_2d_mu = list(df_ws_segments.items())[segment]
    yaw = df_pos_segments_avg[segment]['hdt']
    U = df_speed_segments_avg[segment]['Speed']



    beta_TRF = mu_deg - yaw # Might have to use re_range() here to avoid negative angles
    beta_TRF = re_range(beta_TRF) # ensure that beta_TRF is within the range of [0, 360]

    responseSpectrum_measured_curr_seg_sliced, ws_2d_intrp, fe_sliced  = interpolateFreq(f0, fe, segment, timestamp,  responseSpectrum_measured, ws_2d_mu, beta_TRF, mu_deg, step, nr_freqs, plot = False)


    return fe_sliced, f0, ws_2d_intrp, beta_TRF, fe, U, responseSpectrum_measured_curr_seg_sliced

def printTableParams(alpha):

   desired_order = ['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'L_3', 'B_3', 'T_3', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2']
   alpha_list = alpha
   if (type(alpha) != list):
   # Extract the corresponding values from the Series
      alpha_list = [alpha[key] for key in desired_order if key in alpha]

   # Create a DataFrame with the desired order as the header and values as the second row
   df = pd.DataFrame([alpha_list], columns=desired_order)

   # Display the DataFrame as a table
   print(df)



In [ ]:
# --------------------------------------------NEW CODE--------------------------------------------

def readResults(segment, nr_run, nr_freqs, iterations):
  """
  Reads the results from the CSV files based on the iteration number provided.

  :param nr_run: The iteration number of the results to read.
  :return: A tuple of DataFrames containing results from specific folders.
  """

  
  df_results_alpha_path = f"df_results_ALL_combined/alpha_segment_{segment}_fe_{nr_freqs}_mtd_PYGAD_12_param_runs_{iterations}_all_responses_iter_{nr_run}.csv"

  df_results_alpha_current = pd.read_csv(df_results_alpha_path)
  

  return df_results_alpha_current

def readResultsAll_including_statistics_combined(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init, alpha_init_8_param, fmax):
  nperseg = 2048
  fe, responseSpectrum_measured = getListResponseSpectrum(df_motions_segments, nperseg,plot = None, save = False) # this is given in encounter freq

  df_error = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])


  df_results_alpha = pd.DataFrame(columns=['L_1', 'B_1', 'T_1', 'gain_heave', 'L_2', 'B_2', 'T_2', 'L_3', 'B_3', 'T_3', 'C_WP_2', 'GM_T_2', 'delta_2', 'tau_2', 'd_IMU_2', 'Error'])
  
  for segment in range(start_seg, stop_seg):
    #ic(segment)

    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current = readResults(segment, nr_run, nr_freqs, iterations)
    
    # calculates the error for each response for the best resulting parameters of each segment
    df_error_current = calculateErrorBetweenSpectra_combined_15_param(df_results_alpha_current, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)

    if segment == 0:
      df_error = pd.DataFrame(df_error_current, columns=df_error.columns)
      df_results_alpha = pd.DataFrame(df_results_alpha_current, columns=df_results_alpha.columns)
    else:
      df_error = pd.concat([df_error, df_error_current], ignore_index=True)
      df_results_alpha = pd.concat([df_results_alpha, df_results_alpha_current], ignore_index=True)
    
  # Single Mean
  alpha_SM = simpleMean(df_results_alpha)
  # Weighted Averaged. Is a dictionar
  alpha_WA, segments_below_acc_fitness, weights = weightedAvg(df_results_alpha, fmax = fmax)
  ic(len(segments_below_acc_fitness))
  ic(segments_below_acc_fitness)


  print("alpha Single Mean")
  printTableParams(alpha_SM)

  print("alpha Weighted average")
  printTableParams(alpha_WA)
  #nr_segments = stop_seg - start_seg
  #nr_nonzero_weights =  nr_segments - len(segments_below_acc_fitness)

  df_error_SM = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_WA_std = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])
  df_error_init = pd.DataFrame(columns=['Heave', 'Roll_rad', 'Pitch_rad'])

  for segment in range(start_seg, stop_seg):
    #ic(segment)
    segment_data = prepare_segment_data_no_plot(f0, fe, mu_deg, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg, responseSpectrum_measured, step, nr_freqs, savePolar = False)

    df_results_alpha_current = readResults(segment, nr_run, nr_freqs, iterations)
  
    
    # calculates the error for each response for the best resulting parameters of each segment
    #calculateErrorBetweenSpectra_combined_15_param
    df_error_current_SM = calculateErrorBetweenSpectra_combined_15_param(alpha_SM, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_WA = calculateErrorBetweenSpectra_combined_15_param(alpha_WA, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data)
    df_error_current_init = calculateErrorBetweenSpectra_combined_init(alpha_init_8_param, segment, df_ws_segments, df_pos_segments_avg, df_speed_segments_avg,  mu_deg, fe, segment_data) # for calculating 

    ###
    if segment == 0:
      df_error_SM = pd.DataFrame(df_error_current_SM, columns=df_error_SM.columns)
      df_error_WA = pd.DataFrame(df_error_current_WA, columns=df_error_WA.columns)
      df_error_init = pd.DataFrame(df_error_current_init, columns=df_error_init.columns)

    else:
      df_error_SM = pd.concat([df_error_SM, df_error_current_SM], ignore_index=True)
      df_error_WA = pd.concat([df_error_WA, df_error_current_WA], ignore_index=True)
      df_error_init = pd.concat([df_error_init, df_error_current_init], ignore_index=True)
    ###

    if segment not in segments_below_acc_fitness:

      if df_error_WA_std.empty and df_error_WA_std.isna().all().all():
        df_error_WA_std = pd.DataFrame(df_error_current_WA, columns=df_error_WA_std.columns)

      else:
        df_error_WA_std = pd.concat([df_error_WA_std, df_error_current_WA], ignore_index=True)

  init_mean = df_error_init.mean()
  init_std = df_error_init.std()    

  segment_specific_tuning_mean = df_error.mean()
  segment_specific_tuning_std = df_error.std()

  SM_mean = df_error_SM.mean()
  SM_std = df_error_SM.std()

  WA_mean = df_error_WA.mean()
  WA_std = df_error_WA_std.std()
  


  WA_std_own_func = weighted_std(df_error_WA_std, weights)
  ic(WA_std_own_func)


  data = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std

  #printStatsTable(data)

  data_w_own_std = init_mean, init_std, segment_specific_tuning_mean, segment_specific_tuning_std, SM_mean, SM_std, WA_mean, WA_std_own_func
  printStatsTable(data_w_own_std)



In [ ]:


alpha_init_8_param = [L, B, T, C_WP, GM_T, delta, tau, d_IMU] # parameters which are to be optimized


alpha_init_combined = [L, B, T, gain_heave, L, B, T, C_WP, GM_T, delta, tau, d_IMU]

alpha_init_combined_all = [L, B, T, gain_heave, L, B, T, L, B, T, C_WP, GM_T, delta, tau, d_IMU]

varbound_combined_all = np.array([
[(1/2)*L, (3/2)*L],   # L_1 - Params for heave
[(1/2)*B, (3/2)*B],   # B_1
[(1/2)*T, (3/2)*T],   # T_1
[0.1, 7],     # gain heave,
[(1/2)*L, (3/2)*L],   # L_2 --  param for roll
[(1/4)*B, 2*B],   # B_2
[(1/4)*T, 2*T],   # T_2 -- 
[(1/2)*L, (3/2)*L],   # L_3 --  param for pitch
[(1/4)*B, 2*B],   # B_3
[(1/4)*T, 2.5*T],   # T_3 --  
[(1/2)*C_WP, (3/2)*C_WP],     # C_WP
[0.1, 2*B],   # GM_T
[0.1, 0.99],     # delta. if it is 1 a singularity occurs
[0.1, 1],     # tau
[5, 1/2 * L]    # d_IMU
])

nr_run = 1 # test that 23 and 24 is the same
step = 1
nr_freqs = 1000

start_seg = 0
stop_seg = 136
nr_segments = stop_seg - start_seg

#param_int = 4
iterations = 200

#fmax = 2.3 # PYGAD 100 iters
#fmax = 1.85 # PYGAD 150 iters
#fmax = 2.2 # PYGAD 200 iters
fmax =  2.25# PYGAD 300 iters

#fmax = 2.3 #LBFGSB
#fmax = 2.3 #NM

readResultsAll_including_statistics_combined(iterations,start_seg, stop_seg, nr_freqs, nr_run, df_motions_segments, alpha_init_combined_all, alpha_init_8_param, fmax)


### Misc plots

In [ ]:
def printSeaPeak(tp):
        print(f"peak period: {tp}")
        fp = 1/tp
        print(f"peak frequency: {fp}")
        return fp

def plotWaveSpectra(df_ws_segments, segment, mu_rad, om0, save = False):
        timestamp, ws = list(df_ws_segments.items())[segment]
        print(f" the wave spectra is calculated at timestamp : {timestamp}")
        
        #print(f"this is segment array after masking freq: {segment_array}")
        #responseSpectrum_calculated = responseSpectrum_calculated[segment].loc[mask]
        title_fig = f'Wave spectra for segment: {segment} at time: {timestamp}'       
        
        linewidth_measured = 2
        measured_color = 'g'      # green
        # A bright red for error
        measured_marker = 's'     # Square marker

        mu_rad_strings = ['{:.2f}'.format(x) for x in mu_rad]
        mu_deg = (mu_rad/np.pi) * 180 
        mu_deg_strings = ['{:.2f}'.format(x) for x in mu_deg]
        dir_len = len(mu_rad)
        dir_indeces = np.arange(dir_len)

        print(f"this is dir_indeces length : {dir_len} and value: {dir_indeces}")
        print(f"this is the type of om0 : {type(om0)}")
        print(f"this is omega0 length {len(om0)} and value : {om0}")

        for index in range(dir_len):
                ws_per_dir = ws[: , index]
                direction_string = mu_rad_strings[index]
                direction_string_deg = mu_deg_strings[index]
                plt.plot(om0, ws_per_dir)
                
                # Set titles and labels
                plt.title(f"Wave spectra for direction: {direction_string} rad or {direction_string_deg} deg. Time: {timestamp}")
                plt.xlabel(r"$Frequency [Hz]$")
                plt.ylabel("[m^2 *s/rad]")

                # Add legend
                #"ax.legend()"
                plt.grid(True)
                title_file = f'seg_{segment}_WaveSpectra_dir_{index}'

                if (save):
                        folder_path = 'Plots/WaveSpectra'
                        saveSVG(title_file, folder_path)

                # Adjust layout for a cleaner look
                plt.tight_layout()

                plt.show()

segment = 0

print("for swell")
tp_swell = 6.934275

fp_swell = printSeaPeak(tp_swell)

print("for sea")
tp_sea = 3.23489

fp_sea = printSeaPeak(tp_sea)

plotWaveSpectra(df_ws_segments, segment, mu_rad, om0, save = False)

